# Initialization

In [8]:
# IMPORTS
import warnings
warnings.filterwarnings("ignore")

import azure.functions as func
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

#from config import * 
from datetime import datetime, timedelta
from newspaper import Article
# from newspaper import Config as npConfig
import pandas as pd
import numpy as np 
import re
import numbers

import logging
logging.getLogger('azure').propagate = False
logging.getLogger('urllib3').propagate = False
logging.getLogger('scrapy').propagate = False

In [9]:
# CONFIG

# Search Word
searchword = 'LCH SA'
#searchword = 'EUREX'
cpt_dict = {
    'LCH SA':['BNP Paribas', 'Citigroup'],# 'Nomura', 
              #'Morgan Stanley', 'Deutsche Bank'],
    'EUREX':['HSBC', 'Morgan Stanley']#, 'Dankse Bank'], 
             #'Nomura', 'BNP Paribas']
    }

# Search time 
searchdate = datetime.today().strftime('%m/%d/%Y') 
search_days = 7  

# Twitter Keys
api_key = 'Oc9II7RV8McqVCXjU9MybOvnP' 
api_secret_key = 'KNGE1JX1xIKbwUoucdr2s3ZBwQBY6SYBAHY3qEpGWreQANaNZR'
access_token = '1347130231908421632-Wqvur6Qc1Emo3uj1U2Meyz2uJ6X45R'
access_token_secret = 'zZ7NEnJh7F3LnPUBjTjkhCUJhFpCuGndcmw2rGvUWrGSL'

# Azure Cognitive Service Keys
cog_endpoint = 'https://eocjmcognitiveservicesconnection.cognitiveservices.azure.com/'
cog_key = '1b79872142ca4bf482381e946bbb6dc2'

# DF columns and path
columns = ['API', 'Link', 'Date', 'Title', 'Article', 'Summary', 
               'Title_Sentiment', 'Title_Positv', 
               'Title_Neutral', 'Title_Negativ',
               'Article_Sentiment', 'Article_Positv', 
               'Article_Neutral', 'Article_Negativ'
              ] 
root_path = r'C:\Users\Johannes\Documents\Data Science\EOC_Newsscraper\eoc_newsscraper_'
file_format = '.csv'

In [10]:
# FUNCTIONS

# Message and DF 
def get_init_message(searchword):
    """"""
    # Date 
    date = datetime.today().strftime('%Y-%m-%d')
    time = datetime.today().strftime('%H:%M')
    message = date+' '+time +'\n'
    
    message += 'WebScraper: ' + searchword+ '\n\n'
    
    return message 
def get_init_df():
    """"""
    return pd.DataFrame(columns=columns)
def save_csv(dff, searchword):
    """"""
    date = datetime.today().strftime('%Y%m%d')
    path = root_path +  searchword+'_'+ date + file_format
    dff.to_csv(path, index=False)

# Azure     
def authenticate_azure_textanalytics_client():
    ta_credential = AzureKeyCredential(cog_key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=cog_endpoint, 
            credential=ta_credential)
    return text_analytics_client
def getAzureSentiment(text, client):
    """"""
    documents = [text[:5120]] # limit to 5120 characters. This is the azure data limit for text analytics
    response = client.analyze_sentiment(documents = documents)[0]
    document_sentiment = response.sentiment
    score_positive = response.confidence_scores.positive
    score_neutral = response.confidence_scores.neutral
    score_negative = response.confidence_scores.negative
    return document_sentiment, score_positive, score_neutral, score_negative
def get_newssearch_client():
    """"""
    # Import Libraries
    from azure.cognitiveservices.search.newssearch import NewsSearchClient
    from msrest.authentication import CognitiveServicesCredentials
    
    # Credentials
    subscription_key = "5521444b482f4dd6a800e85fdfb1c636"
    ENDPOINT = "https://api.bing.microsoft.com/v7.0/search?q=microsoft+devices"
    
    # Client
    bing_client = NewsSearchClient(
            endpoint=ENDPOINT,
            credentials=CognitiveServicesCredentials(subscription_key)
        )
    
    return bing_client

# Add news to df
def articles_to_df(urls, dff, api, article_summary=None):
    """"""
    
    # Authenticate connection to azure text analytics service
    client = authenticate_azure_textanalytics_client()

    for url in urls:
        # Initialize 
        title, text, summary='','',''
        date = ''
        url_api = api
        try:
            # Download Article via newspaper3k
            article = Article(url)
            article.download()
            article.parse()
            article.nlp()

            title = article.title
            text = article.text
            summary = article.summary
            date = article.publish_date
            # If newspaper3k did not fail but did not download anything in fact
            if(title==''):
                raise 
        except:
            if(article_summary==None):
                print('Article of '+api+' could not be fetched: ' + url)
                continue
            else:
                # Get index to take right pos at dictionary
                index = urls.index(url)
                
                title = article_summary.get('title')[index]
                text = article_summary.get('summary')[index]
                summary = article_summary.get('summary')[index]
                date = article_summary.get('date')[index]
                if('media' in article_summary.keys()):
                    url_api = url_api+' '+article_summary.get('media')[index]  
        
        try:
            # Call Sentiment Analizer
            title_sentiment, title_pos, title_neu, title_neg = getAzureSentiment(title, client)
            text_sentiment, text_pos, text_neu, text_neg = getAzureSentiment(text, client)
            # Add article to df 
            dff = dff.append({'API':url_api,
                                'Link':url, 
                                'Date':date, 
                                'Title':title, 
                                'Article':text, 
                                'Summary':summary, 
                                'Title_Sentiment':title_sentiment,
                                'Title_Positv':title_pos, 
                                'Title_Neutral':title_neu, 
                                'Title_Negativ':title_neg, 
                                'Article_Sentiment':text_sentiment,
                                'Article_Positv':text_pos, 
                                'Article_Neutral':text_neu, 
                                'Article_Negativ':text_neg}, 
                               ignore_index=True) 
        except:
            print('Azure Sentiment could not be called. ' + url)
    
    return dff
def tweets_to_df(dff, tweets_data, searchword): # Only Risk.Net 
    """"""
    

    # Authenticate connection to azure text analytics service
    client = authenticate_azure_textanalytics_client()

    for i in range(len(tweets_data[0])):
        # Initialize 
        # tweets_data : [texts, dates, users]
        title = tweets_data[0][i]
        date = tweets_data[1][i]
        link = 'User: '+ tweets_data[2][i]
        
        # Verify that tweet is actually about LCH SA (TO BE IMPROVED)
        #if ((False==(('LCH' in title)or('lch sa' in title))
        #     or('Light combat Helic' in title))):
        if ((False==(searchword.lower() in title.lower()))):
            continue

        try:
            # Call Sentiment Analizer
            title_sentiment, title_pos, title_neu, title_neg = getAzureSentiment(title, client)
        except:
            print('Twitter: Azure Sentiment could not be called.')

        dff = dff.append({'API':'Twitter',
                        'Link':link, 
                        'Date':date, 
                        'Title':title, 
                        'Article':'', 
                        'Summary':'', 
                        'Title_Sentiment':title_sentiment,
                        'Title_Positv':title_pos, 
                        'Title_Neutral':title_neu, 
                        'Title_Negativ':title_neg, 
                        'Article_Sentiment':'',
                        'Article_Positv':'', 
                        'Article_Neutral':'', 
                        'Article_Negativ':''}, 
                       ignore_index=True) 
    
    return dff
def ft_news_to_df(ft_dict, dff):
    """"""
    ft_url_base = 'https://www.ft.com'
    
    for i in range(len(ft_dict.get('Link'))):
        # Get Article 
        title = ft_dict.get('Title')[i]
        date = ft_dict.get('Date')[i][:10]
        link = ft_url_base + ft_dict.get('Link')[i]
        
        # Authenticate connection to azure text analytics service
        client = authenticate_azure_textanalytics_client()
        try:
          # Call Sentiment Analizer
          title_sentiment, title_pos, title_neu, title_neg = getAzureSentiment(title, client)
        except:
            print('Twitter: Azure Sentiment could not be called.')

        dff = dff.append({'API':'FinancialTimes',
                            'Link':link, 
                            'Date':date, 
                            'Title':title, 
                            'Article':'', 
                            'Summary':'', 
                            'Title_Sentiment':title_sentiment,
                            'Title_Positv':title_pos, 
                            'Title_Neutral':title_neu, 
                            'Title_Negativ':title_neg, 
                            'Article_Sentiment':'',
                            'Article_Positv':'', 
                            'Article_Neutral':'', 
                            'Article_Negativ':''}, 
                           ignore_index=True) 
    
    return dff
    
# Fetch News via APIs
def add_google_news(dff, searchword):
    """"""
    #Import API
    from GoogleNews import GoogleNews

    # Max and Min Date of Search
    start_dt = (datetime.today() - timedelta(days=search_days)).strftime('%m/%d/%Y')
    end_dt = datetime.today().strftime('%m/%d/%Y')

    # Fetch News 
    googlenews=GoogleNews(start=start_dt,end=end_dt, lang='en') # language english
    googlenews.search(searchword)
    result=googlenews.result()

    # List of news urls 
    urls = [news.get('link') for news in result]
    title_list = [news.get('title') for news in result]
    media_list = [news.get('media') for news in result]
    date_list = [news.get('datetime').strftime('%Y/%m/%d') for news in result]
    summary_list = [news.get('desc') for news in result]
    article_summary = {'title':title_list,
                       'media': media_list,
                       'date':date_list,
                       'summary':summary_list}
    
    dff = articles_to_df(urls, dff, api='Google', article_summary=article_summary)
    
    return dff
def add_scrapy_news(dff, searchword):
    """"""
    # Import
    import scrapy
    from scrapy.crawler import CrawlerProcess
    import logging
    logging.getLogger('scrapy').propagate = False

    # Create the Spider class
    class news_Spider(scrapy.Spider):
        name = "News_Spider"
        global scrapy_dict
        global ft_dict

        # start_requests method
        def start_requests(self):
            risknet_url = 'https://www.risk.net/search/articles?query='+searchword.replace(' ', '+')+'&sort_by=relevance&date-filter=7+day'
            ft_url = 'https://www.ft.com/search?q='+searchword.replace(' ', '%20')+'&sort=date&dateRange=now-7d&expandRefinements=true'
            urls = [risknet_url]#,
                    #'https://www.ft.com/stream/3da687f2-11a7-4e28-a18d-17b77aa18460']
            for url in urls:
                yield scrapy.Request(url = url, callback = self.parse_front)

        # First parsing method
        def parse_front(self, response):
            risk_base_url = 'https://www.risk.net'
            ft_base_url = 'https://www.ft.com'
            if(response.url[:len(risk_base_url)]==risk_base_url):
                # Extract Link
                link = response.xpath('//h5[@class="article-title"]/a/@href')
                link_extract = link.extract()
                scrapy_dict['risk']=link_extract
            elif(response.url[:len(ft_base_url)]==ft_base_url):
                link = response.xpath('//*[@id="stream"]/div[1]/ul/li/div[2]/div/div/div/div[2]/a/@href') 
                link_extract = link.extract()
                scrapy_dict['ft']= link_extract 
                ft_dict['Link'] = link_extract
                # Extract Title
                title = response.xpath('//*[@id="stream"]/div[1]/ul/li/div[2]/div/div/div/div[2]/a/text()') 
                ft_dict['Title'] = title.extract()
                # Extract Date
                date = response.xpath('//div[@class="stream-card__date"]/time/@datetime') 
                ft_dict['Date'] = date.extract()

    scrapy_dict = dict()
    ft_dict = dict()

    # Run the Spider
    process = CrawlerProcess()
    process.crawl(news_Spider)
    process.start()

    # Build Link List
    risk_url_base = 'http://www.risk.net'

    # Add Risk.Net News 
    urls = [url if (url[:4]=='http') else risk_url_base+url for url in scrapy_dict.get('risk')]
    dff = articles_to_df(urls, dff, api='Risk.Net')
    
    # Add FT News 
    #dff = ft_news_to_df(ft_dict, dff)

    return dff    
def add_twitter_news(dff, searchword):
    """"""
    #Import
    import tweepy
    
    # Consumer keys and access tokens, used for OAuth
    consumer_key = api_key
    consumer_secret = api_secret_key

    # OAuth process, using the keys and tokens
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    # Creation of the actual interface, using authentication
    api = tweepy.API(auth)
    
    # Fetch Tweets 
    searchterm = searchword + ' -filter:retweets' # Exclude Retweets
    tweets = api.search(q=searchterm, lang="en" ) #  , result_type='recent'
    
    def clearnUpTweet(txt):
        """"""
        txt = re.sub(r'@[A-Za-z0-9_]+', '', txt) # Remove mentions # [A-Za-z0-9_]+
        txt = re.sub(r'#', ' ', txt) # Remove hashtags
        txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', txt) # Remove urls
        txt = re.sub(r'&gt;', ' is greater than ', txt) # 
        txt = re.sub(r'&gt;', ' is smaller than ', txt) # 
        txt = re.sub(r'\n', ' ', txt) #
        txt = re.sub(r'w/', 'with', txt) # 
        return txt
    
    # Tweets Data
    texts = [clearnUpTweet(tweet.text) for tweet in tweets]
    dates = [tweet.created_at.date() for tweet in tweets]
    users = [tweet.user.name for tweet in tweets] # Bug: This is not the actual user that posted it
    tweets_data = [texts, dates, users]
    
    # Add Tweets to DF
    dff = tweets_to_df(dff, tweets_data, searchword)
    
    return dff
def add_bing_news(dff, searchword):
    """"""
    # Authenticate connection to azure bing news search service
    bing_client = get_newssearch_client()
    
    # Fetch News
    results = bing_client.news.search(query=searchword,
            #market="en-uk", NOT WORKING 
            freshness="Week",
            #setLang:"en", NOT WORKING
            sort_by="Date"
        )
    
    urls = [article.get('url') for article in results.as_dict().get('value')]
    title_list = [article.get('name') for article in results.as_dict().get('value')]
    summary_list = [article.get('description') for article in results.as_dict().get('value')]
    date_list = [article.get('date_published') for article in results.as_dict().get('value')]
    date_list = [datetime.strptime(date[:10], '%Y-%m-%d').strftime('%Y/%m/%d') for date in date_list]
    media_list = [article.get('provider')[0].get('name') for article in results.as_dict().get('value')]
    
    article_summary = {'title':title_list,
                       'media': media_list,
                       'date':date_list,
                       'summary':summary_list}
    
    dff = articles_to_df(urls, dff, api='Bing', article_summary=article_summary)
    
    return dff

#  Additional Features
def verify_articles(dff, searchword, delete=False):
    """"""
    # Authenticate Azure text analysis client 
    ta_client = authenticate_azure_textanalytics_client()
    
    def get_text_topics(client, text):
        """"""
        text = text[:5120]
        document = [text]

        try:
            response = client.extract_key_phrases(document)
            result = [doc for doc in response if not doc.is_error]
            topics = result[0].key_phrases
            topics = [topic.lower() for topic in topics]
        except: 
            topics = []
        topics = str(topics)[1:-1]
        return topics
    def language_detection_example(text, client):
        try:
            documents = [text[:5120]]
            response = client.detect_language(documents = documents)[0]
            language = response.primary_language.name
            return language
        except Exception as err:
            return 'Unknown'
        
    verified_col = []
    verified_flg_col = []
    for row in dff.itertuples():
        verified = '' 
        verified_flg = True
        #Check Language of Article
        text = row.Article if len(row.Article)>25 else row.Title
        language = language_detection_example(row.Article, ta_client)
        if (language != 'English'):
            verified += 'Detected language is ' + language + '. '
            verified_flg = False

        # Check if Article is actually about searchword
        title_topic_list = get_text_topics(ta_client, row.Title)
        if (searchword.lower() not in title_topic_list):
            verified += searchword + ' is not subject of the title.'
            verified_flg = False

        # Is verified
        verified_col.append(verified)
        verified_flg_col.append(verified_flg)

    dff['Verified'] = verified_col
    dff['Verified_Flag'] = verified_flg_col
    
    if(delete):
        dff = dff[dff.Verified_Flag==True]
    
    return dff
def add_keyword_search(dff):
    """"""
    # Keyword Search
    def find_keywords(article):
        """"""
        from fuzzysearch import find_near_matches
        keywords = ['fraud', 'claim', 'incident', 'bankruptcy', 'fine', 'bankrupt',
                    'Fraud', 'Claim', 'Incident', 'Bankruptcy', 'Fine', 'Bankrupt']
        words = ''
        for keyword in keywords:
            count = len(find_near_matches(keyword, article, max_l_dist=0))
            if(count>0):
                words += str(count)+'x '+keyword+', '
        if(len(words)>1):
            words = words[:-2]
        else:
            words = '-'
        return words
    
    dff['Found_Keywords'] = dff.Article.apply(find_keywords)
    return dff
def add_ner_search(dff):
    """"""
    
    # Authenticate connection to azure text analytics service
    text_analytics_client = authenticate_azure_textanalytics_client()
    
    # NER Search
    def get_ner_string(article):
        """"""
        try:
            article = article[:5120] # Max length for article # should be split in multisearch
            document = [article]

            response = text_analytics_client.recognize_entities(document , language="en")
            result = [doc for doc in response if not doc.is_error]

            ner_entities = []
            ner_categories, ner_scores = [], []

            for entity in result[0].entities:
                if (entity.category in ['Organization']):#,'Skill']):
                    ner_entities.append(entity.text) 
                    ner_categories.append(entity.category)
                    #ner_scores += entity.confidence_score

            return str(list(set(ner_entities)))[1:-1]
        except:
            return '-'
    
    # Add Ner column 
    dff['NER'] = dff.Title.apply(get_ner_string)+dff.Article.apply(get_ner_string)
    
    return dff 
def add_topic_search(dff):
    """"""
    # Authenticate Azure text analysis client 
    text_analytics_client = authenticate_azure_textanalytics_client()
    
    def get_article_topics(article):
        """"""
        article = article[:5120]
        document = [article]

        try:
            response = text_analytics_client.extract_key_phrases(document)
            result = [doc for doc in response if not doc.is_error]
            topics = result[0].key_phrases
        except: 
            topics = []
        topics = str(topics)[1:-1]
        return topics
    
    # Add Ner column 
    dff['Topics'] = dff.Title.apply(get_article_topics)+dff.Article.apply(get_article_topics)
    
    return dff 

# Counterparty DF 
def get_ctp_df(searchword, cpt_df):
    """"""
    # Initialize CPT df
    cpt_df['CPT'] = ''
    # If Counterparties are known then make a search
    if (searchword in cpt_dict.keys()):
        # for each counterparty 
        for cpt in cpt_dict.get(searchword):
            # Initial cpt df 
            tmp_cpt_df = get_init_df()
            # Add news via APIs
            tmp_cpt_df = add_google_news(tmp_cpt_df, cpt)
            tmp_cpt_df = add_bing_news(tmp_cpt_df, cpt)
            tmp_cpt_df = add_twitter_news(tmp_cpt_df, cpt)    
            # Verify articles
            tmp_cpt_df = verify_articles(tmp_cpt_df, cpt, True)
            # Add CPT column and append tmp_cpt_df to cpt_df
            tmp_cpt_df['CPT'] = cpt
            cpt_df = cpt_df.append(tmp_cpt_df, ignore_index=True)
        # Features
        # Find Keywords like Fraud, etc.
        cpt_df = add_keyword_search(cpt_df)    
        # Ner and Topic Search 
        cpt_df = add_ner_search(cpt_df)
        cpt_df = add_topic_search(cpt_df)
    return cpt_df

# Text Message 
def get_sentiment_message(dff, message):
    """"""
    if(len(dff)>0):
        # Overall Sentiment  
        sentiment_list = [*dff.Title_Sentiment] + [*dff.Article_Sentiment]
        sentiment = max(set(sentiment_list), key = sentiment_list.count) 
        message += 'The overall sentiment of the articles was '+sentiment + '.\n\n'
        # Mean sentiment
        def str_mean(x):
            x = np.array([val for val in x if isinstance(val, numbers.Number)])
            return str(round(x.mean(),2))

        message += 'Mean Sentiment: Title ('\
                                    + str_mean(dff.Title_Positv)+', '\
                                    + str_mean(dff.Title_Neutral)+', '\
                                    + str_mean(dff.Title_Negativ)+'); '\
                                +'Article ('+str_mean(dff.Article_Positv)+', '\
                                            +str_mean(dff.Article_Positv)+', '\
                                            +str_mean(dff.Article_Negativ)+').+\n\n'
        # Negativ Articles
        # Articles with a negativ score >0.33
        neg_th =  0.33
        neg_df = dff[(dff.Title_Negativ>neg_th)|(dff.Title_Negativ>neg_th)]
        neg_count=len(neg_df) 
        if(neg_count==1):
            message += 'There is '+str(neg_count) + ' article with a negativ sentiment:\n'
        elif(neg_count>1):
            message += 'There are '+str(neg_count) + ' articles with a negativ sentiment:\n'
        for row in neg_df.itertuples():
             message += '\t'+row.Title + ' ['+row.Title_Sentiment+' title ('\
                                                +str(row.Title_Positv)+', '\
                                                +str(row.Title_Neutral)+', '\
                                                +str(row.Title_Negativ)+'); '\
                                            +row.Article_Sentiment+' article ('\
                                                +str(row.Article_Positv)+', '\
                                                +str(row.Article_Neutral)+', '\
                                                +str(row.Article_Negativ)+')]+\n\n'

        return message
    else:
        print("No news were fetched.")
        return ''

# Main

In [11]:
# Main
def main(searchword):
    
    # Init
    message = get_init_message(searchword)
    df = get_init_df()
    cpt_df = get_init_df()
    
    # Add News 
    df = add_google_news(df, searchword) # API can be blocked via Azure
    df = add_scrapy_news(df, searchword) # For now only for LCH SA
    df = add_bing_news(df, searchword)
    df = add_twitter_news(df, searchword)
    
    # Add specific Articles
    urls = ['https://www.vanityfair.com/news/2020/06/bankrupt-hertz-is-a-pandemic-zombie',
            #'https://www.forbes.com/sites/jackkelly/2020/05/23/hertz-files-for-bankruptcy-after-16000-employees-were-let-go-and-ceo-made-over-9-million/'
            'https://www.nbcnews.com/politics/meet-the-press/hertz-jcpenny-j-crew-join-list-businesses-filing-bankruptcy-n1213961',
            'https://edition.cnn.com/2020/05/22/business/hertz-bankruptcy/index.html'
           ]  
    #urls=['http://citywireselector.com/news/bnp-paribas-am-revamps-thematic-fund-to-tackle-inequalities/a1470113?ref=citywire_global_latest_news_list']
    #df = articles_to_df(urls, df, 'Urls')
    
    
    # Verification Step: Are news actually about searchword
    df = verify_articles(df, searchword, False)
    
    # Find Keywords like Fraud, etc.
    df = add_keyword_search(df)
    
    # Ner and Topic Search 
    df = add_ner_search(df)
    df = add_topic_search(df)
    
    #Get Overall Sentiment
    message = get_sentiment_message(df, message)
    
    # Get News on biggest counterparties
    cpt_df = get_ctp_df(searchword, cpt_df)
    
    #Save results 
    save_csv(df, searchword)
    
    return message, df, cpt_df


searchword='LCH SA'
#searchword='EUREX'
#searchword='HERTZ'
message, df, cpt_df = main(searchword)
# import pprint as pp
# pp.pprint(message)

2021-03-03 13:26:36 [msrest.universal_http.requests] DEBUG: Configuring retry: max_retries=3, backoff_factor=0.8, max_backoff=90
2021-03-03 13:26:36 [msrest.universal_http] DEBUG: Configuring redirects: allow=True, max=30
2021-03-03 13:26:36 [msrest.universal_http] DEBUG: Configuring request: timeout=100, verify=True, cert=None
2021-03-03 13:26:36 [msrest.universal_http] DEBUG: Configuring proxies: ''
2021-03-03 13:26:36 [msrest.universal_http] DEBUG: Evaluate proxies against ENV settings: True
2021-03-03 13:26:38 [tweepy.binder] DEBUG: PARAMS: {'q': b'LCH SA -filter:retweets', 'lang': b'en'}
2021-03-03 13:26:38 [requests_oauthlib.oauth1_auth] DEBUG: Signing request <PreparedRequest [GET]> using client <Client client_key=Oc9II7RV8McqVCXjU9MybOvnP, client_secret=****, resource_owner_key=1347130231908421632-Wqvur6Qc1Emo3uj1U2Meyz2uJ6X45R, resource_owner_secret=****, signature_method=HMAC-SHA1, signature_type=AUTH_HEADER, callback_uri=None, rsa_key=None, verifier=None, realm=None, encodin

2021-03-03 13:27:42 [msrest.universal_http] DEBUG: Evaluate proxies against ENV settings: True
2021-03-03 13:27:45 [newspaper.article] DEBUG: Download failed on URL https://www.forbes.com/sites/palashghosh/2021/03/03/as-jane-fraser-heads-up-citi-female-ceos-at-large-banks-remain-few-and-far-between/ because of 403 Client Error: Max restarts limit reached for url: https://www.forbes.com/sites/palashghosh/2021/03/03/as-jane-fraser-heads-up-citi-female-ceos-at-large-banks-remain-few-and-far-between/
2021-03-03 13:27:45 [newspaper.article] DEBUG: Download failed on URL https://www.wsj.com/articles/greensill-capital-planning-to-file-for-insolvency-in-u-k-this-week-11614768903 because of 403 Client Error: Forbidden for url: https://www.wsj.com/articles/greensill-capital-planning-to-file-for-insolvency-in-u-k-this-week-11614768903
2021-03-03 13:27:54 [tweepy.binder] DEBUG: PARAMS: {'q': b'Citigroup -filter:retweets', 'lang': b'en'}
2021-03-03 13:27:54 [requests_oauthlib.oauth1_auth] DEBUG: Si

In [135]:
message

"2021-03-03 13:16\nWebScraper: CCP\n\nThe overall sentiment of the articles was negative.\n\nMean Sentiment: Title (0.04, 0.42, 0.54); Article (0.1, 0.1, 0.39).+\n\nThere are 20 articles with a negativ sentiment:\n\tChinese Companies Buy up UK Private Schools, Sparking Fears of CCP Influence [negative title (0.0, 0.04, 0.96); neutral article (0.04, 0.95, 0.01)]+\n\n\tCCP using war on graft to clamp down on Xi opponents [negative title (0.0, 0.0, 1.0); neutral article (0.0, 1.0, 0.0)]+\n\n\tFake '1992 consensus' continues to drag down cross-strait relations [negative title (0.0, 0.01, 0.99); negative article (0.03, 0.21, 0.76)]+\n\n\tUpdates on CCP Virus: China's Top COVID-19 Expert Predicts the CCP Virus Outbreak May End This Summer [negative title (0.0, 0.05, 0.95); neutral article (0.16, 0.62, 0.22)]+\n\n\tUpdates on CCP Virus: New Mexico Sees Spike in Daily Case ... [negative title (0.01, 0.14, 0.85); negative article (0.0, 0.21, 0.79)]+\n\n\tThe empire strikes back: what the Qing d

In [136]:
df

,API,Link,Date,Title,Article,Summary,Title_Sentiment,Title_Positv,Title_Neutral,Title_Negativ,Article_Sentiment,Article_Positv,Article_Neutral,Article_Negativ,Verified,Verified_Flag,Found_Keywords,NER,Topics
0,Google Radio Free Asia,https://www.rfa.org/english/news/china/schools...,2021/02/27,"Chinese Companies Buy up UK Private Schools, S...","""The CCP's investment in British private educa...","""The CCP's investment in British private educa...",negative,0.00,0.04,0.96,neutral,0.04,0.95,0.01,,True,-,'CCP''CCP',"'UK Private Schools', 'Fears of CCP Influence'..."
1,Google The Standard,https://www.standardmedia.co.ke/opinion/articl...,2021/02/27,CCP using war on graft to clamp down on Xi opp...,Central Commission for Discipline Inspection (...,Central Commission for Discipline Inspection (...,negative,0.00,0.00,1.00,neutral,0,1,0,,True,-,'CCP''Central Commission for Discipline Inspec...,"'graft', 'CCP', 'war', 'Xi opponents''CCDI', '..."
2,Google Taiwan News,https://www.taiwannews.com.tw/en/news/4136706,2021/02/27,Fake '1992 consensus' continues to drag down c...,Tensions with the Chinese Communist Party (CCP...,Tensions with the Chinese Communist Party (CCP...,negative,0.00,0.01,0.99,negative,0.03,0.21,0.76,CCP is not subject of the title.,False,-,"'CCP', 'DPP', 'KMT', 'Chinese Communist Party'","'cross-strait relations', 'consensus''Chinese ..."
3,Google,https://www.esma.europa.eu/press-news/esma-new...,2021/02/24,publishes Guidelines to harmonise CCP supervis...,The objective of these guidelines is to ensure...,The objective of these guidelines is to ensure...,neutral,0.01,0.97,0.02,neutral,0.04,0.94,0.02,,True,-,'CCP',"'CCP supervisory reviews', 'evaluation', 'EMIR..."
4,Google,https://www.esma.europa.eu/press-news/esma-new...,2021/03/02,SEEKS EXPERTS ON CENTRAL COUNTERPARTIES TO JOI...,The CCP PC is responsible for preparing ESMA's...,The CCP PC is responsible for preparing ESMA's...,neutral,0.02,0.98,0.00,neutral,0.01,0.96,0.03,CCP is not subject of the title.,False,-,"'CCP', 'ESMA'","'CENTRAL COUNTERPARTIES', 'CONSULTATIVE GROUP'..."
5,Google,https://www.gov.uk/government/consultations/ex...,2021/02/25,Expanded Resolution Regime for Central Counter...,An expanded CCP resolution regime would give t...,An expanded CCP resolution regime would give t...,neutral,0.12,0.87,0.01,negative,0,0,1,CCP is not subject of the title.,False,-,'Bank of England',"'Central Counterparties', 'Expanded Resolution..."
6,Google Times of India,https://timesofindia.indiatimes.com/city/goa/p...,2021/02/26,Panaji rival group gears up to unveil 30 names...,The team will announce the names of candidates...,The team will announce the names of candidates...,neutral,0.05,0.82,0.13,neutral,0.02,0.97,0.01,,True,-,"'CCP', 'Panaji''CCP'","'names', 'Panaji rival group', 'CCP polls''day..."
7,Google,https://www.accountancydaily.co/cch_uk/tat/new...,2021/02/25,Government consults on expanded resolution reg...,These powers would also limit risks to public ...,These powers would also limit risks to public ...,neutral,0.03,0.95,0.02,negative,0,0,1,,True,-,'CCP',"'expanded resolution regime', 'Government cons..."
8,Google NTD,https://mb.ntd.com/updates-on-ccp-virus-recent...,2021/03/03,Updates on CCP Virus: China's Top COVID-19 Exp...,Drugmaker Merck & Co. will help produce rival ...,Drugmaker Merck & Co. will help produce rival ...,negative,0.00,0.05,0.95,neutral,0.16,0.62,0.22,,True,-,"'Merck & Co.', 'Johnson & Johnson'","'CCP Virus Outbreak', ""China's"", 'Expert', 'CO..."
9,Google NTD,https://mb.ntd.com/updates-on-ccp-virus-new-me...,2021/02/28,Updates on CCP Virus: New Mexico Sees Spike in...,Nearly 30 percent of the new cases involved st...,Nearly 30 percent of the new cases involved st...,negative,0.01,0.14,0.85,negative,0,0.21,0.79,,True,-,,"'New Mexico', 'Spike', 'CCP Virus', 'Daily Cas..."


In [18]:
df.Article_Positv

0    0.75
1    0.00
2    0.01
3    0.00
4    0.00
5    0.00
6    0.00
7    0.30
8    0.45
Name: Article_Positv, dtype: float64

In [22]:

pos = round((df.Title_Positv.sum() + df.Article_Positv.sum())/(2*len(df)),2) # Instead div by count 
neu = round((df.Title_Neutral.sum() + df.Article_Neutral.sum())/(2*len(df)),2)
neg = round((df.Title_Negativ.sum() + df.Article_Negativ.sum())/(2*len(df)),2)
pos, neu, neg

# If pos > 0.5 --> positiv
# If neg > 0.5 --> negativ
# If pos&neg>0.3 --> mixed 
# Else Neutral

(0.09, 0.37, 0.54)

In [13]:
cpt_df# =  get_ctp_df(searchword)

,API,Article,Article_Negativ,Article_Neutral,Article_Positv,Article_Sentiment,CPT,Date,Link,Summary,Title,Title_Negativ,Title_Neutral,Title_Positv,Title_Sentiment,Verified,Verified_Flag,Found_Keywords,NER,Topics
0,Google citywireselector.com,The strategy will be renamed as the BNP Pariba...,0.03,0.79,0.18,neutral,BNP Paribas,2021/02/24,http://citywireselector.com/news/bnp-paribas-a...,The strategy will be renamed as the BNP Pariba...,BNP Paribas AM revamps thematic fund to tackle...,0.01,0.95,0.04,neutral,,True,-,'BNP Paribas AM''BNP Paribas Inclusive Growth',"'revamps thematic fund', 'BNP Paribas', 'inequ..."
1,Google,"Boudewijn Dierick, BNP Paribas’ head of ABS an...",0,0.99,0.01,neutral,BNP Paribas,None,https://www.globalcapital.com/article/b1qp0968...,"Boudewijn Dierick, BNP Paribas’ head of ABS an...",Head of ABS and covered bonds leaves BNP Paribas,0.00,0.99,0.01,neutral,,True,-,"'ABS', 'BNP Paribas''BNP Paribas', 'Auxmoney I...","'covered bonds', 'Head of ABS', 'BNP Paribas''..."
2,Google,Three new exchange traded funds issued by BNP ...,0.94,0.06,0,negative,BNP Paribas,2021-02-24 00:00:00,https://www.etfexpress.com/2021/02/24/296290/b...,Three new exchange traded funds issued by BNP ...,BNP Paribas Easy launches sustainable eurozone...,0.00,0.54,0.46,neutral,,True,-,"'BNP Paribas Easy', 'Xetra''BNP Paribas Easy',...","'emerging markets bonds ETFs', 'BNP Paribas Ea..."
3,Google,Alternative Lending\n\nBoudewijn Dierick will ...,0.02,0.91,0.07,neutral,BNP Paribas,None,https://www.altfi.com/article/7615_auxmoney-hi...,Alternative LendingBoudewijn Dierick will run ...,Auxmoney hires ex-BNP Paribas MD to lead its j...,0.00,1.00,0.00,neutral,,True,-,"'BNP Paribas MD', 'Auxmoney''Barclaycard Germa...","'Auxmoney', 'ex-BNP Paribas MD', 'investments ..."
4,Google,"Trade Finance Market Size, Status and Forecast...",0.01,0.48,0.51,positive,BNP Paribas,None,https://bisouv.com/tag/bnp-paribas/,"Trade Finance Market Size, Status and Forecast...",BNP Paribas – The Bisouv Network,0.00,1.00,0.00,neutral,,True,-,"'BNP Paribas', 'The Bisouv Network''ICBC', 'Af...","'BNP Paribas', 'Bisouv Network''Trade Finance ..."
5,Google,BNP Paribas returned to the Swiss franc market...,0.35,0.16,0.49,mixed,BNP Paribas,None,https://www.globalcapital.com/article/b1qqdlrd...,BNP Paribas returned to the Swiss franc market...,BNP Paribas serves up Swissie return,0.06,0.92,0.02,neutral,,True,-,"'BNP Paribas', 'Swissie''Zürcher Kantonalbank'...","'BNP Paribas', 'Swissie return''senior non-pre..."
6,Google,A judge hearing a million-euro case filed agai...,0.88,0.1,0.02,negative,BNP Paribas,None,https://www.maltatoday.com.mt/news/court_and_p...,A judge hearing a million-euro case filed agai...,BNP Paribas ‘anti-semitism’ row: Malta judge s...,0.00,1.00,0.00,neutral,,True,-,"'BNP Paribas''BNP', 'Court of Appeal', 'BNP Pa...","'row', 'Malta judge', 'anti-semitism', 'bank',..."
7,Google,NEW YORK • Citigroup Inc is studying options f...,0.52,0.12,0.36,mixed,Citigroup,2021-02-22 05:00:00+08:00,https://www.straitstimes.com/business/banking/...,NEW YORK • Citigroup Inc is studying options f...,Citigroup looking to divest some foreign consu...,0.29,0.62,0.09,neutral,,True,-,"'Citigroup''Bloomberg', 'Citibanamex', 'CNN', ...","'foreign consumer units', 'Citigroup'""Citigrou..."
8,Google,Find this comment offensive?\n\nChoose your re...,1,0,0,negative,Citigroup,None,https://economictimes.indiatimes.com/news/inte...,Find this comment offensive?\nChoose your reas...,Citigroup considering divestiture of some fore...,0.07,0.91,0.02,neutral,,True,-,"'Bloomberg Law', 'Citigroup'","'foreign consumer units - Bloomberg Law', 'div..."
9,Google,"Bloomberg\n\n(Bloomberg) -- While 2020 raged, ...",0.66,0.05,0.29,mixed,Citigroup,None,https://finance.yahoo.com/news/1-citigroup-con...,And Buffett isn’t usually shy about sharing th...,UPDATE 1-Citigroup considering divestiture of ...,0.02,0.98,0.00,neutral,,True,"1x claim, 1x fine","'Bloomberg Law', 'Citigroup''Bloomberg', 'Berk...","'d

## Test

In [46]:
# Initialize CPT df
cpt_df = get_init_df()
cpt_df['CPT'] = ''
# If Counterparties are known then make a search
if (searchword in cpt_dict.keys()):
    # for each counterparty 
    for cpt in cpt_dict.get(searchword):
        print(cpt)
        # Initial cpt df 
        tmp_cpt_df = get_init_df()
        # Add news via APIs
        tmp_cpt_df = add_google_news(tmp_cpt_df, cpt)
        tmp_cpt_df = add_bing_news(tmp_cpt_df, cpt)
        tmp_cpt_df = add_twitter_news(tmp_cpt_df, cpt) 
        print(len(tmp_cpt_df))
        # Verify articles
        tmp_cpt_df = verify_articles(tmp_cpt_df, cpt, False)
        print(len(tmp_cpt_df))
        # Add CPT column and append tmp_cpt_df to cpt_df
        tmp_cpt_df['CPT'] = cpt
        cpt_df = cpt_df.append(tmp_cpt_df, ignore_index=True)
    # Features
    # Find Keywords like Fraud, etc.
    cpt_df = add_keyword_search(cpt_df)    
    # Ner and Topic Search 
    cpt_df = add_ner_search(cpt_df)
    cpt_df = add_topic_search(cpt_df)


BNP Paribas


2021-02-24 16:33:06 [msrest.universal_http.requests] DEBUG: Configuring retry: max_retries=3, backoff_factor=0.8, max_backoff=90
2021-02-24 16:33:06 [msrest.universal_http] DEBUG: Configuring redirects: allow=True, max=30
2021-02-24 16:33:06 [msrest.universal_http] DEBUG: Configuring request: timeout=100, verify=True, cert=None
2021-02-24 16:33:06 [msrest.universal_http] DEBUG: Configuring proxies: ''
2021-02-24 16:33:06 [msrest.universal_http] DEBUG: Evaluate proxies against ENV settings: True


Azure Sentiment could not be called. https://video.capital.fr/toutes-les-videos/nord-une-soixantaine-de-plaintes-pour-des-pannes-apres-un-passage-a-une-station-service_9770


2021-02-24 16:33:19 [tweepy.binder] DEBUG: PARAMS: {'q': b'BNP Paribas -filter:retweets', 'lang': b'en'}
2021-02-24 16:33:19 [requests_oauthlib.oauth1_auth] DEBUG: Signing request <PreparedRequest [GET]> using client <Client client_key=Oc9II7RV8McqVCXjU9MybOvnP, client_secret=****, resource_owner_key=1347130231908421632-Wqvur6Qc1Emo3uj1U2Meyz2uJ6X45R, resource_owner_secret=****, signature_method=HMAC-SHA1, signature_type=AUTH_HEADER, callback_uri=None, rsa_key=None, verifier=None, realm=None, encoding=utf-8, decoding=None, nonce=None, timestamp=None>
2021-02-24 16:33:19 [requests_oauthlib.oauth1_auth] DEBUG: Including body in call to sign: False
2021-02-24 16:33:19 [oauthlib.oauth1.rfc5849] DEBUG: Collected params: [('q', 'BNP Paribas -filter:retweets'), ('lang', 'en'), ('oauth_nonce', '126853994978314780351614180799'), ('oauth_timestamp', '1614180799'), ('oauth_version', '1.0'), ('oauth_signature_method', 'HMAC-SHA1'), ('oauth_consumer_key', 'Oc9II7RV8McqVCXjU9MybOvnP'), ('oauth_token

27
27
Citigroup


2021-02-24 16:33:37 [msrest.universal_http.requests] DEBUG: Configuring retry: max_retries=3, backoff_factor=0.8, max_backoff=90
2021-02-24 16:33:37 [msrest.universal_http] DEBUG: Configuring redirects: allow=True, max=30
2021-02-24 16:33:37 [msrest.universal_http] DEBUG: Configuring request: timeout=100, verify=True, cert=None
2021-02-24 16:33:37 [msrest.universal_http] DEBUG: Configuring proxies: ''
2021-02-24 16:33:37 [msrest.universal_http] DEBUG: Evaluate proxies against ENV settings: True
2021-02-24 16:33:49 [tweepy.binder] DEBUG: PARAMS: {'q': b'Citigroup -filter:retweets', 'lang': b'en'}
2021-02-24 16:33:49 [requests_oauthlib.oauth1_auth] DEBUG: Signing request <PreparedRequest [GET]> using client <Client client_key=Oc9II7RV8McqVCXjU9MybOvnP, client_secret=****, resource_owner_key=1347130231908421632-Wqvur6Qc1Emo3uj1U2Meyz2uJ6X45R, resource_owner_secret=****, signature_method=HMAC-SHA1, signature_type=AUTH_HEADER, callback_uri=None, rsa_key=None, verifier=None, realm=None, enco

28
28


In [51]:
cpt_df[(cpt_df.Verified_Flag==False)&(cpt_df.API=='Twitter')]

,API,Article,Article_Negativ,Article_Neutral,Article_Positv,Article_Sentiment,CPT,Date,Link,Summary,Title,Title_Negativ,Title_Neutral,Title_Positv,Title_Sentiment,Verified,Verified_Flag,Found_Keywords,NER,Topics
19,Twitter,,,,,,BNP Paribas,2021-02-24,User: Ericom Software Inc.,,News: Ericom appoints Industry Advisory Board...,0.00,1.00,0.00,neutral,Detected language is Unknown.,False,-,"'Ericom', 'Industry Advisory Board', 'Becca Ch...",'Industry Advisory Board members Ramy Houssain...
20,Twitter,,,,,,BNP Paribas,2021-02-24,User: StockMarketNews,,BNP Paribas London : Form 8.3 - William Hill Plc,0.00,1.00,0.00,neutral,Detected language is Unknown.,False,-,"'William Hill Plc', 'BNP Paribas London'-","'Form', 'BNP Paribas London', 'William Hill Plc'"
21,Twitter,,,,,,BNP Paribas,2021-02-24,User: ETF Bulletin,,BNP Paribas Easy launches sustainable eurozone...,0.00,0.60,0.40,neutral,Detected language is Unknown.,False,-,"'Xetra', 'BNP Paribas Easy'-","'emerging markets bonds ETFs', 'BNP Paribas Ea..."
22,Twitter,,,,,,BNP Paribas,2021-02-24,User: @Cbonds_Global,,Volume: benchmark size deal Bookrunners and ...,0.00,0.99,0.01,neutral,Detected language is Unknown.,False,-,"'HSBC', 'BNP Paribas', 'Goldman Sachs'-","'Lead Managers', 'Goldman Sachs', 'BNP Paribas..."
23,Twitter,,,,,,BNP Paribas,2021-02-24,User: @Cbonds_Global,,Volume: benchmark size deal Bookrunners and ...,0.00,0.99,0.01,neutral,Detected language is Unknown.,False,-,"'Santander', 'Morgan Stanley', 'BNP Paribas'-","'Lead Managers', 'BNP Paribas', 'benchmark siz..."
24,Twitter,,,,,,BNP Paribas,2021-02-24,User: Quant Data,,Barclays Upgrades BNP Paribas to Equal-Weight ...,0.01,0.96,0.03,neutral,Detected language is Unknown.,False,-,"'Barclays', 'BNP Paribas'-","'Equal-Weight Related Tickers', 'Barclays Upg..."
25,Twitter,,,,,,BNP Paribas,2021-02-24,User: Luxembourg City Film Festival,,2021 DOCUMENTARY JURY The winner of the DOCUM...,0.01,0.70,0.29,neutral,Detected language is Unknown.,False,-,"'BGL BNP PARIBAS', 'EUROPA'-","'DOCUMENTARY AWARD', 'DOCUMENTARY JURY', 'BGL ..."
26,Twitter,,,,,,BNP Paribas,2021-02-24,User: Shifting Wealth,,BNP Paribas upgraded to Equal Weight from Unde...,0.66,0.13,0.21,negative,Detected language is Unknown.,False,-,"'Barclays', 'BNP Paribas'-","'Equal Weight', 'Barclays', 'BNP Paribas', 'BN..."
47,Twitter,,,,,,Citigroup,2021-02-24,User: Al Sosa,,Bloomberg: HSBC and Citigroup: Global Retail B...,0.01,0.97,0.02,neutral,Detected language is Unknown.,False,-,"'Bloomberg', 'HSBC', 'Citigroup'-","'Global Retail Banking', 'HSBC', 'Citigroup', ..."
48,Twitter,,,,,,Citigroup,2021-02-24,User: That Writer Guy,,Citigroup is being sued by a landowner who wan...,0.96,0.00,0.04,negative,Detected language is Unknown.,False,-,"'Citigroup', 'Superfund'-","'Superfund site', 'finance giant', 'landowner'..."


In [32]:
# url = 'https://video.capital.fr/toutes-les-videos/nord-une-soixantaine-de-plaintes-pour-des-pannes-apres-un-passage-a-une-station-service_9770'

# client = authenticate_azure_textanalytics_client()
# article = Article(url)
# article.download()
# article.parse()
# article.nlp()

# title = article.title
# text = article.text
# summary = article.summary
# date = article.publish_date

# title_sentiment, title_pos, title_neu, title_neg = getAzureSentiment(title, client)
# text_sentiment, text_pos, text_neu, text_neg = getAzureSentiment(text, client)

In [37]:
ta_client = authenticate_azure_textanalytics_client()

def get_text_topics(client, text):
    """"""
    text = text[:5120]
    document = [text]

    try:
        response = text_analytics_client.extract_key_phrases(document)
        result = [doc for doc in response if not doc.is_error]
        topics = result[0].key_phrases
        topics = [topic.lower() for topic in topics]
    except: 
        topics = []
    topics = str(topics)[1:-1]
    return topics
def language_detection_example(text, client):
    try:
        documents = [text[:5120]]
        response = client.detect_language(documents = documents)[0]
        language = response.primary_language.name
        return language
    except Exception as err:
        return 'Unknown'

verified_col = []
verified_flg_col = []

verified = '' 
verified_flg = True

# Check if Article is actually about searchword
title_topic_list = get_text_topics(ta_client, row.Title)
if (searchword.lower() not in title_topic_list):
    verified += searchword + ' is not subject of the title.'
    verified_flg = False

In [44]:
document = [text]


response = ta_client.extract_key_phrases(document)
result = [doc for doc in response if not doc.is_error]
# topics = result[0].key_phrases
# topics = [topic.lower() for topic in topics]

# topics = str(topics)[1:-1]

In [50]:
# BNP Paribas
row = cpt_df.iloc[9]
row.Link

'https://investors-corner.bnpparibas-am.com/investing/introducing-new-esg-and-carbon-footprint-reporting/'

# Concept, Bugs, Backlog

__DF__  
api link date title article summary media | titlesentiment (positive, neutral negative) | articlesentiment (positive, neutral negative)

tweet date account | sentiment (positive, neutral negative)

- __Enhance Sentiment Analysis__ 

- Azure Sentiment Analysis failed on: 
    - Google: http://citywireselector.com/news/bnp-paribas-am-revamps-thematic-fund-to-tackle-inequalities/a1470113?ref=citywire_global_latest_news_list
    - https://www.dailytelegraph.com.au/business/citigroup-examines-future-of-global-consumer-businesses/news-story/f26fb2b22b858ec7b96940459ce655fa
    - https://www.theaustralian.com.au/business/global-petition-to-help-third-world-countries-get-medical-resources-to-fight-covid/news-story/04cc094181ab3260ceaeefd660884d42
- Fix Twitter
- Add Links to scrapy search urls
    - ['https://www.vindobona.org/article/vienna-stock-exchange-new-calculation-technology-increases-quality',
 'https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/']
    - Scrapy Last in Search and always activated
- Change Scrapy FT searchsite
- FT Subscription?
- Make Scrapy repeatable
- Suppress Scrapy Log
- Backtest with Hertz
- Change NER and Topicsearch max limit to multisearch

## Backtesting Example

Hertz filed bankruptcy on the 22nd of May 2020.

## Temp

# Feature Tests

In [4]:
# Initialize DF
columns = ['API', 'Link', 'Date', 'Title', 'Article', 'Summary', 'Topics',
           'Found_Keywords', 'Overall_Sentiment', 'Verified', 
           'Title_Sentiment', 'Title_Positv', 'Title_Neutral', 'Title_Negativ',
           'Article_Sentiment', 'Article_Positv', 'Article_Neutral', 'Article_Negativ'
          ]
dff = pd.DataFrame(columns=columns)
searchword='EUREX'  
dff = add_bing_news(dff, searchword)
#dff = add_scrapy_news(dff, searchword)
message = ''

In [7]:
dff = df.copy()

## Azure Sentiment Analyzer

In [4]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

In [5]:
# Azure Cognitive Service Keys
cog_endpoint = 'https://eocjmcognitiveservicesconnection.cognitiveservices.azure.com/'
cog_key = '1b79872142ca4bf482381e946bbb6dc2'

text = "Bankrupt Hertz Is a Pandemic Zombie!"

In [6]:
# Azure Client
ta_credential = AzureKeyCredential(cog_key)
client = TextAnalyticsClient(
                endpoint=cog_endpoint, 
                credential=ta_credential)

In [8]:
documents = [text[:5120]] 
response = client.analyze_sentiment(documents = documents)[0]
document_sentiment = response.sentiment
document_sentiment

## Scrapy 

In [7]:
html = """<!DOCTYPE html><html lang="en-GB" class="no-js core o-typography--loading-sans o-typography--loading-sans-bold o-typography--loading-display o-typography--loading-display-bold" data-o-component="o-typography" style="overflow-x:hidden;background-color:#fff1e5;color:#33302e"><head><meta charSet="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=edge"/><meta name="viewport" content="width=device-width, initial-scale=1"/><title>bolsonaro | Search | Financial Times</title><meta name="description" content="News, analysis and comment from the Financial Times, the worldʼs leading global business publication"/><meta name="robots" content="index,follow,max-snippet:200,max-image-preview:large"/><meta name="google-site-verification" content="4-t8sFaPvpO5FH_Gnw1dkM28CQepjzo8UjjAkdDflTw"/><script type="application/ld+json">{"@context":"http://schema.org","@type":"WebSite","name":"Financial Times","alternateName":"FT.com","url":"http://www.ft.com"}</script><meta property="fb:pages" content="8860325749"/><meta property="twitter:site" content="@FinancialTimes"/><link rel="icon" type="image/svg+xml" href="https://www.ft.com/__origami/service/image/v2/images/raw/ftlogo-v1%3Abrand-ft-logo-square-coloured?source=update-logos&format=svg"/><link rel="alternate icon" type="image/png" href="https://www.ft.com/__origami/service/image/v2/images/raw/ftlogo-v1%3Abrand-ft-logo-square-coloured?source=update-logos&format=png&width=32&height=32" sizes="32x32"/><link rel="alternate icon" type="image/png" href="https://www.ft.com/__origami/service/image/v2/images/raw/ftlogo-v1%3Abrand-ft-logo-square-coloured?source=update-logos&format=png&width=194&height=194" sizes="194x194"/><link rel="apple-touch-icon" href="https://www.ft.com/__origami/service/image/v2/images/raw/ftlogo-v1%3Abrand-ft-logo-square-coloured?source=update-logos&format=png&width=180&height=180" sizes="180x180"/><link rel="manifest" href="https://www.ft.com/__assets/creatives/manifest/manifest-v6.json"/><link rel="preconnect" href="https://spoor-api.ft.com"/><link rel="preconnect" href="https://session-next.ft.com" crossorigin="use-credentials"/><link rel="preconnect" href="https://ads-api.ft.com"/><link rel="preconnect" href="https://securepubads.g.doubleclick.net"/><link rel="preload" as="script" href="https://polyfill.io/v3/polyfill.min.js?features=default%2Ces5%2Ces2015%2Ces2016%2Ces2017%2CEventSource%2Cfetch%2CHTMLPictureElement%2CIntersectionObserver%2CNodeList.prototype.forEach&source=next"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/webpack-runtime.fb5e068cad11.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/page-kit-components.1513323d75c9.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/n-myft-ui.78a50e3b14f3.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/n-syndication.6e7d857310aa.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/next-myft-client.02db23fe8243.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-header.420bcf0da422.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/n-feedback.660bffa3ce99.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-forms.250ed6a82193.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/n-notification.84cea8e6e685.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/n-ui-foundations.1b0df1266cc8.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/n-topic-search.0c43cda5a57d.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/next-session-client.a488dae6869c.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-footer.57a8408c9b7e.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-overlay.7b4d116df926.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-toggle.5a3270c518f4.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-tooltip.0796fdb88c18.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-lazy-load.4de28350ea06.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-typography.ce9b4ffdce4b.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-viewport.065b502163a9.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/superstore.0c70d59fdcef.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-grid.80dbe6cd6af0.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-layers.1f9711dad6b6.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-utils.ebe34c02de08.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/financial-times-o-tracking.48d47e20d695.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/financial-times-o-ads.0d30a8c04dfc.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/shared.stable.4968dd492964.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/financial-times-o-viewport.407fd17a304c.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/financial-times-n-tracking.7ccb42af2e1a.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/financial-times-o-grid.8e531e66a66a.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/financial-times-o-utils.039203628d34.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/regenerator-runtime.46f0c42efcfc.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/scripts.43dd2b34f27e.bundle.js" type="application/javascript"/><link rel="preload" as="font" href="https://www.ft.com/__origami/service/build/v2/files/o-fonts-assets@1.5.0/MetricWeb-Regular.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="preload" as="font" href="https://www.ft.com/__origami/service/build/v2/files/o-fonts-assets@1.5.0/MetricWeb-Semibold.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="preload" as="font" href="https://www.ft.com/__origami/service/build/v2/files/o-fonts-assets@1.5.0/FinancierDisplayWeb-Regular.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="preload" as="font" href="https://www.ft.com/__origami/service/build/v2/files/o-fonts-assets@1.5.0/FinancierDisplayWeb-Bold.woff2" type="font/woff2" crossorigin="anonymous"/><script id="initial-props" type="application/json"></script><link rel="stylesheet" href="https://www.ft.com/__assets/hashed/page-kit/vendors~page-kit-layout-styles.573f9dcf07e4.css"/><link rel="stylesheet" href="https://www.ft.com/__assets/hashed/page-kit/styles.69c84ceec394.css"/><script type="application/json" id="page-kit-bootstrap-config">{"trackErrors":true,"core":["https://polyfill.io/v3/polyfill.min.js?features=HTMLPictureElement&source=next"],"enhanced":["https://polyfill.io/v3/polyfill.min.js?features=default%2Ces5%2Ces2015%2Ces2016%2Ces2017%2CEventSource%2Cfetch%2CHTMLPictureElement%2CIntersectionObserver%2CNodeList.prototype.forEach&source=next","https://www.ft.com/__assets/hashed/page-kit/webpack-runtime.fb5e068cad11.bundle.js","https://www.ft.com/__assets/hashed/page-kit/page-kit-components.1513323d75c9.bundle.js","https://www.ft.com/__assets/hashed/page-kit/n-myft-ui.78a50e3b14f3.bundle.js","https://www.ft.com/__assets/hashed/page-kit/n-syndication.6e7d857310aa.bundle.js","https://www.ft.com/__assets/hashed/page-kit/next-myft-client.02db23fe8243.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-header.420bcf0da422.bundle.js","https://www.ft.com/__assets/hashed/page-kit/n-feedback.660bffa3ce99.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-forms.250ed6a82193.bundle.js","https://www.ft.com/__assets/hashed/page-kit/n-notification.84cea8e6e685.bundle.js","https://www.ft.com/__assets/hashed/page-kit/n-ui-foundations.1b0df1266cc8.bundle.js","https://www.ft.com/__assets/hashed/page-kit/n-topic-search.0c43cda5a57d.bundle.js","https://www.ft.com/__assets/hashed/page-kit/next-session-client.a488dae6869c.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-footer.57a8408c9b7e.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-overlay.7b4d116df926.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-toggle.5a3270c518f4.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-tooltip.0796fdb88c18.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-lazy-load.4de28350ea06.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-typography.ce9b4ffdce4b.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-viewport.065b502163a9.bundle.js","https://www.ft.com/__assets/hashed/page-kit/superstore.0c70d59fdcef.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-grid.80dbe6cd6af0.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-layers.1f9711dad6b6.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-utils.ebe34c02de08.bundle.js","https://www.ft.com/__assets/hashed/page-kit/financial-times-o-tracking.48d47e20d695.bundle.js","https://www.ft.com/__assets/hashed/page-kit/financial-times-o-ads.0d30a8c04dfc.bundle.js","https://www.ft.com/__assets/hashed/page-kit/shared.stable.4968dd492964.bundle.js","https://www.ft.com/__assets/hashed/page-kit/financial-times-o-viewport.407fd17a304c.bundle.js","https://www.ft.com/__assets/hashed/page-kit/financial-times-n-tracking.7ccb42af2e1a.bundle.js","https://www.ft.com/__assets/hashed/page-kit/financial-times-o-grid.8e531e66a66a.bundle.js","https://www.ft.com/__assets/hashed/page-kit/financial-times-o-utils.039203628d34.bundle.js","https://www.ft.com/__assets/hashed/page-kit/regenerator-runtime.46f0c42efcfc.bundle.js","https://www.ft.com/__assets/hashed/page-kit/scripts.43dd2b34f27e.bundle.js"]}</script><script>;(function () {
      var doc = document.documentElement
      var isEnhanced = isEnhancedBrowser()
      var scriptsConfig = getScriptsConfig()
      var scriptsToLoad = []
      var currentScript = document.scripts[document.scripts.length - 1]

      doc.className = doc.className.replace('no-js', 'js')

      if (isEnhanced) {
        doc.className = doc.className.replace('core', 'enhanced')
        Array.prototype.push.apply(scriptsToLoad, scriptsConfig.enhanced)
      } else {
        Array.prototype.push.apply(scriptsToLoad, scriptsConfig.core)
      }

      for (var i = 0, len = scriptsToLoad.length; i < len; i++) {
        loadScript(scriptsToLoad[i])
      }

      function scriptLoadError(error) {
        var script = error.target ? error.target.src : null

        if (script) {
          console.error('The script ' + script + ' failed to load') // eslint-disable-line no-console
        }

        if (/enhanced/.test(doc.className)) {
          console.warn('Script loading failed, reverting to core experience') // eslint-disable-line no-console
          doc.className = doc.className.replace('enhanced', 'core')
        }

        if (scriptsConfig.trackErrors) {
          addErrorTrackingPixel(script)
        }
      }

      function loadScript(src) {
        var script = document.createElement('script')
        script.onerror = scriptLoadError
        script.async = false
        script.src = src
        currentScript.parentNode.insertBefore(script, currentScript)
      }

      // "Cut the mustard" test
      // by Maggie Allen and Matt Hinchliffe November 2018
      function isEnhancedBrowser() {
        var script = document.createElement('script')
        var input = document.createElement('input')

        return (
          'visibilityState' in document && // not supported by old Android (4.0-4.4) without a prefix
          'indeterminate' in input && // not supported by BB 10
          'flex' in doc.style && // not supported by old Safari (< 9) or IE 6-10
          'async' in script // not supported by old Opera (Presto engine < 15)
        )
      }

      function getScriptsConfig() {
        var scriptsConfigEl = document.getElementById('page-kit-bootstrap-config')
        var scriptsConfig = { core: [], enhanced: [], trackErrors: false }

        if (scriptsConfigEl) {
          try {
            scriptsConfig = JSON.parse(scriptsConfigEl.innerHTML)
          } catch (error) {
            console.error('Bootstrap configuration error', error) // eslint-disable-line no-console
          }
        }

        return scriptsConfig
      }

      function addErrorTrackingPixel(script) {
        var img = new Image()

        var data = JSON.stringify({
          category: 'javascript',
          action: 'load-error',
          system: {
            source: 'page-kit'
          },
          context: {
            script: script
          }
        })

        img.src = 'https://spoor-api.ft.com/px.gif?data=' + encodeURIComponent(data)
      }
    })()
    </script><script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
              new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
                j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
                'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
            })(window,document,'script','dataLayer','GTM-NWQJW68');</script><script>(function loadCustomFonts() {
        var rootElement = document.documentElement;
        if (/(^|\s)o-typography-fonts-loaded=1(;|$)/.test(document.cookie)) {
            var fontLabels = ['sans', 'sans-bold', 'display', 'display-bold'];
            for (var i = 0; i < fontLabels.length; i++) {
                rootElement.className = rootElement.className.replace('o-typography--loading-' + fontLabels[i], '');
            }
        }
    }());</script></head><body><noscript><iframe src="https://www.googletagmanager.com/ns.html?id=GTM-NWQJW68" height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript><div class="n-layout"><a data-trackable="a11y-skip-to-help" class="n-layout__skip-link" href="https://www.ft.com/accessibility">Accessibility help</a><a data-trackable="a11y-skip-to-navigation" class="n-layout__skip-link" href="#site-navigation">Skip to navigation</a><a data-trackable="a11y-skip-to-content" class="n-layout__skip-link" href="#site-content">Skip to content</a><a data-trackable="a11y-skip-to-footer" class="n-layout__skip-link" href="#site-footer">Skip to footer</a><div class="n-layout__row n-layout__row--header"><div class="n-layout__header-before"><script type="application/json" id="n-ads-options">{}</script><div class="o-ads o-ads--center " data-o-ads-name="leaderboard" data-o-ads-targeting="pos=top" data-o-ads-formats-large="SuperLeaderboard,Leaderboard,Responsive" data-o-ads-formats-extra="Billboard,SuperLeaderboard,Leaderboard,Responsive" style="width:100%;text-align:center"></div></div><header id="site-navigation" class="o-header o-header--simple" data-o-component="o-header" data-o-header--no-js="true" tabindex="-1"><div class="o-header__row o-header__anon" data-trackable="header-anon"><ul class="o-header__anon-list"><li class="o-header__anon-item"><a class="o-header__anon-link" href="/login?location=/search" data-trackable="Sign In">Sign In</a></li><li class="o-header__anon-item"><a class="o-header__anon-link" href="/products?segmentId=f860e6c2-18af-ab30-cd5e-6e3a456f9265" data-trackable="Subscribe">Subscribe</a></li></ul></div><div class="o-header__row o-header__top" data-trackable="header-top"><div class="o-header__container"><div class="o-header__top-wrapper"><div class="o-header__top-column o-header__top-column--left"><a href="#o-header-drawer" class="o-header__top-link o-header__top-link--menu" aria-controls="o-header-drawer" title="Open side navigation menu" data-trackable="drawer-toggle"><span class="o-header__top-link-label">Open side navigation menu</span></a><a href="#o-header-search-primary" class="o-header__top-link o-header__top-link--search" aria-controls="o-header-search-primary" title="Open search bar" data-trackable="search-toggle"><span class="o-header__top-link-label">Open search bar</span></a></div><div class="o-header__top-column o-header__top-column--center"><a class="o-header__top-logo" style="background-image:none" data-trackable="logo" href="/" title="Go to Financial Times homepage"><svg viewBox="0 0 1054 86"><title>Financial Times</title><path d="M26.177 72.609c0 5.938 1.697 7.295 12.554 7.295v3.732H.9v-3.732c7.464 0 10.01-.679 10.01-7.125V12.215c0-6.447-2.546-7.126-10.01-7.126V1.357h63.448l.34 22.563h-3.054C59.937 6.956 55.696 5.43 39.919 5.43H26.008v33.59h11.196c10.688 0 11.367-1.697 12.215-10.179h3.054v24.599h-3.054c-.848-8.482-1.527-10.01-12.215-10.01H26.008v29.18h.17zm46.314 11.027v-3.732c7.465 0 10.01-.679 10.01-7.125V12.215c0-6.447-2.545-7.126-10.01-7.126V1.357h35.287V5.09c-7.465 0-10.01.679-10.01 7.126v60.564c0 6.446 2.545 7.125 10.01 7.125v3.732H72.49zm115.36 1.357l-56.323-69.725V72.44c0 6.617 4.58 7.634 12.385 7.634v3.733h-29.858v-3.733c7.803 0 12.045-1.017 12.045-7.634V8.991c-3.563-3.732-6.108-3.902-12.045-3.902V1.357h26.465l43.09 55.475V12.554c0-6.616-4.58-7.634-12.384-7.634V1.357h30.027V5.09c-7.803 0-12.045 1.018-12.045 7.635v72.27h-1.357zm40.207-1.357h-29.689v-3.732c7.804 0 11.367-1.018 13.911-7.804L239.085.509h7.464l28.84 72.1c2.545 6.447 3.732 7.125 9.67 7.125v3.732h-34.438v-3.732c10.518 0 11.536-.848 8.99-7.125l-8.481-21.884h-25.787L217.71 71.93c-2.375 6.447 1.357 7.804 10.518 7.804v3.902h-.17zm-1.188-37.153h22.393l-11.705-29.518-10.688 29.518zm135.548 38.51l-56.153-69.725V72.44c0 6.617 4.58 7.634 12.384 7.634v3.733h-29.18v-3.733c7.126 0 11.367-1.017 11.367-7.634V9.161c-4.071-3.732-7.125-4.072-13.91-4.072V1.357h28.16l43.09 55.475V12.554c0-6.616-4.58-7.634-12.383-7.634V1.357h29.858V5.09c-7.804 0-12.045 1.018-12.045 7.635v72.27h-1.188zm83.297-83.805h2.036l1.187 24.598-3.053.17c-2.036-14.08-9.5-21.545-23.242-21.545-15.268 0-26.804 13.063-26.804 33.081 0 25.617 16.116 40.206 33.08 40.206 7.296 0 13.912-2.035 20.358-8.99l2.376 2.374c-5.26 7.465-15.608 13.742-29.52 13.742-20.696 0-41.732-15.608-41.732-41.734C380.4 17.813 399.57 0 422.813 0c11.027 0 16.795 4.75 19.848 4.75 1.357 0 2.375-1.187 3.054-3.562zm12.723 82.448v-3.732c7.465 0 10.01-.679 10.01-7.125V12.215c0-6.447-2.545-7.126-10.01-7.126V1.357h35.287V5.09c-7.464 0-10.01.679-10.01 7.126v60.564c0 6.446 2.546 7.125 10.01 7.125v3.732h-35.287zm68.538 0h-27.653v-3.732c6.108 0 9.331-1.018 11.876-7.804L538.003.509h7.464l28.84 72.1c2.545 6.447 3.733 7.125 9.67 7.125v3.732H549.54v-3.732c10.518 0 11.536-.848 8.991-7.125l-8.482-21.884h-25.786l-7.635 21.205c-2.375 6.447 1.358 7.804 10.519 7.804v3.902h-.17zm-1.188-37.153h22.394l-11.536-29.518-10.858 29.518zm63.957 37.153v-3.732c7.465 0 10.01-.679 10.01-7.125V12.215c0-6.447-2.545-7.126-10.01-7.126V1.357h35.117V5.09c-7.464 0-9.84.679-9.84 7.126v61.073c0 5.428 2.715 6.107 7.126 6.107h4.241c15.947 0 21.036-2.375 25.447-20.527l3.054.339-2.545 24.26h-62.6v.17zM760.75 1.357l.339 23.92h-3.054C756.34 7.634 752.098 5.43 736.32 5.43h-5.089v67.18c0 6.447 2.375 7.295 12.554 7.295v3.732h-40.376v-3.732c10.179 0 12.723-1.018 12.723-7.295V5.429h-5.089c-15.777 0-20.018 2.205-21.715 19.848h-3.053l.339-23.92h74.136zm7.973 82.28v-3.733c7.465 0 10.01-.679 10.01-7.125V12.215c0-6.447-2.545-7.126-10.01-7.126V1.357h35.287V5.09c-7.465 0-10.01.679-10.01 7.126v60.564c0 6.446 2.545 7.125 10.01 7.125v3.732h-35.287zM910.21 1.356V5.09c-7.465 0-10.688.34-10.01 6.956l6.447 61.073c.679 6.277 3.054 6.955 10.518 6.955v3.733h-35.117v-3.733c7.295 0 9.84-.678 9.331-6.955L884.762 8.99l-25.956 76.172h-1.018L832.34 8.822l-6.108 64.126c-.678 6.447 3.733 7.125 11.197 7.125v3.733h-27.483v-3.733c7.465 0 10.01-1.187 10.518-7.125l6.447-61.073c.679-6.446-2.545-6.955-10.01-6.955V1.357h28.84l17.305 56.153 18.661-56.153h28.5zm65.653 52.082h-3.053c-.849-8.482-1.527-10.01-12.215-10.01H948.04v29.859c0 5.428 2.715 6.107 7.125 6.107h6.786c15.947 0 21.036-2.375 25.447-20.527l3.054.339-2.884 24.26h-64.805v-3.733c7.464 0 10.009-.678 10.009-7.125V12.215c0-6.447-2.545-7.126-10.01-7.126V1.357h62.261l.34 20.527h-3.054c-1.866-14.59-5.598-16.286-21.885-16.286H948.21v33.42h12.554c10.687 0 11.366-1.696 12.214-10.178h3.054v24.599h-.17zm65.484 13.232c0-7.464-4.75-11.196-12.893-15.777l-13.063-6.786c-9.84-5.259-15.607-11.027-15.607-21.375C999.783 9.84 1010.81 0 1025.23 0c9.84 0 14.929 4.75 17.813 4.75 1.866 0 2.714-1.187 3.562-3.562h2.375l1.188 23.072-3.054.17c-1.696-11.198-9.67-19.85-20.866-19.85-8.483 0-14.081 5.09-14.081 12.215 0 7.804 5.937 11.027 12.554 14.59l11.196 5.937c10.519 5.768 17.983 11.536 17.983 22.563 0 14.59-12.554 24.939-28.161 24.939-11.028 0-16.456-5.26-19.34-5.26-1.866 0-2.884 1.697-3.732 4.242h-2.376l-1.696-24.43 3.054-.339c2.375 15.268 12.893 21.545 23.411 21.545 8.822-.17 16.286-4.071 16.286-13.91z" fill="#231F20" fill-rule="evenodd"></path></svg></a></div><div class="o-header__top-column o-header__top-column--right"><a class="o-header__top-link o-header__top-link--myft" href="/myft" data-trackable="my-ft" data-tour-stage="myFt" aria-label="My F T"><span class="o-header__visually-hidden">myFT</span></a></div></div></div></div><div id="o-header-search-primary" class="o-header__row o-header__search o-header__search--primary" data-trackable="header-search" data-o-header-search="true"><div class="o-header__container"><form class="o-header__search-form" action="/search" role="search" aria-label="Site search" data-n-topic-search="true" data-n-topic-search-categories="concepts,equities" data-n-topic-search-view-all="true"><label class="o-header__visually-hidden" for="o-header-search-term-primary">Search the <abbr title="Financial Times">FT</abbr></label><input type="text" class="o-header__search-term" id="o-header-search-term-primary" name="q" autoComplete="off" autoCorrect="off" autoCapitalize="off" spellcheck="false" data-trackable="search-term" placeholder="Search the FT" data-n-topic-search-input="true"/><button class="o-header__search-submit" type="submit" data-trackable="search-submit">Search</button><button class="o-header__search-close o--if-js" type="button" aria-controls="o-header-search-primary" title="Close search bar" data-trackable="close"><span class="o-header__visually-hidden">Close search bar</span></button></form></div></div><nav id="o-header-nav-desktop" class="o-header__row o-header__nav o-header__nav--desktop" role="navigation" aria-label="Primary navigation" data-trackable="header-nav:desktop"><div class="o-header__container"><ul class="o-header__nav-list o-header__nav-list--left" data-trackable="primary-nav"><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/" id="o-header-link-0" data-trackable="Home">Home</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/world" id="o-header-link-1" data-trackable="World">World</a><div class="o-header__mega" id="o-header-mega-1" role="group" aria-labelledby="o-header-link-1" data-o-header-mega="true" data-trackable="meganav | World"><div class="o-header__container"><div class="o-header__mega-wrapper"><div class="o-header__mega-column o-header__mega-column--subsections" data-trackable="sections"><div class="o-header__mega-heading">Sections</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world" data-trackable="link">World Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/global-economy" data-trackable="link">Global Economy</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/uk" data-trackable="link">UK</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/us" data-trackable="link">US</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/asia-pacific/china" data-trackable="link">China</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/africa" data-trackable="link">Africa</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/asia-pacific" data-trackable="link">Asia Pacific</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/emerging-markets" data-trackable="link">Emerging Markets</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/europe" data-trackable="link">Europe</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/americas" data-trackable="link">Americas</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/mideast" data-trackable="link">Middle East and North Africa</a></li></ul></div></div><div class="o-header__mega-column o-header__mega-column--articles" data-trackable="popular"><div class="o-header__mega-heading">Most Read</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/6f6235b5-624c-4ce3-9807-42e2dc548306" data-trackable="link">Johnson puts final touches to cautious easing of England lockdown</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/c837135e-60be-4ec6-ac14-78d500775222" data-trackable="link">How EU migrant exodus is reshaping communities across the UK</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/7b0db882-a369-4e32-a86a-eb7fda2a0da0" data-trackable="link">Racial inequality plagues US vaccine rollout</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/919d16c8-0a40-4389-bac9-04a48fdb0a36" data-trackable="link">BioNTech/Pfizer Covid vaccine no longer needs ultra-cold storage</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/dc74a4fc-9b24-4e77-a8dd-c055f5e0c884" data-trackable="link">Climate graphic of the week: Polar vortex sends Texas into deep freeze</a></li></ul></div></div></div></div></div></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/world/us" id="o-header-link-2" data-trackable="US">US</a><div class="o-header__mega" id="o-header-mega-2" role="group" aria-labelledby="o-header-link-2" data-o-header-mega="true" data-trackable="meganav | US"><div class="o-header__container"><div class="o-header__mega-wrapper"><div class="o-header__mega-column o-header__mega-column--subsections" data-trackable="sections"><div class="o-header__mega-heading">Sections</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/us" data-trackable="link">US Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/us-economy" data-trackable="link">US Economy</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/us" data-trackable="link">US Companies</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/us/politics" data-trackable="link">US Politics & Policy</a></li></ul></div></div><div class="o-header__mega-column o-header__mega-column--articles" data-trackable="popular"><div class="o-header__mega-heading">Most Read</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/afc414f9-c6a1-4f37-afab-276d98973a09" data-trackable="link">Asset managers rush to shore up portfolios against inflation </a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/0e746280-e72c-4087-9c0d-df2a7af82b77" data-trackable="link">Bills mount in Texas power market after freeze sends prices soaring</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/4fbebc3a-d714-4d79-9805-613303a29420" data-trackable="link">Google fires top AI ethicist</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/0a3d0af8-7092-44c3-9c98-a513a22629be" data-trackable="link">‘Green bubble’ warnings grow as money pours into renewable stocks </a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/7b0db882-a369-4e32-a86a-eb7fda2a0da0" data-trackable="link">Racial inequality plagues US vaccine rollout</a></li></ul></div></div></div></div></div></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/companies" id="o-header-link-3" data-trackable="Companies">Companies</a><div class="o-header__mega" id="o-header-mega-3" role="group" aria-labelledby="o-header-link-3" data-o-header-mega="true" data-trackable="meganav | Companies"><div class="o-header__container"><div class="o-header__mega-wrapper"><div class="o-header__mega-column o-header__mega-column--subsections" data-trackable="sections"><div class="o-header__mega-heading">Sections</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies" data-trackable="link">Companies Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/energy" data-trackable="link">Energy</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/financials" data-trackable="link">Financials</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/health" data-trackable="link">Health</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/industrials" data-trackable="link">Industrials</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/media" data-trackable="link">Media</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/professional-services" data-trackable="link">Professional Services</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/retail-consumer" data-trackable="link">Retail & Consumer</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/technology" data-trackable="link">Tech Sector</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/telecoms" data-trackable="link">Telecoms</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/transport" data-trackable="link">Transport</a></li></ul></div></div><div class="o-header__mega-column o-header__mega-column--articles" data-trackable="popular"><div class="o-header__mega-heading">Most Read</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/d02d4b14-30c5-4667-a21b-c436b2446ca8" data-trackable="link">Europe’s investment bankers fend off bleak backdrop to scoop up bonuses</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/0e746280-e72c-4087-9c0d-df2a7af82b77" data-trackable="link">Bills mount in Texas power market after freeze sends prices soaring</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/38c3670c-3b0f-41e6-874e-0f9eee553744" data-trackable="link">HSBC intensifies pivot to Asia with job moves and US exit</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/b7919190-7f08-4717-ad86-11fba03bd519" data-trackable="link">Centrica chief vows to ‘strip out the rubbish’ to revive group’s fortunes</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/0934a38d-8177-4579-b6cb-ef70eecfe578" data-trackable="link">Former Bank of England governor Mark Carney joins Stripe board</a></li></ul></div></div></div></div></div></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/technology" id="o-header-link-4" data-trackable="Tech">Tech</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/markets" id="o-header-link-5" data-trackable="Markets">Markets</a><div class="o-header__mega" id="o-header-mega-5" role="group" aria-labelledby="o-header-link-5" data-o-header-mega="true" data-trackable="meganav | Markets"><div class="o-header__container"><div class="o-header__mega-wrapper"><div class="o-header__mega-column o-header__mega-column--subsections" data-trackable="sections"><div class="o-header__mega-heading">Sections</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/markets" data-trackable="link">Markets Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/alphaville" data-trackable="link">Alphaville</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="https://markets.ft.com/data" data-trackable="link">Markets Data</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/capital-markets" data-trackable="link">Capital Markets</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/commodities" data-trackable="link">Commodities</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/currencies" data-trackable="link">Currencies</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/equities" data-trackable="link">Equities</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/fund-management" data-trackable="link">Fund Management</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/ft-trading-room" data-trackable="link">Trading</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/moral-money" data-trackable="link">Moral Money</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="https://etf.ft.com" data-trackable="link">ETF Hub</a></li></ul></div></div><div class="o-header__mega-column o-header__mega-column--articles" data-trackable="popular"><div class="o-header__mega-heading">Most Read</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/da5c30a8-6fab-4131-b6bd-f8f05dcf5a46" data-trackable="link">UK competition watchdog warns Big Tech of coming antitrust probes</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/683effc4-993a-4baf-bc17-8ba70b96c06a" data-trackable="link">Bond trading finally dragged into the digital age</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/0e746280-e72c-4087-9c0d-df2a7af82b77" data-trackable="link">Bills mount in Texas power market after freeze sends prices soaring</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/84012e15-554c-485c-8064-dbc6b4098bca" data-trackable="link">Will inflation data justify investors’ concerns?</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/054d7d05-ed69-445b-bfa3-c64397d5fa74" data-trackable="link">Should equity investors worry about rising interest rates?</a></li></ul></div></div></div></div></div></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/climate-capital" id="o-header-link-6" data-trackable="Climate">Climate</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/opinion" id="o-header-link-7" data-trackable="Opinion">Opinion</a><div class="o-header__mega" id="o-header-mega-7" role="group" aria-labelledby="o-header-link-7" data-o-header-mega="true" data-trackable="meganav | Opinion"><div class="o-header__container"><div class="o-header__mega-wrapper"><div class="o-header__mega-column o-header__mega-column--subsections" data-trackable="sections"><div class="o-header__mega-heading">Sections</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/opinion" data-trackable="link">Opinion Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/columnists" data-trackable="link">Columnists</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/ft-view" data-trackable="link">The FT View</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/the-big-read" data-trackable="link">The Big Read</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/lex" data-trackable="link">Lex</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/obituaries" data-trackable="link">Obituaries</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/letters" data-trackable="link">Letters</a></li></ul></div></div><div class="o-header__mega-column o-header__mega-column--articles" data-trackable="popular"><div class="o-header__mega-heading">Most Read</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/f5c833f1-22d4-4d4e-81e2-b0ee67743909" data-trackable="link">Brexit hits bordeaux buffs</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/ebd22d66-c4bb-47a0-8a34-e3cf8e0472b4" data-trackable="link">Toxic offices don’t change when they go virtual</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/217f6d28-5a3e-48e0-bf6e-c2618da8f34b" data-trackable="link">Why once successful countries like the UK get left behind</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/97441795-4701-4473-8230-1cf62114655b" data-trackable="link">Britney Spears: our part in her downfall</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/054d7d05-ed69-445b-bfa3-c64397d5fa74" data-trackable="link">Should equity investors worry about rising interest rates?</a></li></ul></div></div></div></div></div></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/work-careers" id="o-header-link-8" data-trackable="Work & Careers">Work & Careers</a><div class="o-header__mega" id="o-header-mega-8" role="group" aria-labelledby="o-header-link-8" data-o-header-mega="true" data-trackable="meganav | Work & Careers"><div class="o-header__container"><div class="o-header__mega-wrapper"><div class="o-header__mega-column o-header__mega-column--subsections" data-trackable="sections"><div class="o-header__mega-heading">Sections</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/work-careers" data-trackable="link">Work & Careers Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="https://rankings.ft.com/businessschoolrankings/rankings" data-trackable="link">Business School Rankings</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/business-education" data-trackable="link">Business Education</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/entrepreneurship" data-trackable="link">Entrepreneurship</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/recruitment" data-trackable="link">Recruitment</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/business-books" data-trackable="link">Business Books</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/business-travel" data-trackable="link">Business Travel</a></li></ul></div></div><div class="o-header__mega-column o-header__mega-column--articles" data-trackable="popular"><div class="o-header__mega-heading">Most Read</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/ebd22d66-c4bb-47a0-8a34-e3cf8e0472b4" data-trackable="link">Toxic offices don’t change when they go virtual</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/1e07c7fd-b0bf-405d-ac4a-979990b2a7da" data-trackable="link">Delta’s Ed Bastian: building staff loyalty to survive a crisis</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/8025b36e-7f70-4072-97c7-3fde860883cf" data-trackable="link">The boardroom’s futile pursuit of purpose</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/ea4ff3d0-cbaa-4197-b2b8-6e92b1a822c3" data-trackable="link">How do I develop professionally while on maternity leave?</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/5c6742c6-b101-4bdb-9244-6d719460ec2e" data-trackable="link">Forced mediocrity can boost your creativity</a></li></ul></div></div></div></div></div></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/life-arts" id="o-header-link-9" data-trackable="Life & Arts">Life & Arts</a><div class="o-header__mega" id="o-header-mega-9" role="group" aria-labelledby="o-header-link-9" data-o-header-mega="true" data-trackable="meganav | Life & Arts"><div class="o-header__container"><div class="o-header__mega-wrapper"><div class="o-header__mega-column o-header__mega-column--subsections" data-trackable="sections"><div class="o-header__mega-heading">Sections</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/life-arts" data-trackable="link">Life & Arts Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/arts" data-trackable="link">Arts</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/books" data-trackable="link">Books</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/food-drink" data-trackable="link">Food & Drink</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/magazine" data-trackable="link">FT Magazine</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/house-home" data-trackable="link">House & Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/style" data-trackable="link">Style</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/travel" data-trackable="link">Travel</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/globetrotter" data-trackable="link">FT Globetrotter</a></li></ul></div></div><div class="o-header__mega-column o-header__mega-column--articles" data-trackable="popular"><div class="o-header__mega-heading">Most Read</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/c11bb885-1274-4677-ba05-fcbac67dc808" data-trackable="link">Bill Gates: My green manifesto</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/97441795-4701-4473-8230-1cf62114655b" data-trackable="link">Britney Spears: our part in her downfall</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/f5c833f1-22d4-4d4e-81e2-b0ee67743909" data-trackable="link">Brexit hits bordeaux buffs</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/468d42c9-c107-4518-9716-ca841076af30" data-trackable="link">Line of fire: what it is to be a man in today’s British army</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/1ca7a45d-57a4-4ca3-9130-84e109ed1ad7" data-trackable="link">Four hands-free cameras for epic content creation</a></li></ul></div></div></div></div></div></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/htsi" id="o-header-link-10" data-trackable="How to Spend It">How to Spend It</a></li></ul><ul class="o-header__nav-list o-header__nav-list--right" data-trackable="user-nav"><li class="o-header__nav-item"><a class="o-header__nav-link" href="/login?location=/search" data-trackable="Sign In">Sign In</a></li><li class="o-header__nav-item o-header__nav-item--hide-s"><a class="o-header__nav-button" role="button" href="/products?segmentId=f860e6c2-18af-ab30-cd5e-6e3a456f9265" data-trackable="Subscribe">Subscribe</a></li></ul></div></nav></header><header class="o-header o-header--simple o-header--sticky o--if-js" data-o-component="o-header" data-o-header--sticky="true" aria-hidden="true"><div class="o-header__row o-header__top" data-trackable="header-sticky"><div class="o-header__container"><div class="o-header__top-wrapper"><div class="o-header__top-column o-header__top-column--left"><a href="#" class="o-header__top-link o-header__top-link--menu" aria-controls="o-header-drawer" data-trackable="drawer-toggle" tabindex="-1"><span class="o-header__top-link-label">Menu</span></a><a href="#" class="o-header__top-link o-header__top-link--search" aria-controls="o-header-search-sticky" data-trackable="search-toggle" tabindex="-1"><span class="o-header__top-link-label">Search</span></a></div><div class="o-header__top-column o-header__top-column--center"><div class="o-header__top-takeover"><div class="o-header__nav"><ul class="o-header__nav-list o-header__nav-list--left" data-trackable="primary-nav"><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/" data-trackable="Home" tabindex="-1">Home</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/world" data-trackable="World" tabindex="-1">World</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/world/us" data-trackable="US" tabindex="-1">US</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/companies" data-trackable="Companies" tabindex="-1">Companies</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/technology" data-trackable="Tech" tabindex="-1">Tech</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/markets" data-trackable="Markets" tabindex="-1">Markets</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/climate-capital" data-trackable="Climate" tabindex="-1">Climate</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/opinion" data-trackable="Opinion" tabindex="-1">Opinion</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/work-careers" data-trackable="Work & Careers" tabindex="-1">Work & Careers</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/life-arts" data-trackable="Life & Arts" tabindex="-1">Life & Arts</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/htsi" data-trackable="How to Spend It" tabindex="-1">How to Spend It</a></li></ul></div></div><a class="o-header__top-logo o-header__hide--L" data-trackable="logo" href="/" title="Go to Financial Times homepage" tabindex="-1"><span class="o-header__visually-hidden">Financial Times</span></a></div><div class="o-header__top-column o-header__top-column--right"><div class="o-header__nav"><ul class="o-header__nav-list o-header__nav-list--right" data-trackable="user-nav"><li class="o-header__nav-item"><a class="o-header__nav-link" href="/login?location=/search" data-trackable="Sign In" tabindex="-1">Sign In</a></li><li class="o-header__nav-item o-header__nav-item--hide-s"><a class="o-header__nav-button" role="button" href="/products?segmentId=f860e6c2-18af-ab30-cd5e-6e3a456f9265" data-trackable="Subscribe" tabindex="-1">Subscribe</a></li></ul></div></div></div></div></div><div id="o-header-search-sticky" class="o-header__row o-header__search o-header__search--sticky" data-trackable="header-search" data-o-header-search="true"><div class="o-header__container"><form class="o-header__search-form" action="/search" role="search" aria-label="Site search" data-n-topic-search="true" data-n-topic-search-categories="concepts,equities" data-n-topic-search-view-all="true"><label class="o-header__visually-hidden" for="o-header-search-term-sticky">Search the <abbr title="Financial Times">FT</abbr></label><input type="text" class="o-header__search-term" id="o-header-search-term-sticky" name="q" autoComplete="off" autoCorrect="off" autoCapitalize="off" spellcheck="false" data-trackable="search-term" placeholder="Search the FT" data-n-topic-search-input="true"/><button class="o-header__search-submit" type="submit" data-trackable="search-submit">Search</button><button class="o-header__search-close o--if-js" type="button" aria-controls="o-header-search-sticky" title="Close search bar" data-trackable="close"><span class="o-header__visually-hidden">Close search bar</span></button></form></div></div></header></div><div class="n-layout__row n-layout__row--content"><div><div class="o-grid-container">

        <script id="js-search-status" type="application/json">
            {"searchQuery":"bolsonaro","searchResults":3,"searchPage":1,"searchSort":"relevance","searchMode":"quality","activeContentTypeFilter":true,"activeConceptFilter":false,"activeDateFilter":true,"hasRelatedTopics":false,"hasRelatedLinks":false,"hasSuggestions":false}
        </script>

        <div class="search-layout ">

            <header class="search-layout__header">
                <h1 class="search-layout__title">Search Results</h1>
                <div class="search-head" data-trackable="search-head">
                    <form class="search-head__form" role="search" data-trackable="search-form">
                        <input type="hidden" name="sort" value="relevance">
                        <label class="search-head__form-label o-forms-field o-forms-field--optional">
                            <span class="search-head__form-title o-forms-title o-normalise-visually-hidden">
                                <span class="search-head__form-title-main o-forms-title__main">Query</span>
                            </span>
                            <span class="search-head__form-submit-wrapper o-forms-input o-forms-input--text o-forms-input--suffix">
                                <input required autocomplete="off" type="text" name="q" placeholder="please enter a search term" value="bolsonaro">
                                <button class="search-head__form-submit o-buttons o-buttons--primary o-buttons--big o-buttons-icon o-buttons-icon--search o-buttons-icon--icon-only">
                                    <span class="o-buttons-icon__label">Search</span>
                                </button>
                            </span>
                        </label>
                    </form>

                    <div class="search-head__notes">
                        Need some help searching? <a href="#help-tooltip" id="help-tooltip-target">View our hints and tips</a>.
                        <div id="help-tooltip" class="search-head__notes-tooltip">
                            <div class="search-tips">
                                <p>Hints and tips:</p>
                                <ul>
                                    <li>Put a phrase in double quotes to search for an exact match, E.G. <a href="?q=%22theresa+may%22">"Theresa May"</a>.</li>
                                    <li>Combine searches with operators (AND, OR, NOT) between words, E.G. <a href="?q=oil+AND+gas+NOT+europe">Oil AND gas NOT Europe</a>.</li>
                                    <li>Use parentheses to group operators, E.G. <a href="?q=elon+musk+AND+%28tesla+OR+spacex%29">Elon Musk AND (Tesla OR SpaceX)</a>.</li>
                                </ul>
                            </div>
                        </div>
                    </div>

                    <div class="search-head__active" data-trackable="active-filters">
                        <h2 class="search-head__active-heading o-normalise-visually-hidden">Active refinements</h2>
                                <a class="search-head__active-label" href="?q&#x3D;bolsonaro&dateRange&#x3D;now-7d&sort&#x3D;relevance&expandRefinements&#x3D;true" data-trackable="active-filter">
                                    Articles
                                </a>

                                <a class="search-head__active-label" href="?q&#x3D;bolsonaro&contentType&#x3D;article&sort&#x3D;relevance&expandRefinements&#x3D;true" data-trackable="active-filter">
                                    Last 7 days
                                </a>



                            <a
                                    href="?q=bolsonaro"
                                    rel="nofollow"
                                    data-trackable="clear-all">
                                Clear all
                            </a>

                    </div>
                </div>
            </header>

            <main class="search-layout__main" role="main" id="site-content">



                <div class="search-results o-teaser-collection" data-trackable="search-results">
                    <div class="search-results__heading">
                        <div class="search-results__heading-refine">
                                <div class="search-results__heading-sort">
                                    <div class="search-sort" data-trackable="sort-order">
                                        <div class="search-sort__label">
                                            Sort by
                                        </div>
                                        <div class="search-sort__buttons-group o-buttons-group">
                                                <a
                                                    class="o-buttons o-buttons--secondary search-sort__link js-sort-link"
                                                    aria-selected="true"
                                                    href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&expandRefinements&#x3D;false"
                                                    rel="nofollow"
                                                    aria-label="Relevance  (this option is currently selected)"
                                                    data-trackable="sort-item">
                                                    Relevance
                                                </a>
                                                <a
                                                    class="o-buttons o-buttons--secondary search-sort__link js-sort-link"
                                                    aria-selected="false"
                                                    href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&sort&#x3D;date&expandRefinements&#x3D;false"
                                                    rel="nofollow"

                                                    data-trackable="sort-item">
                                                    Date
                                                </a>
                                        </div>
                                    </div>
                                </div>
                                <div class="search-results__heading-refine-toggle">
                                    <a
                                        href="?q=bolsonaro"
                                        aria-expanded="true"
                                        class="search-results__heading-refine-toggle__link" id="js-refinements-toggle">
                                        <span class="search-results__heading-refine-toggle__expanded-text">
                                            Fewer search options
                                        </span>
                                        <span class="search-results__heading-refine-toggle__collapsed-text">
                                            Refine your search
                                        </span>
                                    </a>
                                </div>
                        </div>
                        <div class="search-results__heading-title">
                            <h2 class="o-teaser-collection__heading o-teaser-collection__heading--half-width">
                                Viewing results 1‒3 of 3
                            </h2>
                        </div>
                    </div>

                    <div id="js-refinements" class="search-results__refine" >
                        <dl>
                            <div class="search-results__refinements">
                                <div class="search-results__refinement">
                                    <dt class="search-refine__list-name">Date</dt>
                                    <dd class="search-refine__list-body" data-trackable="by-date">
                                        <ul class="search-refine__params search-refine__params--filters">
                                                <li class="search-refine__params-item" aria-current="true">
                                                    <a class="search-refine__params-link search-refine__params-link--selected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow" aria-label="Last 7 days  (this option is currently selected)" data-trackable="params-item">
                                                        Last 7 days
                                                    </a>
                                                </li>
                                                <li class="search-refine__params-item" >
                                                    <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-30d&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                        Last 30 days
                                                    </a>
                                                </li>
                                                <li class="search-refine__params-item" >
                                                    <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-1y&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                        Last year
                                                    </a>
                                                </li>
                                        </ul>

                                        <form class="search-refine__form search-refine__form--date">
                                            <input type="hidden" name="expandRefinements" value="true">
                                            <input type="hidden" name="q" value="bolsonaro">
                                            <input type="hidden" name="contentType" value="article">

                                            <label class="search-refine__form-label o-forms-field">
                                                <span class="search-refine__form-title o-forms-title">
                                                    <span class="search-refine__form-title-main o-forms-title__main">From</span>
                                                </span>
                                                <span class="search-refine__form-input-wrapper o-forms-input o-forms-input--text">
                                                    <input type="date" class="search-refine__form-input" name="dateFrom" id="dateFrom" value="" placeholder="dd/mm/yyyy">
                                                </span>
                                            </label>

                                            <label class="search-refine__form-label o-forms-field">
                                                <span class="search-refine__form-title o-forms-title">
                                                    <span class="search-refine__form-title-main o-forms-title__main">To</span>
                                                </span>
                                                <span class="search-refine__form-input-wrapper o-forms-input o-forms-input--text">
                                                    <input type="date" class="search-refine__form-input" name="dateTo" id="dateTo" value="" placeholder="dd/mm/yyyy">
                                                </span>
                                            </label>
                                            <br>

                                            <button class="search-refine__form-submit o-buttons o-buttons--primary" type="submit" data-trackable="date-submit">
                                                Apply
                                            </button>
                                        </form>

                                    </dd>
                                </div>

                                <div class="search-results__refinement">
                                    <dt class="search-refine__list-name">Type</dt>
                                    <dd class="search-refine__list-body" data-trackable="by-type">
                                        <ul class="search-refine__params search-refine__params--filters">
                                                <li class="search-refine__params-item" aria-current="true">
                                                    <a class="search-refine__params-link search-refine__params-link--selected" href="?q&#x3D;bolsonaro&dateRange&#x3D;now-7d&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow" aria-label="Articles  (this option is currently selected)" data-trackable="params-item">
                                                        Articles
                                                    </a>
                                                </li>
                                                <li class="search-refine__params-item" >
                                                    <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;video&dateRange&#x3D;now-7d&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                        Videos
                                                    </a>
                                                </li>
                                                <li class="search-refine__params-item" >
                                                    <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;podcast&dateRange&#x3D;now-7d&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                        Podcasts
                                                    </a>
                                                </li>
                                        </ul>
                                    </dd>
                                </div>
                            </div>

                            <div class="search-results__refinements">
                                    <div class="search-results__refinement">
                                        <dt class="search-refine__list-name">Topic</dt>
                                        <dd class="search-refine__list-body" data-trackable="by-Topic">
                                            <ul class="search-refine__params search-refine__params--facets">
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;3f74186b-57e2-4dc6-b70b-5380c1c784ea&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Politics&nbsp;<span class="search-refine__params-count">3</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;82645c31-4426-4ef5-99c9-9df6e0940c00&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            World&nbsp;<span class="search-refine__params-count">3</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;d065b0cc-0b23-4d37-80a1-eee6aef95d41&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Americas politics & policy&nbsp;<span class="search-refine__params-count">3</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;fab33c6e-17a3-4c4b-b457-1c8e7aceff6d&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Emerging markets&nbsp;<span class="search-refine__params-count">3</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;10e9e5c9-538d-4574-88ab-532fb120d8ea&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Brazilian politics&nbsp;<span class="search-refine__params-count">2</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;3c608fef-7e4f-4e44-ade5-caa90d353308&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Oil&nbsp;<span class="search-refine__params-count">2</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;51379838-c026-4cd2-8dfb-da63f7d0b123&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Commodities&nbsp;<span class="search-refine__params-count">2</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;5216ff2e-a6ed-42b2-a613-109fc6491ba2&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Energy sector&nbsp;<span class="search-refine__params-count">2</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;5f93fe95-f487-4aba-abce-39e7d783047d&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Americas companies&nbsp;<span class="search-refine__params-count">2</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;67fb70ae-b970-4bd1-92ea-761618ae8cb7&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Natural resources&nbsp;<span class="search-refine__params-count">2</span>
                                                        </a>
                                                    </li>
                                            </ul>
                                        </dd>
                                    </div>
                                    <div class="search-results__refinement">
                                        <dt class="search-refine__list-name">Organisation</dt>
                                        <dd class="search-refine__list-body" data-trackable="by-Organisation">
                                            <ul class="search-refine__params search-refine__params--facets">
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;58616188-5719-463b-a083-cf09e85d074f&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Petroleo Brasileiro SA&nbsp;<span class="search-refine__params-count">2</span>
                                                        </a>
                                                    </li>
                                            </ul>
                                        </dd>
                                    </div>
                                    <div class="search-results__refinement">
                                        <dt class="search-refine__list-name">Location</dt>
                                        <dd class="search-refine__list-body" data-trackable="by-Location">
                                            <ul class="search-refine__params search-refine__params--facets">
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;412524e0-9fc1-4225-9879-106749874eb4&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Latin America&nbsp;<span class="search-refine__params-count">3</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;76ec50eb-1ebb-4931-8e34-8f601a16dcca&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Americas&nbsp;<span class="search-refine__params-count">3</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;cdfd594a-43a6-45cf-b036-dd813391eef6&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            South America&nbsp;<span class="search-refine__params-count">3</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;a3c8036c-9f8b-47a8-ab62-245a2634760a&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Brazil&nbsp;<span class="search-refine__params-count">2</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;e695b435-e4c1-4e1f-b2c2-38e804b074be&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Argentina&nbsp;<span class="search-refine__params-count">1</span>
                                                        </a>
                                                    </li>
                                            </ul>
                                        </dd>
                                    </div>
                                    <div class="search-results__refinement">
                                        <dt class="search-refine__list-name">Genre</dt>
                                        <dd class="search-refine__list-body" data-trackable="by-Genre">
                                            <ul class="search-refine__params search-refine__params--facets">
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;a579350c-61ce-4c00-97ca-ddaa2e0cacf6&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            News&nbsp;<span class="search-refine__params-count">2</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;6da31a37-691f-4908-896f-2829ebe2309e&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Opinion&nbsp;<span class="search-refine__params-count">1</span>
                                                        </a>
                                                    </li>
                                            </ul>
                                        </dd>
                                    </div>
                                    <div class="search-results__refinement">
                                        <dt class="search-refine__list-name">Author</dt>
                                        <dd class="search-refine__list-body" data-trackable="by-Author">
                                            <ul class="search-refine__params search-refine__params--facets">
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;0b36ba4d-898e-4246-9be7-634f7e21c2d9&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Bryan Harris&nbsp;<span class="search-refine__params-count">2</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;d10a192b-cff8-4871-a487-75c92dc64784&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Michael Pooler&nbsp;<span class="search-refine__params-count">2</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;bolsonaro&contentType&#x3D;article&dateRange&#x3D;now-7d&concept&#x3D;ff90298f-c3fb-4fd7-9fab-b2bccc43b5c6&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Michael Stott&nbsp;<span class="search-refine__params-count">1</span>
                                                        </a>
                                                    </li>
                                            </ul>
                                        </dd>
                                    </div>
                            </div>
                        </dl>
                    </div>


                        <ul class="search-results__list" data-trackable="search-results-list">
                                <li class="search-results__list-item" data-trackable="item-0">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="5178dabd-ed17-491d-aaba-0627ea266568"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/stream/58616188-5719-463b-a083-cf09e85d074f" aria-label="Category: Petroleo Brasileiro SA">Petroleo Brasileiro SA</a></div><div class="o-teaser__heading"><a href="/content/5178dabd-ed17-491d-aaba-0627ea266568" data-trackable="heading-link" class="js-teaser-heading-link"><span>Brazil’s <mark class="search-item__highlight">Bolsonaro</mark> intervenes to oust Petrobras chief   </span></a></div><p class="o-teaser__standfirst"><a href="/content/5178dabd-ed17-491d-aaba-0627ea266568" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...Brazilian oil producer Petrobras was plunged into turmoil after the country’s president Jair <mark class="search-item__highlight">Bolsonaro</mark> abruptly announced a plan to replace its chief executive with an army general following recent...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2021-02-20T01:36:32+0000">February 20, 2021</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="5178dabd-ed17-491d-aaba-0627ea266568"
                                        data-myft-ui="saved"
                                        action="/myft/save/5178dabd-ed17-491d-aaba-0627ea266568"
                                        data-js-action="/__myft/api/core/saved/content/5178dabd-ed17-491d-aaba-0627ea266568?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="5178dabd-ed17-491d-aaba-0627ea266568" 
                                                aria-label="Save Brazil’s Bolsonaro intervenes to oust Petrobras chief    to myFT for later"
                                                title="Save Brazil’s Bolsonaro intervenes to oust Petrobras chief    to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/5178dabd-ed17-491d-aaba-0627ea266568" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/https%3A%2F%2Fd1e00ek4ebabms.cloudfront.net%2Fproduction%2F6ed2d364-6a13-4692-a6b0-339a58e1091a.jpg?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-1">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image o-teaser--opinion js-teaser" data-id="4873f5b3-5359-442f-ac75-ddd345313927"><div class="o-teaser__content"><div class="o-teaser__meta"><span class="o-teaser__tag-prefix">Global Insight</span><a class="o-teaser__tag" data-trackable="teaser-tag" href="/michael-stott" aria-label="Category: Michael Stott">Michael Stott</a></div><div class="o-teaser__heading"><a href="/content/4873f5b3-5359-442f-ac75-ddd345313927" data-trackable="heading-link" class="js-teaser-heading-link">Alberto Fernández, Latin America’s unlikely leader</a></div><p class="o-teaser__standfirst"><a href="/content/4873f5b3-5359-442f-ac75-ddd345313927" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...Francis, who is Argentine, and the Catholic values they share, the foreign ministry in Buenos Aires said.

                                    By contrast, Brazil’s hard-right leader Jair <mark class="search-item__highlight">Bolsonaro</mark> and Mexico’s López Obrador were both on...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2021-02-22T05:00:29+0000">February 22, 2021</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="4873f5b3-5359-442f-ac75-ddd345313927"
                                        data-myft-ui="saved"
                                        action="/myft/save/4873f5b3-5359-442f-ac75-ddd345313927"
                                        data-js-action="/__myft/api/core/saved/content/4873f5b3-5359-442f-ac75-ddd345313927?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="4873f5b3-5359-442f-ac75-ddd345313927" 
                                                aria-label="Save Alberto Fernández, Latin America’s unlikely leader to myFT for later"
                                                title="Save Alberto Fernández, Latin America’s unlikely leader to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/4873f5b3-5359-442f-ac75-ddd345313927" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/https%3A%2F%2Fd1e00ek4ebabms.cloudfront.net%2Fproduction%2F109e6380-5bff-4098-8aa3-2a8ed128f156.jpg?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-2">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="68b0c6cf-7d78-4e0e-9025-bfaca7e098e2"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/stream/58616188-5719-463b-a083-cf09e85d074f" aria-label="Category: Petroleo Brasileiro SA">Petroleo Brasileiro SA</a></div><div class="o-teaser__heading"><a href="/content/68b0c6cf-7d78-4e0e-9025-bfaca7e098e2" data-trackable="heading-link" class="js-teaser-heading-link">Brazilian markets rattled by Bolsonaro’s removal of Petrobras chief</a></div><p class="o-teaser__standfirst"><a href="/content/68b0c6cf-7d78-4e0e-9025-bfaca7e098e2" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...after <mark class="search-item__highlight">Bolsonaro</mark> publicly criticised incumbent chief executive Roberto Castello Branco over recent increases in the price of petrol and diesel, which have sparked unrest among truckers and threats of strikes...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2021-02-22T15:36:44+0000">February 22, 2021</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="68b0c6cf-7d78-4e0e-9025-bfaca7e098e2"
                                        data-myft-ui="saved"
                                        action="/myft/save/68b0c6cf-7d78-4e0e-9025-bfaca7e098e2"
                                        data-js-action="/__myft/api/core/saved/content/68b0c6cf-7d78-4e0e-9025-bfaca7e098e2?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="68b0c6cf-7d78-4e0e-9025-bfaca7e098e2" 
                                                aria-label="Save Brazilian markets rattled by Bolsonaro’s removal of Petrobras chief to myFT for later"
                                                title="Save Brazilian markets rattled by Bolsonaro’s removal of Petrobras chief to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/68b0c6cf-7d78-4e0e-9025-bfaca7e098e2" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/https%3A%2F%2Fd1e00ek4ebabms.cloudfront.net%2Fproduction%2F5d0c04cd-ba56-4255-8832-2e2b239476c6.jpg?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                        </ul>
                        <div class="search-results__pagination">
                            <div class="search-pagination o-buttons-pagination" data-trackable="search-pagination">
                                <a class="search-pagination__prev-page o-buttons o-buttons--secondary o-buttons-icon o-buttons-icon--arrow-left o-buttons--big o-buttons-icon--icon-only" disabled rel="nofollow" data-trackable="prev-page">
                                    <span class="o-buttons-icon__label">Previous page</span>
                                </a>
                                <span class="search-pagination__page">Page <strong>1</strong> of 1</span>
                                <a class="search-pagination__next-page o-buttons o-buttons--secondary o-buttons-icon o-buttons-icon--arrow-right o-buttons--big o-buttons-icon--icon-only" disabled rel="nofollow" data-trackable="next-page">
                                    <span class="o-buttons-icon__label">Next page</span>
                                </a>
                            </div>
                        </div>
                </div>

            </main>

            <nav class="search-layout__sidebar" id="site-sidebar">
                            <div
                                class="concept-list"
                                data-trackable="concept-list">
                                    <h2 class="concept-list__title">
                                            Topics relevant to this search
                                    </h2>
                                <ul class="concept-list__list">
                                        <li class="concept-list__list-item">
                                            <a
                                                href="https://www.ft.com/stream/2fc407d7-c4d8-41f8-903c-5af26d12f09a"
                                                data-trackable="concept"
                                                class="concept-list__concept">
                                                Jair Bolsonaro
                                            </a>
                                                <form
                                                    class="n-myft-ui n-myft-ui--follow "
                                                    method="GET"
                                                    data-myft-ui="follow"
                                                    data-concept-id="2fc407d7-c4d8-41f8-903c-5af26d12f09a"

                                                            action="/myft/add/2fc407d7-c4d8-41f8-903c-5af26d12f09a"
                                                            data-js-action="/__myft/api/core/followed/concept/2fc407d7-c4d8-41f8-903c-5af26d12f09a?method=put"
                                                    >
                                                    <input
                                                        data-myft-csrf-token
                                                        value=""
                                                        type="hidden"
                                                        name="token">
                                                    <div
                                                        class="n-myft-ui__announcement o-normalise-visually-hidden"
                                                        aria-live="assertive"
                                                        data-pressed-text="Now following Jair Bolsonaro."
                                                        data-unpressed-text="No longer following Jair Bolsonaro."
                                                    ></div>
                                                    <button
                                                            aria-pressed="false"
                                                            aria-label="Add Jair Bolsonaro to myFT"
                                                            title="Add Jair Bolsonaro to myFT"
                                                            data-alternate-label="Remove Jair Bolsonaro from myFT"
                                                                    data-alternate-text="Added"
                                                        class="
                                                            n-myft-follow-button"
                                                        data-concept-id="2fc407d7-c4d8-41f8-903c-5af26d12f09a" 
                                                        data-trackable="follow"
                                                        type="submit">Add to myFT</button>
                                                </form>
                                        </li>
                                        <li class="concept-list__list-item">
                                            <a
                                                href="https://www.ft.com/brazilian-politics"
                                                data-trackable="concept"
                                                class="concept-list__concept">
                                                Brazilian politics
                                            </a>
                                                <form
                                                    class="n-myft-ui n-myft-ui--follow "
                                                    method="GET"
                                                    data-myft-ui="follow"
                                                    data-concept-id="10e9e5c9-538d-4574-88ab-532fb120d8ea"

                                                            action="/myft/add/10e9e5c9-538d-4574-88ab-532fb120d8ea"
                                                            data-js-action="/__myft/api/core/followed/concept/10e9e5c9-538d-4574-88ab-532fb120d8ea?method=put"
                                                    >
                                                    <input
                                                        data-myft-csrf-token
                                                        value=""
                                                        type="hidden"
                                                        name="token">
                                                    <div
                                                        class="n-myft-ui__announcement o-normalise-visually-hidden"
                                                        aria-live="assertive"
                                                        data-pressed-text="Now following Brazilian politics."
                                                        data-unpressed-text="No longer following Brazilian politics."
                                                    ></div>
                                                    <button
                                                            aria-pressed="false"
                                                            aria-label="Add Brazilian politics to myFT"
                                                            title="Add Brazilian politics to myFT"
                                                            data-alternate-label="Remove Brazilian politics from myFT"
                                                                    data-alternate-text="Added"
                                                        class="
                                                            n-myft-follow-button"
                                                        data-concept-id="10e9e5c9-538d-4574-88ab-532fb120d8ea" 
                                                        data-trackable="follow"
                                                        type="submit">Add to myFT</button>
                                                </form>
                                        </li>
                                        <li class="concept-list__list-item">
                                            <a
                                                href="https://www.ft.com/stream/58616188-5719-463b-a083-cf09e85d074f"
                                                data-trackable="concept"
                                                class="concept-list__concept">
                                                Petroleo Brasileiro SA
                                            </a>
                                                <form
                                                    class="n-myft-ui n-myft-ui--follow "
                                                    method="GET"
                                                    data-myft-ui="follow"
                                                    data-concept-id="58616188-5719-463b-a083-cf09e85d074f"

                                                            action="/myft/add/58616188-5719-463b-a083-cf09e85d074f"
                                                            data-js-action="/__myft/api/core/followed/concept/58616188-5719-463b-a083-cf09e85d074f?method=put"
                                                    >
                                                    <input
                                                        data-myft-csrf-token
                                                        value=""
                                                        type="hidden"
                                                        name="token">
                                                    <div
                                                        class="n-myft-ui__announcement o-normalise-visually-hidden"
                                                        aria-live="assertive"
                                                        data-pressed-text="Now following Petroleo Brasileiro SA."
                                                        data-unpressed-text="No longer following Petroleo Brasileiro SA."
                                                    ></div>
                                                    <button
                                                            aria-pressed="false"
                                                            aria-label="Add Petroleo Brasileiro SA to myFT"
                                                            title="Add Petroleo Brasileiro SA to myFT"
                                                            data-alternate-label="Remove Petroleo Brasileiro SA from myFT"
                                                                    data-alternate-text="Added"
                                                        class="
                                                            n-myft-follow-button"
                                                        data-concept-id="58616188-5719-463b-a083-cf09e85d074f" 
                                                        data-trackable="follow"
                                                        type="submit">Add to myFT</button>
                                                </form>
                                        </li>
                                        <li class="concept-list__list-item">
                                            <a
                                                href="https://www.ft.com/companies/americas"
                                                data-trackable="concept"
                                                class="concept-list__concept">
                                                Americas companies
                                            </a>
                                                <form
                                                    class="n-myft-ui n-myft-ui--follow "
                                                    method="GET"
                                                    data-myft-ui="follow"
                                                    data-concept-id="5f93fe95-f487-4aba-abce-39e7d783047d"

                                                            action="/myft/add/5f93fe95-f487-4aba-abce-39e7d783047d"
                                                            data-js-action="/__myft/api/core/followed/concept/5f93fe95-f487-4aba-abce-39e7d783047d?method=put"
                                                    >
                                                    <input
                                                        data-myft-csrf-token
                                                        value=""
                                                        type="hidden"
                                                        name="token">
                                                    <div
                                                        class="n-myft-ui__announcement o-normalise-visually-hidden"
                                                        aria-live="assertive"
                                                        data-pressed-text="Now following Americas companies."
                                                        data-unpressed-text="No longer following Americas companies."
                                                    ></div>
                                                    <button
                                                            aria-pressed="false"
                                                            aria-label="Add Americas companies to myFT"
                                                            title="Add Americas companies to myFT"
                                                            data-alternate-label="Remove Americas companies from myFT"
                                                                    data-alternate-text="Added"
                                                        class="
                                                            n-myft-follow-button"
                                                        data-concept-id="5f93fe95-f487-4aba-abce-39e7d783047d" 
                                                        data-trackable="follow"
                                                        type="submit">Add to myFT</button>
                                                </form>
                                        </li>
                                        <li class="concept-list__list-item">
                                            <a
                                                href="https://www.ft.com/companies/oil-gas"
                                                data-trackable="concept"
                                                class="concept-list__concept">
                                                Oil & Gas industry
                                            </a>
                                                <form
                                                    class="n-myft-ui n-myft-ui--follow "
                                                    method="GET"
                                                    data-myft-ui="follow"
                                                    data-concept-id="9f991f8b-1d17-4d99-82f4-0cd8fe50b59b"

                                                            action="/myft/add/9f991f8b-1d17-4d99-82f4-0cd8fe50b59b"
                                                            data-js-action="/__myft/api/core/followed/concept/9f991f8b-1d17-4d99-82f4-0cd8fe50b59b?method=put"
                                                    >
                                                    <input
                                                        data-myft-csrf-token
                                                        value=""
                                                        type="hidden"
                                                        name="token">
                                                    <div
                                                        class="n-myft-ui__announcement o-normalise-visually-hidden"
                                                        aria-live="assertive"
                                                        data-pressed-text="Now following Oil & Gas industry."
                                                        data-unpressed-text="No longer following Oil & Gas industry."
                                                    ></div>
                                                    <button
                                                            aria-pressed="false"
                                                            aria-label="Add Oil & Gas industry to myFT"
                                                            title="Add Oil & Gas industry to myFT"
                                                            data-alternate-label="Remove Oil & Gas industry from myFT"
                                                                    data-alternate-text="Added"
                                                        class="
                                                            n-myft-follow-button"
                                                        data-concept-id="9f991f8b-1d17-4d99-82f4-0cd8fe50b59b" 
                                                        data-trackable="follow"
                                                        type="submit">Add to myFT</button>
                                                </form>
                                        </li>
                                </ul>
                            </div>
                    <div
                        class="o-ads o-ads--center o-ads--reserve-"
                        data-o-ads-name="search-refine-mid"
                        data-o-ads-targeting="pos=mid;"
                        data-o-ads-formats-small="false"
                        data-o-ads-formats-medium="MediumRectangle"
                        data-o-ads-formats-large="MediumRectangle,HalfPage,Responsive"
                        data-o-ads-formats-extra="MediumRectangle,HalfPage,Responsive"
                        aria-hidden="true">
                    </div>
            </nav>

        </div>
    </div></div></div><div class="n-layout__row n-layout__row--footer"><div class="n-layout__footer-before"><div class="n-feedback__container n-feedback--hidden"></div></div><footer id="site-footer" class="o-footer o-footer--theme-dark" data-o-component="o-footer"><div class="o-footer__container"><div class="o-footer__row"><h2 class="o-normalise-visually-hidden">Useful links</h2><nav class="o-footer__matrix" role="navigation" aria-label="Useful links"><div class="o-footer__matrix-group o-footer__matrix-group--1"><h3 class="o-footer__matrix-title">Support</h3><div class="o-footer__matrix-content" id="o-footer-section-0"><div class="o-footer__matrix-column"><a class="o-footer__matrix-link" href="/tour" data-trackable="View Site Tips">View Site Tips</a><a class="o-footer__matrix-link" href="https://help.ft.com" data-trackable="Help Centre">Help Centre</a><a class="o-footer__matrix-link" href="https://aboutus.ft.com/en-gb/contact-us/" data-trackable="Contact Us">Contact Us</a><a class="o-footer__matrix-link" href="https://www.ft.com/aboutus" data-trackable="About Us">About Us</a><a class="o-footer__matrix-link" href="/accessibility" data-trackable="Accessibility">Accessibility</a><a class="o-footer__matrix-link" href="/tour/myft" data-trackable="myFT Tour">myFT Tour</a><a class="o-footer__matrix-link" href="https://aboutus.ft.com/en-gb/careers/" data-trackable="Careers">Careers</a></div></div></div><div class="o-footer__matrix-group o-footer__matrix-group--1"><h3 class="o-footer__matrix-title">Legal & Privacy</h3><div class="o-footer__matrix-content" id="o-footer-section-1"><div class="o-footer__matrix-column"><a class="o-footer__matrix-link" href="https://help.ft.com/help/legal-privacy/terms-conditions/" data-trackable="Terms & Conditions">Terms & Conditions</a><a class="o-footer__matrix-link" href="https://help.ft.com/help/legal-privacy/privacy/" data-trackable="Privacy Policy">Privacy Policy</a><a class="o-footer__matrix-link" href="https://help.ft.com/help/legal-privacy/cookies/" data-trackable="Cookies">Cookies</a><a class="o-footer__matrix-link" href="https://help.ft.com/help/legal-privacy/copyright/copyright-policy/" data-trackable="Copyright">Copyright</a><a class="o-footer__matrix-link" href="https://help.ft.com/help/legal/slavery-statement/" data-trackable="Slavery Statement & Policies">Slavery Statement & Policies</a></div></div></div><div class="o-footer__matrix-group o-footer__matrix-group--2"><h3 class="o-footer__matrix-title" aria-controls="o-footer-section-2">Services</h3><div class="o-footer__matrix-content" id="o-footer-section-2"><div class="o-footer__matrix-column"><a class="o-footer__matrix-link" href="https://live.ft.com/" data-trackable="FT Live">FT Live</a><a class="o-footer__matrix-link" href="/securedrop" data-trackable="Share News Tips Securely" data-o-tracking-do-not-track="true">Share News Tips Securely</a><a class="o-footer__matrix-link" href="https://www.ft.com/products" data-trackable="Individual Subscriptions">Individual Subscriptions</a><a class="o-footer__matrix-link" href="https://enterprise.ft.com/en-gb/services/group-subscriptions/group-contact-us/?segmentId=383c7f63-abf4-b62d-cb33-4c278e6fdf61&cpccampaign=B2B_link_ft.com_footer" data-trackable="Group Subscriptions">Group Subscriptions</a><a class="o-footer__matrix-link" href="https://enterprise.ft.com/en-gb/services/republishing/" data-trackable="Republishing">Republishing</a><a class="o-footer__matrix-link" href="http://www.businessesforsale.com/ft2/notices" data-trackable="Contracts & Tenders">Contracts & Tenders</a></div><div class="o-footer__matrix-column"><a class="o-footer__matrix-link" href="https://www.exec-appointments.com/" data-trackable="Executive Job Search">Executive Job Search</a><a class="o-footer__matrix-link" href="http://fttoolkit.co.uk/" data-trackable="Advertise with the FT">Advertise with the FT</a><a class="o-footer__matrix-link" href="https://twitter.com/financialtimes" data-trackable="Follow the FT on Twitter">Follow the FT on Twitter</a><a class="o-footer__matrix-link" href="https://transact.ft.com/" data-trackable="FT Transact">FT Transact</a><a class="o-footer__matrix-link" href="https://enterprise.ft.com/en-gb/services/group-subscriptions/education/" data-trackable="Secondary Schools">Secondary Schools</a></div></div></div><div class="o-footer__matrix-group o-footer__matrix-group--2"><h3 class="o-footer__matrix-title" aria-controls="o-footer-section-3">Tools</h3><div class="o-footer__matrix-content" id="o-footer-section-3"><div class="o-footer__matrix-column"><a class="o-footer__matrix-link" href="https://markets.ft.com/data/portfolio/dashboard" data-trackable="Portfolio">Portfolio</a><a class="o-footer__matrix-link" href="https://www.ft.com/todaysnewspaper" data-trackable="Today&#x27;s Newspaper (ePaper)">Today&#x27;s Newspaper (ePaper)</a><a class="o-footer__matrix-link" href="https://markets.ft.com/data/alerts/" data-trackable="Alerts Hub">Alerts Hub</a><a class="o-footer__matrix-link" href="https://rankings.ft.com/businessschoolrankings/rankings" data-trackable="Business School Rankings">Business School Rankings</a></div><div class="o-footer__matrix-column"><a class="o-footer__matrix-link" href="https://kat.ft.com/" data-trackable="Enterprise Tools">Enterprise Tools</a><a class="o-footer__matrix-link" href="/news-feed" data-trackable="News feed">News feed</a><a class="o-footer__matrix-link" href="/newsletters" data-trackable="Newsletters">Newsletters</a><a class="o-footer__matrix-link" href="https://markets.ft.com/research/Markets/Currencies?segid=70113" data-trackable="Currency Converter">Currency Converter</a></div></div></div></nav></div><div class="o-footer__external-link o-footer__matrix-title"><a class="o-footer__more-from-ft o-footer__matrix-title" href="https://ft.com/more-from-ft-group" data-trackable="more-from-ft">More from the FT Group</a></div><div class="o-footer__copyright"><small>Markets data delayed by at least 15 minutes. © THE FINANCIAL TIMES LTD 2021. <abbr title="Financial Times" aria-label="F T">FT</abbr> and ‘Financial Times’ are trademarks of The Financial Times Ltd.<br/>The Financial Times and its journalism are subject to a self-regulation regime under the <a href="https://aboutus.ft.com/en-gb/ft-editorial-code/" aria-label="F T Editorial Code of Practice">FT Editorial Code of Practice</a>.</small></div></div><div class="o-footer__brand"><div class="o-footer__container"><div class="o-footer__brand-logo"></div></div></div></footer><div class="n-layout__footer-after"><script data-pixel-src="https://spoor-api.ft.com/px.gif?data=%7B%22category%22%3A%22page%22%2C%22action%22%3A%22view%22%2C%22system%22%3A%7B%22source%22%3A%22non-ctm%22%7D%2C%22context%22%3A%7B%22name%22%3A%22search-page%22%2C%22product%22%3A%22next%22%2C%22data%22%3A%7B%22source%22%3A%22%5BSOURCE%5D%22%7D%7D%7D">(function coreExperience() {
        if (/\bcore\b/.test(document.documentElement.className)) {
          var currentScript = document.scripts[document.scripts.length - 1];
          var img = new Image();
          img.src = currentScript.getAttribute('data-pixel-src');
        }
      })();</script><noscript><img src="https://spoor-api.ft.com/px.gif?data=%7B%22category%22%3A%22page%22%2C%22action%22%3A%22view%22%2C%22system%22%3A%7B%22source%22%3A%22non-ctm%22%7D%2C%22context%22%3A%7B%22name%22%3A%22search-page%22%2C%22product%22%3A%22next%22%2C%22data%22%3A%7B%22source%22%3A%22%5BSOURCE%5D%22%7D%7D%7D"/></noscript></div></div><div class="o-header__drawer" id="o-header-drawer" role="navigation" aria-label="Drawer menu" data-o-header-drawer="true" data-o-header-drawer--no-js="true" data-trackable="drawer" data-trackable-terminate="true"><div class="o-header__drawer-inner"><div class="o-header__drawer-tools"><button type="button" class="o-header__drawer-tools-close" title="Close drawer menu" aria-controls="o-header-drawer" data-trackable="close"><span class="o-header__visually-hidden">Close drawer menu</span></button><a class="o-header__drawer-tools-logo" href="/" data-trackable="logo"><span class="o-header__visually-hidden">Financial Times</span></a><p class="o-header__drawer-current-edition">International Edition</p></div><div class="o-header__drawer-search"><form class="o-header__drawer-search-form" action="/search" role="search" aria-label="Site search" data-n-topic-search="true" data-n-topic-search-categories="concepts,equities" data-n-topic-search-view-all="true"><label class="o-header__visually-hidden" for="o-header-drawer-search-term">Search the <abbr title="Financial Times">FT</abbr></label><input type="text" class="o-header__drawer-search-term" id="o-header-drawer-search-term" name="q" autoComplete="off" autoCorrect="off" autoCapitalize="off" spellcheck="false" placeholder="Search the FT" data-trackable="search-term" data-n-topic-search-input="true"/><button class="o-header__drawer-search-submit" type="submit" data-trackable="search-submit"><span class="o-header__visually-hidden">Search</span></button></form></div><nav class="o-header__drawer-menu o-header__drawer-menu--primary o-header__drawer-menu--border"><ul class="o-header__drawer-menu-list"><li class="o-header__drawer-menu-item" data-trackable="edition-switcher"><a class="o-header__drawer-menu-link" href="/?edition=uk" data-trackable="uk">Switch to UK Edition</a></li></ul><ul class="o-header__drawer-menu-list"><li class="o-header__drawer-menu-item o-header__drawer-menu-item--heading">Top sections</li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/" data-trackable="Home">Home</a></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/world" data-trackable="World">World</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-1" data-trackable="sub-level-toggle | World">Show more World</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-1" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/global-economy" data-trackable="Global Economy">Global Economy</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/uk" data-trackable="UK">UK</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/us" data-trackable="US">US</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/asia-pacific/china" data-trackable="China">China</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/africa" data-trackable="Africa">Africa</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/asia-pacific" data-trackable="Asia Pacific">Asia Pacific</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/emerging-markets" data-trackable="Emerging Markets">Emerging Markets</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/europe" data-trackable="Europe">Europe</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/americas" data-trackable="Americas">Americas</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/mideast" data-trackable="Middle East and North Africa">Middle East and North Africa</a></li></ul></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/world/us" data-trackable="US">US</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-2" data-trackable="sub-level-toggle | US">Show more US</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-2" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/us-economy" data-trackable="US Economy">US Economy</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/us" data-trackable="US Companies">US Companies</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/us/politics" data-trackable="US Politics & Policy">US Politics & Policy</a></li></ul></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/companies" data-trackable="Companies">Companies</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-3" data-trackable="sub-level-toggle | Companies">Show more Companies</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-3" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/energy" data-trackable="Energy">Energy</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/financials" data-trackable="Financials">Financials</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/health" data-trackable="Health">Health</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/industrials" data-trackable="Industrials">Industrials</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/media" data-trackable="Media">Media</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/professional-services" data-trackable="Professional Services">Professional Services</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/retail-consumer" data-trackable="Retail & Consumer">Retail & Consumer</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/technology" data-trackable="Tech Sector">Tech Sector</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/telecoms" data-trackable="Telecoms">Telecoms</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/transport" data-trackable="Transport">Transport</a></li></ul></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/technology" data-trackable="Tech">Tech</a></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/markets" data-trackable="Markets">Markets</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-5" data-trackable="sub-level-toggle | Markets">Show more Markets</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-5" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/alphaville" data-trackable="Alphaville">Alphaville</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="https://markets.ft.com/data" data-trackable="Markets Data">Markets Data</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/capital-markets" data-trackable="Capital Markets">Capital Markets</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/commodities" data-trackable="Commodities">Commodities</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/currencies" data-trackable="Currencies">Currencies</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/equities" data-trackable="Equities">Equities</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/fund-management" data-trackable="Fund Management">Fund Management</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/ft-trading-room" data-trackable="Trading">Trading</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/moral-money" data-trackable="Moral Money">Moral Money</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="https://etf.ft.com" data-trackable="ETF Hub">ETF Hub</a></li></ul></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/climate-capital" data-trackable="Climate">Climate</a></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/opinion" data-trackable="Opinion">Opinion</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-7" data-trackable="sub-level-toggle | Opinion">Show more Opinion</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-7" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/columnists" data-trackable="Columnists">Columnists</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/ft-view" data-trackable="The FT View">The FT View</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/the-big-read" data-trackable="The Big Read">The Big Read</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/lex" data-trackable="Lex">Lex</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/obituaries" data-trackable="Obituaries">Obituaries</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/letters" data-trackable="Letters">Letters</a></li></ul></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/work-careers" data-trackable="Work & Careers">Work & Careers</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-8" data-trackable="sub-level-toggle | Work & Careers">Show more Work & Careers</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-8" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="https://rankings.ft.com/businessschoolrankings/rankings" data-trackable="Business School Rankings">Business School Rankings</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/business-education" data-trackable="Business Education">Business Education</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/entrepreneurship" data-trackable="Entrepreneurship">Entrepreneurship</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/recruitment" data-trackable="Recruitment">Recruitment</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/business-books" data-trackable="Business Books">Business Books</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/business-travel" data-trackable="Business Travel">Business Travel</a></li></ul></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/life-arts" data-trackable="Life & Arts">Life & Arts</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-9" data-trackable="sub-level-toggle | Life & Arts">Show more Life & Arts</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-9" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/arts" data-trackable="Arts">Arts</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/books" data-trackable="Books">Books</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/food-drink" data-trackable="Food & Drink">Food & Drink</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/magazine" data-trackable="FT Magazine">FT Magazine</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/house-home" data-trackable="House & Home">House & Home</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/style" data-trackable="Style">Style</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/travel" data-trackable="Travel">Travel</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/globetrotter" data-trackable="FT Globetrotter">FT Globetrotter</a></li></ul></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/personal-finance" data-trackable="Personal Finance">Personal Finance</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-10" data-trackable="sub-level-toggle | Personal Finance">Show more Personal Finance</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-10" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/personal-finance/mortgages" data-trackable="Property & Mortgages">Property & Mortgages</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/personal-finance/investments" data-trackable="Investments">Investments</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/personal-finance/pensions" data-trackable="Pensions">Pensions</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/personal-finance/tax" data-trackable="Tax">Tax</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/personal-finance/banking-savings" data-trackable="Banking & Savings">Banking & Savings</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/personal-finance/advice-commment" data-trackable="Advice & Comment">Advice & Comment</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/next-act" data-trackable="Next Act">Next Act</a></li></ul></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/htsi" data-trackable="How to Spend It">How to Spend It</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/special-reports" data-trackable="Special Reports">Special Reports</a></li><li class="o-header__drawer-menu-item o-header__drawer-menu-item--heading">FT recommends</li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/graphics" data-trackable="Graphics">Graphics</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/lex" data-trackable="Lex">Lex</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/alphaville" data-trackable="Alphaville">Alphaville</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/lunch-with-the-ft" data-trackable="Lunch with the FT">Lunch with the FT</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/globetrotter" data-trackable="FT Globetrotter">FT Globetrotter</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="https://www.ft.com/tech-asia" data-trackable="#techAsia">#techAsia</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/moral-money" data-trackable="Moral Money">Moral Money</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/ftfm" data-trackable="FTfm">FTfm</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/newsletters" data-trackable="Newsletters">Newsletters</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/video" data-trackable="Video">Video</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="https://www.ft.com/podcasts" data-trackable="Podcasts">Podcasts</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/news-feed" data-trackable="News feed">News feed</a></li><li class="o-header__drawer-menu-item o-header__drawer-menu-item--divide"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--secondary" href="/myft" data-trackable="myFT">myFT</a></li><li class="o-header__drawer-menu-item "><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--secondary" href="https://markets.ft.com/data/portfolio/dashboard" data-trackable="Portfolio">Portfolio</a></li><li class="o-header__drawer-menu-item "><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--secondary" href="https://www.ft.com/todaysnewspaper" data-trackable="Today&#x27;s Newspaper (ePaper)">Today&#x27;s Newspaper (ePaper)</a></li><li class="o-header__drawer-menu-item "><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--secondary" href="https://www.ft.com/crossword" data-trackable="Crossword">Crossword</a></li><li class="o-header__drawer-menu-item "><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--secondary" href="https://www.ft.com/tour/apps" data-trackable="Our Apps">Our Apps</a></li></ul></nav><nav class="o-header__drawer-menu o-header__drawer-menu--user" data-trackable="user-nav"><ul class="o-header__drawer-menu-list"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link" href="https://help.ft.com" data-trackable="Help Centre">Help Centre</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link" href="/products?segmentId=d290332b-e8e8-d29b-2ff4-2019b13f008a" data-trackable="Subscribe">Subscribe</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link" href="/login?location=/search" data-trackable="Sign In">Sign In</a></li></ul></nav></div></div></div><script type="application/json" id="page-kit-app-context">{"appName":"search-page","product":"next","edition":"international","appVersion":"8877cff120fdb2086dd6aa207f963e792366ff7d","abTestState":"messageSlotTop:threeMonthsIntroPricePromo,messageSlotBottom:cookieConsentC,apprecommendedarticles:variant1","isProduction":true,"isUserLoggedIn":false,"pageKitVersion":"2.3.0"}</script><script type="application/json" id="page-kit-flags-embed">{"articleRelatedContent":true,"ads":true,"javascript":true,"oTracking":true,"articleComments":true,"frontPageHeaderMarketsData":true,"myFtApi":true,"articleShareButtons":true,"customStreamFragments":true,"frontPageMyFtSection":true,"myFtApiWrite":true,"adTargetingUserApi":true,"videoPlayerAdvertising":true,"streamMostRead":true,"serviceWorker":true,"openGraph":true,"googleConsumerSurvey":true,"nativeAds":true,"myFtDigestPromo":true,"regionalNews":true,"syndication":true,"myFTEmailDisableABAlloc":true,"streamRecommendedStoriesList":true,"myFtNoContentDigests":true,"swAssetCaching":true,"refererCohort":true,"newsletterSignupOnArticle":true,"myFTInstantNotifications":true,"myftPrioritiseTopics":true,"messageSlotTop":"threeMonthsIntroPricePromo","myftPrioritiseTopicsOnHomepage":true,"myftIndustryRecs":true,"qualtrics":true,"showEventPromo":true,"myftConsentSwitch":true,"moatAdsTraffic":true,"manageCookiesWhiteLabel":true,"fcmSwitch":true,"graphicsPage":true,"myftIndustryRecsFeed":true,"myftArticleInstantAlertsCta":true,"b2bLiveChat":true,"AdsPermutive":true,"syndication_promo":true,"topicTracker":true,"messageSlotBottom":"cookieConsentC","enableGTM":true,"myftUnreadFavicon":true,"myftNewNewLabel":true,"myftBoldNewArticles":true,"realUserMonitoringForPerformance":true,"pageAttentionTracking":true,"textCopyTracking":true,"enableIHSindexBasedTargetingScript":true,"enableBrandmetricsSurveyScript":true,"managePremiumCancellationJourney":true,"articleInstantAlertsPromo":true,"manageStepUpCancellationJourney":true,"manageStandardCancellationJourney":true,"coronavirusStats":true,"manageStepUpTransitionJourney":true,"manageDirectCancellationJourney":true,"privShowCCPALinkOnFooter":true,"useXDashForLiveBlogs":true,"customerCareLiveChat":true,"apprecommendedarticles":"variant1","liveBlogsV2NewPostButton":true,"optimizely":true,"accountSettingsBillingDetailsPaypal":true,"graphicSyndication":true,"showApplePay":true,"adsNativeLazyload":true,"adsMoatTimeout":true,"enableRavelinOnProd":true,"enableMarketingAnalyticsTags":true,"liteSubscriptionSmokeTest":true}</script></body></html>"""

In [8]:
from scrapy import Selector

In [9]:
sel = Selector(text=html)

In [12]:
xpath1 = '//*[@id="site-content"]/div/ul/li[1]/div/div/div/div[1]/p/a/span//text()'
xpath2 = '//*[@id="site-content"]/div/ul/li[2]/div/div/div/div[1]/p/a/span/text()'
xpath3 = '//*[@id="site-content"]/div/ul//div/div/div/div[1]/p/a/span//text()'

In [13]:
sel.xpath(xpath3).extract()

['...Brazilian oil producer Petrobras was plunged into turmoil after the country’s president Jair ',
 'Bolsonaro',
 ' abruptly announced a plan to replace its chief executive with an army general following recent...',
 '...Francis, who is Argentine, and the Catholic values they share, the foreign ministry in Buenos Aires said.\n\n                                    By contrast, Brazil’s hard-right leader Jair ',
 'Bolsonaro',
 ' and Mexico’s López Obrador were both on...',
 '...after ',
 'Bolsonaro',
 ' publicly criticised incumbent chief executive Roberto Castello Branco over recent increases in the price of petrol and diesel, which have sparked unrest among truckers and threats of strikes...']

In [26]:
sel.xpath(xpath2).extract()

['...Francis, who is Argentine, and the Catholic values they share, the foreign ministry in Buenos Aires said.\n\n                                    By contrast, Brazil’s hard-right leader Jair ',
 ' and Mexico’s López Obrador were both on...']

In [28]:
html =  '''<!DOCTYPE html><html lang="en-GB" class="no-js core o-typography--loading-sans o-typography--loading-sans-bold o-typography--loading-display o-typography--loading-display-bold" data-o-component="o-typography" style="overflow-x:hidden;background-color:#fff1e5;color:#33302e"><head><meta charSet="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=edge"/><meta name="viewport" content="width=device-width, initial-scale=1"/><title>lch sa | Search | Financial Times</title><meta name="description" content="News, analysis and comment from the Financial Times, the worldʼs leading global business publication"/><meta name="robots" content="index,follow,max-snippet:200,max-image-preview:large"/><meta name="google-site-verification" content="4-t8sFaPvpO5FH_Gnw1dkM28CQepjzo8UjjAkdDflTw"/><script type="application/ld+json">{"@context":"http://schema.org","@type":"WebSite","name":"Financial Times","alternateName":"FT.com","url":"http://www.ft.com"}</script><meta property="fb:pages" content="8860325749"/><meta property="twitter:site" content="@FinancialTimes"/><link rel="icon" type="image/svg+xml" href="https://www.ft.com/__origami/service/image/v2/images/raw/ftlogo-v1%3Abrand-ft-logo-square-coloured?source=update-logos&format=svg"/><link rel="alternate icon" type="image/png" href="https://www.ft.com/__origami/service/image/v2/images/raw/ftlogo-v1%3Abrand-ft-logo-square-coloured?source=update-logos&format=png&width=32&height=32" sizes="32x32"/><link rel="alternate icon" type="image/png" href="https://www.ft.com/__origami/service/image/v2/images/raw/ftlogo-v1%3Abrand-ft-logo-square-coloured?source=update-logos&format=png&width=194&height=194" sizes="194x194"/><link rel="apple-touch-icon" href="https://www.ft.com/__origami/service/image/v2/images/raw/ftlogo-v1%3Abrand-ft-logo-square-coloured?source=update-logos&format=png&width=180&height=180" sizes="180x180"/><link rel="manifest" href="https://www.ft.com/__assets/creatives/manifest/manifest-v6.json"/><link rel="preconnect" href="https://spoor-api.ft.com"/><link rel="preconnect" href="https://session-next.ft.com" crossorigin="use-credentials"/><link rel="preconnect" href="https://ads-api.ft.com"/><link rel="preconnect" href="https://securepubads.g.doubleclick.net"/><link rel="preload" as="script" href="https://polyfill.io/v3/polyfill.min.js?features=default%2Ces5%2Ces2015%2Ces2016%2Ces2017%2CEventSource%2Cfetch%2CHTMLPictureElement%2CIntersectionObserver%2CNodeList.prototype.forEach&source=next"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/webpack-runtime.fb5e068cad11.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/page-kit-components.1513323d75c9.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/n-myft-ui.78a50e3b14f3.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/n-syndication.6e7d857310aa.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/next-myft-client.02db23fe8243.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-header.420bcf0da422.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/n-feedback.660bffa3ce99.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-forms.250ed6a82193.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/n-notification.84cea8e6e685.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/n-ui-foundations.1b0df1266cc8.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/n-topic-search.0c43cda5a57d.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/next-session-client.a488dae6869c.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-footer.57a8408c9b7e.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-overlay.7b4d116df926.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-toggle.5a3270c518f4.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-tooltip.0796fdb88c18.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-lazy-load.4de28350ea06.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-typography.ce9b4ffdce4b.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-viewport.065b502163a9.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/superstore.0c70d59fdcef.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-grid.80dbe6cd6af0.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-layers.1f9711dad6b6.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/o-utils.ebe34c02de08.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/financial-times-o-tracking.48d47e20d695.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/financial-times-o-ads.0d30a8c04dfc.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/shared.stable.4968dd492964.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/financial-times-o-viewport.407fd17a304c.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/financial-times-n-tracking.7ccb42af2e1a.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/financial-times-o-grid.8e531e66a66a.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/financial-times-o-utils.039203628d34.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/regenerator-runtime.46f0c42efcfc.bundle.js" type="application/javascript"/><link rel="preload" as="script" href="https://www.ft.com/__assets/hashed/page-kit/scripts.43dd2b34f27e.bundle.js" type="application/javascript"/><link rel="preload" as="font" href="https://www.ft.com/__origami/service/build/v2/files/o-fonts-assets@1.5.0/MetricWeb-Regular.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="preload" as="font" href="https://www.ft.com/__origami/service/build/v2/files/o-fonts-assets@1.5.0/MetricWeb-Semibold.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="preload" as="font" href="https://www.ft.com/__origami/service/build/v2/files/o-fonts-assets@1.5.0/FinancierDisplayWeb-Regular.woff2" type="font/woff2" crossorigin="anonymous"/><link rel="preload" as="font" href="https://www.ft.com/__origami/service/build/v2/files/o-fonts-assets@1.5.0/FinancierDisplayWeb-Bold.woff2" type="font/woff2" crossorigin="anonymous"/><script id="initial-props" type="application/json"></script><link rel="stylesheet" href="https://www.ft.com/__assets/hashed/page-kit/vendors~page-kit-layout-styles.573f9dcf07e4.css"/><link rel="stylesheet" href="https://www.ft.com/__assets/hashed/page-kit/styles.69c84ceec394.css"/><script type="application/json" id="page-kit-bootstrap-config">{"trackErrors":true,"core":["https://polyfill.io/v3/polyfill.min.js?features=HTMLPictureElement&source=next"],"enhanced":["https://polyfill.io/v3/polyfill.min.js?features=default%2Ces5%2Ces2015%2Ces2016%2Ces2017%2CEventSource%2Cfetch%2CHTMLPictureElement%2CIntersectionObserver%2CNodeList.prototype.forEach&source=next","https://www.ft.com/__assets/hashed/page-kit/webpack-runtime.fb5e068cad11.bundle.js","https://www.ft.com/__assets/hashed/page-kit/page-kit-components.1513323d75c9.bundle.js","https://www.ft.com/__assets/hashed/page-kit/n-myft-ui.78a50e3b14f3.bundle.js","https://www.ft.com/__assets/hashed/page-kit/n-syndication.6e7d857310aa.bundle.js","https://www.ft.com/__assets/hashed/page-kit/next-myft-client.02db23fe8243.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-header.420bcf0da422.bundle.js","https://www.ft.com/__assets/hashed/page-kit/n-feedback.660bffa3ce99.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-forms.250ed6a82193.bundle.js","https://www.ft.com/__assets/hashed/page-kit/n-notification.84cea8e6e685.bundle.js","https://www.ft.com/__assets/hashed/page-kit/n-ui-foundations.1b0df1266cc8.bundle.js","https://www.ft.com/__assets/hashed/page-kit/n-topic-search.0c43cda5a57d.bundle.js","https://www.ft.com/__assets/hashed/page-kit/next-session-client.a488dae6869c.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-footer.57a8408c9b7e.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-overlay.7b4d116df926.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-toggle.5a3270c518f4.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-tooltip.0796fdb88c18.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-lazy-load.4de28350ea06.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-typography.ce9b4ffdce4b.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-viewport.065b502163a9.bundle.js","https://www.ft.com/__assets/hashed/page-kit/superstore.0c70d59fdcef.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-grid.80dbe6cd6af0.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-layers.1f9711dad6b6.bundle.js","https://www.ft.com/__assets/hashed/page-kit/o-utils.ebe34c02de08.bundle.js","https://www.ft.com/__assets/hashed/page-kit/financial-times-o-tracking.48d47e20d695.bundle.js","https://www.ft.com/__assets/hashed/page-kit/financial-times-o-ads.0d30a8c04dfc.bundle.js","https://www.ft.com/__assets/hashed/page-kit/shared.stable.4968dd492964.bundle.js","https://www.ft.com/__assets/hashed/page-kit/financial-times-o-viewport.407fd17a304c.bundle.js","https://www.ft.com/__assets/hashed/page-kit/financial-times-n-tracking.7ccb42af2e1a.bundle.js","https://www.ft.com/__assets/hashed/page-kit/financial-times-o-grid.8e531e66a66a.bundle.js","https://www.ft.com/__assets/hashed/page-kit/financial-times-o-utils.039203628d34.bundle.js","https://www.ft.com/__assets/hashed/page-kit/regenerator-runtime.46f0c42efcfc.bundle.js","https://www.ft.com/__assets/hashed/page-kit/scripts.43dd2b34f27e.bundle.js"]}</script><script>;(function () {
      var doc = document.documentElement
      var isEnhanced = isEnhancedBrowser()
      var scriptsConfig = getScriptsConfig()
      var scriptsToLoad = []
      var currentScript = document.scripts[document.scripts.length - 1]

      doc.className = doc.className.replace('no-js', 'js')

      if (isEnhanced) {
        doc.className = doc.className.replace('core', 'enhanced')
        Array.prototype.push.apply(scriptsToLoad, scriptsConfig.enhanced)
      } else {
        Array.prototype.push.apply(scriptsToLoad, scriptsConfig.core)
      }

      for (var i = 0, len = scriptsToLoad.length; i < len; i++) {
        loadScript(scriptsToLoad[i])
      }

      function scriptLoadError(error) {
        var script = error.target ? error.target.src : null

        if (script) {
          console.error('The script ' + script + ' failed to load') // eslint-disable-line no-console
        }

        if (/enhanced/.test(doc.className)) {
          console.warn('Script loading failed, reverting to core experience') // eslint-disable-line no-console
          doc.className = doc.className.replace('enhanced', 'core')
        }

        if (scriptsConfig.trackErrors) {
          addErrorTrackingPixel(script)
        }
      }

      function loadScript(src) {
        var script = document.createElement('script')
        script.onerror = scriptLoadError
        script.async = false
        script.src = src
        currentScript.parentNode.insertBefore(script, currentScript)
      }

      // "Cut the mustard" test
      // by Maggie Allen and Matt Hinchliffe November 2018
      function isEnhancedBrowser() {
        var script = document.createElement('script')
        var input = document.createElement('input')

        return (
          'visibilityState' in document && // not supported by old Android (4.0-4.4) without a prefix
          'indeterminate' in input && // not supported by BB 10
          'flex' in doc.style && // not supported by old Safari (< 9) or IE 6-10
          'async' in script // not supported by old Opera (Presto engine < 15)
        )
      }

      function getScriptsConfig() {
        var scriptsConfigEl = document.getElementById('page-kit-bootstrap-config')
        var scriptsConfig = { core: [], enhanced: [], trackErrors: false }

        if (scriptsConfigEl) {
          try {
            scriptsConfig = JSON.parse(scriptsConfigEl.innerHTML)
          } catch (error) {
            console.error('Bootstrap configuration error', error) // eslint-disable-line no-console
          }
        }

        return scriptsConfig
      }

      function addErrorTrackingPixel(script) {
        var img = new Image()

        var data = JSON.stringify({
          category: 'javascript',
          action: 'load-error',
          system: {
            source: 'page-kit'
          },
          context: {
            script: script
          }
        })

        img.src = 'https://spoor-api.ft.com/px.gif?data=' + encodeURIComponent(data)
      }
    })()
    </script><script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
              new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
                j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
                'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
            })(window,document,'script','dataLayer','GTM-NWQJW68');</script><script>(function loadCustomFonts() {
        var rootElement = document.documentElement;
        if (/(^|\s)o-typography-fonts-loaded=1(;|$)/.test(document.cookie)) {
            var fontLabels = ['sans', 'sans-bold', 'display', 'display-bold'];
            for (var i = 0; i < fontLabels.length; i++) {
                rootElement.className = rootElement.className.replace('o-typography--loading-' + fontLabels[i], '');
            }
        }
    }());</script></head><body><noscript><iframe src="https://www.googletagmanager.com/ns.html?id=GTM-NWQJW68" height="0" width="0" style="display:none;visibility:hidden"></iframe></noscript><div class="n-layout"><a data-trackable="a11y-skip-to-help" class="n-layout__skip-link" href="https://www.ft.com/accessibility">Accessibility help</a><a data-trackable="a11y-skip-to-navigation" class="n-layout__skip-link" href="#site-navigation">Skip to navigation</a><a data-trackable="a11y-skip-to-content" class="n-layout__skip-link" href="#site-content">Skip to content</a><a data-trackable="a11y-skip-to-footer" class="n-layout__skip-link" href="#site-footer">Skip to footer</a><div class="n-layout__row n-layout__row--header"><div class="n-layout__header-before"><script type="application/json" id="n-ads-options">{}</script><div class="o-ads o-ads--center " data-o-ads-name="leaderboard" data-o-ads-targeting="pos=top" data-o-ads-formats-large="SuperLeaderboard,Leaderboard,Responsive" data-o-ads-formats-extra="Billboard,SuperLeaderboard,Leaderboard,Responsive" style="width:100%;text-align:center"></div></div><header id="site-navigation" class="o-header o-header--simple" data-o-component="o-header" data-o-header--no-js="true" tabindex="-1"><div class="o-header__row o-header__anon" data-trackable="header-anon"><ul class="o-header__anon-list"><li class="o-header__anon-item"><a class="o-header__anon-link" href="/login?location=/search" data-trackable="Sign In">Sign In</a></li><li class="o-header__anon-item"><a class="o-header__anon-link" href="/products?segmentId=f860e6c2-18af-ab30-cd5e-6e3a456f9265" data-trackable="Subscribe">Subscribe</a></li></ul></div><div class="o-header__row o-header__top" data-trackable="header-top"><div class="o-header__container"><div class="o-header__top-wrapper"><div class="o-header__top-column o-header__top-column--left"><a href="#o-header-drawer" class="o-header__top-link o-header__top-link--menu" aria-controls="o-header-drawer" title="Open side navigation menu" data-trackable="drawer-toggle"><span class="o-header__top-link-label">Open side navigation menu</span></a><a href="#o-header-search-primary" class="o-header__top-link o-header__top-link--search" aria-controls="o-header-search-primary" title="Open search bar" data-trackable="search-toggle"><span class="o-header__top-link-label">Open search bar</span></a></div><div class="o-header__top-column o-header__top-column--center"><a class="o-header__top-logo" style="background-image:none" data-trackable="logo" href="/" title="Go to Financial Times homepage"><svg viewBox="0 0 1054 86"><title>Financial Times</title><path d="M26.177 72.609c0 5.938 1.697 7.295 12.554 7.295v3.732H.9v-3.732c7.464 0 10.01-.679 10.01-7.125V12.215c0-6.447-2.546-7.126-10.01-7.126V1.357h63.448l.34 22.563h-3.054C59.937 6.956 55.696 5.43 39.919 5.43H26.008v33.59h11.196c10.688 0 11.367-1.697 12.215-10.179h3.054v24.599h-3.054c-.848-8.482-1.527-10.01-12.215-10.01H26.008v29.18h.17zm46.314 11.027v-3.732c7.465 0 10.01-.679 10.01-7.125V12.215c0-6.447-2.545-7.126-10.01-7.126V1.357h35.287V5.09c-7.465 0-10.01.679-10.01 7.126v60.564c0 6.446 2.545 7.125 10.01 7.125v3.732H72.49zm115.36 1.357l-56.323-69.725V72.44c0 6.617 4.58 7.634 12.385 7.634v3.733h-29.858v-3.733c7.803 0 12.045-1.017 12.045-7.634V8.991c-3.563-3.732-6.108-3.902-12.045-3.902V1.357h26.465l43.09 55.475V12.554c0-6.616-4.58-7.634-12.384-7.634V1.357h30.027V5.09c-7.803 0-12.045 1.018-12.045 7.635v72.27h-1.357zm40.207-1.357h-29.689v-3.732c7.804 0 11.367-1.018 13.911-7.804L239.085.509h7.464l28.84 72.1c2.545 6.447 3.732 7.125 9.67 7.125v3.732h-34.438v-3.732c10.518 0 11.536-.848 8.99-7.125l-8.481-21.884h-25.787L217.71 71.93c-2.375 6.447 1.357 7.804 10.518 7.804v3.902h-.17zm-1.188-37.153h22.393l-11.705-29.518-10.688 29.518zm135.548 38.51l-56.153-69.725V72.44c0 6.617 4.58 7.634 12.384 7.634v3.733h-29.18v-3.733c7.126 0 11.367-1.017 11.367-7.634V9.161c-4.071-3.732-7.125-4.072-13.91-4.072V1.357h28.16l43.09 55.475V12.554c0-6.616-4.58-7.634-12.383-7.634V1.357h29.858V5.09c-7.804 0-12.045 1.018-12.045 7.635v72.27h-1.188zm83.297-83.805h2.036l1.187 24.598-3.053.17c-2.036-14.08-9.5-21.545-23.242-21.545-15.268 0-26.804 13.063-26.804 33.081 0 25.617 16.116 40.206 33.08 40.206 7.296 0 13.912-2.035 20.358-8.99l2.376 2.374c-5.26 7.465-15.608 13.742-29.52 13.742-20.696 0-41.732-15.608-41.732-41.734C380.4 17.813 399.57 0 422.813 0c11.027 0 16.795 4.75 19.848 4.75 1.357 0 2.375-1.187 3.054-3.562zm12.723 82.448v-3.732c7.465 0 10.01-.679 10.01-7.125V12.215c0-6.447-2.545-7.126-10.01-7.126V1.357h35.287V5.09c-7.464 0-10.01.679-10.01 7.126v60.564c0 6.446 2.546 7.125 10.01 7.125v3.732h-35.287zm68.538 0h-27.653v-3.732c6.108 0 9.331-1.018 11.876-7.804L538.003.509h7.464l28.84 72.1c2.545 6.447 3.733 7.125 9.67 7.125v3.732H549.54v-3.732c10.518 0 11.536-.848 8.991-7.125l-8.482-21.884h-25.786l-7.635 21.205c-2.375 6.447 1.358 7.804 10.519 7.804v3.902h-.17zm-1.188-37.153h22.394l-11.536-29.518-10.858 29.518zm63.957 37.153v-3.732c7.465 0 10.01-.679 10.01-7.125V12.215c0-6.447-2.545-7.126-10.01-7.126V1.357h35.117V5.09c-7.464 0-9.84.679-9.84 7.126v61.073c0 5.428 2.715 6.107 7.126 6.107h4.241c15.947 0 21.036-2.375 25.447-20.527l3.054.339-2.545 24.26h-62.6v.17zM760.75 1.357l.339 23.92h-3.054C756.34 7.634 752.098 5.43 736.32 5.43h-5.089v67.18c0 6.447 2.375 7.295 12.554 7.295v3.732h-40.376v-3.732c10.179 0 12.723-1.018 12.723-7.295V5.429h-5.089c-15.777 0-20.018 2.205-21.715 19.848h-3.053l.339-23.92h74.136zm7.973 82.28v-3.733c7.465 0 10.01-.679 10.01-7.125V12.215c0-6.447-2.545-7.126-10.01-7.126V1.357h35.287V5.09c-7.465 0-10.01.679-10.01 7.126v60.564c0 6.446 2.545 7.125 10.01 7.125v3.732h-35.287zM910.21 1.356V5.09c-7.465 0-10.688.34-10.01 6.956l6.447 61.073c.679 6.277 3.054 6.955 10.518 6.955v3.733h-35.117v-3.733c7.295 0 9.84-.678 9.331-6.955L884.762 8.99l-25.956 76.172h-1.018L832.34 8.822l-6.108 64.126c-.678 6.447 3.733 7.125 11.197 7.125v3.733h-27.483v-3.733c7.465 0 10.01-1.187 10.518-7.125l6.447-61.073c.679-6.446-2.545-6.955-10.01-6.955V1.357h28.84l17.305 56.153 18.661-56.153h28.5zm65.653 52.082h-3.053c-.849-8.482-1.527-10.01-12.215-10.01H948.04v29.859c0 5.428 2.715 6.107 7.125 6.107h6.786c15.947 0 21.036-2.375 25.447-20.527l3.054.339-2.884 24.26h-64.805v-3.733c7.464 0 10.009-.678 10.009-7.125V12.215c0-6.447-2.545-7.126-10.01-7.126V1.357h62.261l.34 20.527h-3.054c-1.866-14.59-5.598-16.286-21.885-16.286H948.21v33.42h12.554c10.687 0 11.366-1.696 12.214-10.178h3.054v24.599h-.17zm65.484 13.232c0-7.464-4.75-11.196-12.893-15.777l-13.063-6.786c-9.84-5.259-15.607-11.027-15.607-21.375C999.783 9.84 1010.81 0 1025.23 0c9.84 0 14.929 4.75 17.813 4.75 1.866 0 2.714-1.187 3.562-3.562h2.375l1.188 23.072-3.054.17c-1.696-11.198-9.67-19.85-20.866-19.85-8.483 0-14.081 5.09-14.081 12.215 0 7.804 5.937 11.027 12.554 14.59l11.196 5.937c10.519 5.768 17.983 11.536 17.983 22.563 0 14.59-12.554 24.939-28.161 24.939-11.028 0-16.456-5.26-19.34-5.26-1.866 0-2.884 1.697-3.732 4.242h-2.376l-1.696-24.43 3.054-.339c2.375 15.268 12.893 21.545 23.411 21.545 8.822-.17 16.286-4.071 16.286-13.91z" fill="#231F20" fill-rule="evenodd"></path></svg></a></div><div class="o-header__top-column o-header__top-column--right"><a class="o-header__top-link o-header__top-link--myft" href="/myft" data-trackable="my-ft" data-tour-stage="myFt" aria-label="My F T"><span class="o-header__visually-hidden">myFT</span></a></div></div></div></div><div id="o-header-search-primary" class="o-header__row o-header__search o-header__search--primary" data-trackable="header-search" data-o-header-search="true"><div class="o-header__container"><form class="o-header__search-form" action="/search" role="search" aria-label="Site search" data-n-topic-search="true" data-n-topic-search-categories="concepts,equities" data-n-topic-search-view-all="true"><label class="o-header__visually-hidden" for="o-header-search-term-primary">Search the <abbr title="Financial Times">FT</abbr></label><input type="text" class="o-header__search-term" id="o-header-search-term-primary" name="q" autoComplete="off" autoCorrect="off" autoCapitalize="off" spellcheck="false" data-trackable="search-term" placeholder="Search the FT" data-n-topic-search-input="true"/><button class="o-header__search-submit" type="submit" data-trackable="search-submit">Search</button><button class="o-header__search-close o--if-js" type="button" aria-controls="o-header-search-primary" title="Close search bar" data-trackable="close"><span class="o-header__visually-hidden">Close search bar</span></button></form></div></div><nav id="o-header-nav-desktop" class="o-header__row o-header__nav o-header__nav--desktop" role="navigation" aria-label="Primary navigation" data-trackable="header-nav:desktop"><div class="o-header__container"><ul class="o-header__nav-list o-header__nav-list--left" data-trackable="primary-nav"><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/" id="o-header-link-0" data-trackable="Home">Home</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/world" id="o-header-link-1" data-trackable="World">World</a><div class="o-header__mega" id="o-header-mega-1" role="group" aria-labelledby="o-header-link-1" data-o-header-mega="true" data-trackable="meganav | World"><div class="o-header__container"><div class="o-header__mega-wrapper"><div class="o-header__mega-column o-header__mega-column--subsections" data-trackable="sections"><div class="o-header__mega-heading">Sections</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world" data-trackable="link">World Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/global-economy" data-trackable="link">Global Economy</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/uk" data-trackable="link">UK</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/us" data-trackable="link">US</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/asia-pacific/china" data-trackable="link">China</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/africa" data-trackable="link">Africa</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/asia-pacific" data-trackable="link">Asia Pacific</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/emerging-markets" data-trackable="link">Emerging Markets</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/europe" data-trackable="link">Europe</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/americas" data-trackable="link">Americas</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/mideast" data-trackable="link">Middle East and North Africa</a></li></ul></div></div><div class="o-header__mega-column o-header__mega-column--articles" data-trackable="popular"><div class="o-header__mega-heading">Most Read</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/3a0434e5-8cac-4922-8f0f-062db0604115" data-trackable="link">Schools in England to reopen on March 8 under easing of lockdown</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/2626ec05-5bc5-4121-afc4-be62c29c4894" data-trackable="link">Israel finds BioNTech/Pfizer vaccine reduces virus transmission</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/6d4ff17b-b3a2-4f9a-bb73-84e50ba614b6" data-trackable="link">Covid vaccines cut UK deaths but lockdown still key in reducing infections </a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/13d23af0-5f0c-4215-a017-39f6b145d0e8" data-trackable="link">The medieval German town at the heart of the fight against Covid</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/fc8b51ac-e90c-48d3-bee0-d551303e1863" data-trackable="link">UK banks push for ambitious financial services strategy</a></li></ul></div></div></div></div></div></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/world/us" id="o-header-link-2" data-trackable="US">US</a><div class="o-header__mega" id="o-header-mega-2" role="group" aria-labelledby="o-header-link-2" data-o-header-mega="true" data-trackable="meganav | US"><div class="o-header__container"><div class="o-header__mega-wrapper"><div class="o-header__mega-column o-header__mega-column--subsections" data-trackable="sections"><div class="o-header__mega-heading">Sections</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/us" data-trackable="link">US Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/us-economy" data-trackable="link">US Economy</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/us" data-trackable="link">US Companies</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/world/us/politics" data-trackable="link">US Politics & Policy</a></li></ul></div></div><div class="o-header__mega-column o-header__mega-column--articles" data-trackable="popular"><div class="o-header__mega-heading">Most Read</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/2f5f7799-9886-4327-bb1d-f0c10aa2184b" data-trackable="link">Dozens of Boeing 777 aircraft grounded after United Airlines engine failure </a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/ae8e09ce-de94-48ec-98cb-cd222713a20f" data-trackable="link">US mortgage rates jump on higher Treasury yields</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/4076cf91-ad82-4143-b21b-ea1f9e75c19c" data-trackable="link">Silicon Beach, or just another warm tax haven?</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/736825e3-1893-48d5-b011-53518fdf3680" data-trackable="link">Billionaires’ Row developer raises cash after Manhattan property slump</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/da5c30a8-6fab-4131-b6bd-f8f05dcf5a46" data-trackable="link">UK competition watchdog warns Big Tech of coming antitrust probes</a></li></ul></div></div></div></div></div></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/companies" id="o-header-link-3" data-trackable="Companies">Companies</a><div class="o-header__mega" id="o-header-mega-3" role="group" aria-labelledby="o-header-link-3" data-o-header-mega="true" data-trackable="meganav | Companies"><div class="o-header__container"><div class="o-header__mega-wrapper"><div class="o-header__mega-column o-header__mega-column--subsections" data-trackable="sections"><div class="o-header__mega-heading">Sections</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies" data-trackable="link">Companies Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/energy" data-trackable="link">Energy</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/financials" data-trackable="link">Financials</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/health" data-trackable="link">Health</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/industrials" data-trackable="link">Industrials</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/media" data-trackable="link">Media</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/professional-services" data-trackable="link">Professional Services</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/retail-consumer" data-trackable="link">Retail & Consumer</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/technology" data-trackable="link">Tech Sector</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/telecoms" data-trackable="link">Telecoms</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/companies/transport" data-trackable="link">Transport</a></li></ul></div></div><div class="o-header__mega-column o-header__mega-column--articles" data-trackable="popular"><div class="o-header__mega-heading">Most Read</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/38c3670c-3b0f-41e6-874e-0f9eee553744" data-trackable="link">HSBC intensifies pivot to Asia with job moves and US exit</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/2f5f7799-9886-4327-bb1d-f0c10aa2184b" data-trackable="link">Dozens of Boeing 777 aircraft grounded after United Airlines engine failure </a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/ae8e09ce-de94-48ec-98cb-cd222713a20f" data-trackable="link">US mortgage rates jump on higher Treasury yields</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/f001f3ef-e296-4eb6-b711-5f7cb773e314" data-trackable="link">McKinsey senior echelons vote in referendum on Sneader leadership</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/736825e3-1893-48d5-b011-53518fdf3680" data-trackable="link">Billionaires’ Row developer raises cash after Manhattan property slump</a></li></ul></div></div></div></div></div></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/technology" id="o-header-link-4" data-trackable="Tech">Tech</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/markets" id="o-header-link-5" data-trackable="Markets">Markets</a><div class="o-header__mega" id="o-header-mega-5" role="group" aria-labelledby="o-header-link-5" data-o-header-mega="true" data-trackable="meganav | Markets"><div class="o-header__container"><div class="o-header__mega-wrapper"><div class="o-header__mega-column o-header__mega-column--subsections" data-trackable="sections"><div class="o-header__mega-heading">Sections</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/markets" data-trackable="link">Markets Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/alphaville" data-trackable="link">Alphaville</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="https://markets.ft.com/data" data-trackable="link">Markets Data</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/capital-markets" data-trackable="link">Capital Markets</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/commodities" data-trackable="link">Commodities</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/currencies" data-trackable="link">Currencies</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/equities" data-trackable="link">Equities</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/fund-management" data-trackable="link">Fund Management</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/ft-trading-room" data-trackable="link">Trading</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/moral-money" data-trackable="link">Moral Money</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="https://etf.ft.com" data-trackable="link">ETF Hub</a></li></ul></div></div><div class="o-header__mega-column o-header__mega-column--articles" data-trackable="popular"><div class="o-header__mega-heading">Most Read</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/683effc4-993a-4baf-bc17-8ba70b96c06a" data-trackable="link">Bond trading finally dragged into the digital age</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/da5c30a8-6fab-4131-b6bd-f8f05dcf5a46" data-trackable="link">UK competition watchdog warns Big Tech of coming antitrust probes</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/0e746280-e72c-4087-9c0d-df2a7af82b77" data-trackable="link">Bills mount in Texas power market after freeze sends prices soaring</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/84012e15-554c-485c-8064-dbc6b4098bca" data-trackable="link">Will inflation data justify investors’ concerns?</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/054d7d05-ed69-445b-bfa3-c64397d5fa74" data-trackable="link">Should equity investors worry about rising interest rates?</a></li></ul></div></div></div></div></div></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/climate-capital" id="o-header-link-6" data-trackable="Climate">Climate</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/opinion" id="o-header-link-7" data-trackable="Opinion">Opinion</a><div class="o-header__mega" id="o-header-mega-7" role="group" aria-labelledby="o-header-link-7" data-o-header-mega="true" data-trackable="meganav | Opinion"><div class="o-header__container"><div class="o-header__mega-wrapper"><div class="o-header__mega-column o-header__mega-column--subsections" data-trackable="sections"><div class="o-header__mega-heading">Sections</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/opinion" data-trackable="link">Opinion Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/columnists" data-trackable="link">Columnists</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/ft-view" data-trackable="link">The FT View</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/the-big-read" data-trackable="link">The Big Read</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/lex" data-trackable="link">Lex</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/obituaries" data-trackable="link">Obituaries</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/letters" data-trackable="link">Letters</a></li></ul></div></div><div class="o-header__mega-column o-header__mega-column--articles" data-trackable="popular"><div class="o-header__mega-heading">Most Read</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/217f6d28-5a3e-48e0-bf6e-c2618da8f34b" data-trackable="link">Why once successful countries like the UK get left behind</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/4076cf91-ad82-4143-b21b-ea1f9e75c19c" data-trackable="link">Silicon Beach, or just another warm tax haven?</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/683effc4-993a-4baf-bc17-8ba70b96c06a" data-trackable="link">Bond trading finally dragged into the digital age</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/fa86829e-1eb1-4cb2-b92f-482110a752cd" data-trackable="link">Uber judgment is set to reshape the gig economy</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/ebd22d66-c4bb-47a0-8a34-e3cf8e0472b4" data-trackable="link">Toxic offices don’t change when they go virtual</a></li></ul></div></div></div></div></div></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/work-careers" id="o-header-link-8" data-trackable="Work & Careers">Work & Careers</a><div class="o-header__mega" id="o-header-mega-8" role="group" aria-labelledby="o-header-link-8" data-o-header-mega="true" data-trackable="meganav | Work & Careers"><div class="o-header__container"><div class="o-header__mega-wrapper"><div class="o-header__mega-column o-header__mega-column--subsections" data-trackable="sections"><div class="o-header__mega-heading">Sections</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/work-careers" data-trackable="link">Work & Careers Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="https://rankings.ft.com/businessschoolrankings/rankings" data-trackable="link">Business School Rankings</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/business-education" data-trackable="link">Business Education</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/entrepreneurship" data-trackable="link">Entrepreneurship</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/recruitment" data-trackable="link">Recruitment</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/business-books" data-trackable="link">Business Books</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/business-travel" data-trackable="link">Business Travel</a></li></ul></div></div><div class="o-header__mega-column o-header__mega-column--articles" data-trackable="popular"><div class="o-header__mega-heading">Most Read</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/ebd22d66-c4bb-47a0-8a34-e3cf8e0472b4" data-trackable="link">Toxic offices don’t change when they go virtual</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/5c6742c6-b101-4bdb-9244-6d719460ec2e" data-trackable="link">Forced mediocrity can boost your creativity</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/1e07c7fd-b0bf-405d-ac4a-979990b2a7da" data-trackable="link">Delta’s Ed Bastian: building staff loyalty to survive a crisis</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/8025b36e-7f70-4072-97c7-3fde860883cf" data-trackable="link">The boardroom’s futile pursuit of purpose</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/ea4ff3d0-cbaa-4197-b2b8-6e92b1a822c3" data-trackable="link">How do I develop professionally while on maternity leave?</a></li></ul></div></div></div></div></div></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/life-arts" id="o-header-link-9" data-trackable="Life & Arts">Life & Arts</a><div class="o-header__mega" id="o-header-mega-9" role="group" aria-labelledby="o-header-link-9" data-o-header-mega="true" data-trackable="meganav | Life & Arts"><div class="o-header__container"><div class="o-header__mega-wrapper"><div class="o-header__mega-column o-header__mega-column--subsections" data-trackable="sections"><div class="o-header__mega-heading">Sections</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/life-arts" data-trackable="link">Life & Arts Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/arts" data-trackable="link">Arts</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/books" data-trackable="link">Books</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/food-drink" data-trackable="link">Food & Drink</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/magazine" data-trackable="link">FT Magazine</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/house-home" data-trackable="link">House & Home</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/style" data-trackable="link">Style</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/travel" data-trackable="link">Travel</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/globetrotter" data-trackable="link">FT Globetrotter</a></li></ul></div></div><div class="o-header__mega-column o-header__mega-column--articles" data-trackable="popular"><div class="o-header__mega-heading">Most Read</div><div class="o-header__mega-content"><ul class="o-header__mega-list"><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/c11bb885-1274-4677-ba05-fcbac67dc808" data-trackable="link">Bill Gates: My green manifesto</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/97441795-4701-4473-8230-1cf62114655b" data-trackable="link">Britney Spears: our part in her downfall</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/95f72178-d712-41bc-be53-13588e30025c" data-trackable="link">Magic beans: five of Tokyo’s top spots for coffee</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/f5c833f1-22d4-4d4e-81e2-b0ee67743909" data-trackable="link">Brexit hits bordeaux buffs</a></li><li class="o-header__mega-item"><a class="o-header__mega-link" href="/content/468d42c9-c107-4518-9716-ca841076af30" data-trackable="link">Line of fire: what it is to be a man in today’s British army</a></li></ul></div></div></div></div></div></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/htsi" id="o-header-link-10" data-trackable="How to Spend It">How to Spend It</a></li></ul><ul class="o-header__nav-list o-header__nav-list--right" data-trackable="user-nav"><li class="o-header__nav-item"><a class="o-header__nav-link" href="/login?location=/search" data-trackable="Sign In">Sign In</a></li><li class="o-header__nav-item o-header__nav-item--hide-s"><a class="o-header__nav-button" role="button" href="/products?segmentId=f860e6c2-18af-ab30-cd5e-6e3a456f9265" data-trackable="Subscribe">Subscribe</a></li></ul></div></nav></header><header class="o-header o-header--simple o-header--sticky o--if-js" data-o-component="o-header" data-o-header--sticky="true" aria-hidden="true"><div class="o-header__row o-header__top" data-trackable="header-sticky"><div class="o-header__container"><div class="o-header__top-wrapper"><div class="o-header__top-column o-header__top-column--left"><a href="#" class="o-header__top-link o-header__top-link--menu" aria-controls="o-header-drawer" data-trackable="drawer-toggle" tabindex="-1"><span class="o-header__top-link-label">Menu</span></a><a href="#" class="o-header__top-link o-header__top-link--search" aria-controls="o-header-search-sticky" data-trackable="search-toggle" tabindex="-1"><span class="o-header__top-link-label">Search</span></a></div><div class="o-header__top-column o-header__top-column--center"><div class="o-header__top-takeover"><div class="o-header__nav"><ul class="o-header__nav-list o-header__nav-list--left" data-trackable="primary-nav"><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/" data-trackable="Home" tabindex="-1">Home</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/world" data-trackable="World" tabindex="-1">World</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/world/us" data-trackable="US" tabindex="-1">US</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/companies" data-trackable="Companies" tabindex="-1">Companies</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/technology" data-trackable="Tech" tabindex="-1">Tech</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/markets" data-trackable="Markets" tabindex="-1">Markets</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/climate-capital" data-trackable="Climate" tabindex="-1">Climate</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/opinion" data-trackable="Opinion" tabindex="-1">Opinion</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/work-careers" data-trackable="Work & Careers" tabindex="-1">Work & Careers</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/life-arts" data-trackable="Life & Arts" tabindex="-1">Life & Arts</a></li><li class="o-header__nav-item"><a class="o-header__nav-link o-header__nav-link--primary" href="/htsi" data-trackable="How to Spend It" tabindex="-1">How to Spend It</a></li></ul></div></div><a class="o-header__top-logo o-header__hide--L" data-trackable="logo" href="/" title="Go to Financial Times homepage" tabindex="-1"><span class="o-header__visually-hidden">Financial Times</span></a></div><div class="o-header__top-column o-header__top-column--right"><div class="o-header__nav"><ul class="o-header__nav-list o-header__nav-list--right" data-trackable="user-nav"><li class="o-header__nav-item"><a class="o-header__nav-link" href="/login?location=/search" data-trackable="Sign In" tabindex="-1">Sign In</a></li><li class="o-header__nav-item o-header__nav-item--hide-s"><a class="o-header__nav-button" role="button" href="/products?segmentId=f860e6c2-18af-ab30-cd5e-6e3a456f9265" data-trackable="Subscribe" tabindex="-1">Subscribe</a></li></ul></div></div></div></div></div><div id="o-header-search-sticky" class="o-header__row o-header__search o-header__search--sticky" data-trackable="header-search" data-o-header-search="true"><div class="o-header__container"><form class="o-header__search-form" action="/search" role="search" aria-label="Site search" data-n-topic-search="true" data-n-topic-search-categories="concepts,equities" data-n-topic-search-view-all="true"><label class="o-header__visually-hidden" for="o-header-search-term-sticky">Search the <abbr title="Financial Times">FT</abbr></label><input type="text" class="o-header__search-term" id="o-header-search-term-sticky" name="q" autoComplete="off" autoCorrect="off" autoCapitalize="off" spellcheck="false" data-trackable="search-term" placeholder="Search the FT" data-n-topic-search-input="true"/><button class="o-header__search-submit" type="submit" data-trackable="search-submit">Search</button><button class="o-header__search-close o--if-js" type="button" aria-controls="o-header-search-sticky" title="Close search bar" data-trackable="close"><span class="o-header__visually-hidden">Close search bar</span></button></form></div></div></header></div><div class="n-layout__row n-layout__row--content"><div><div class="o-grid-container">

        <script id="js-search-status" type="application/json">
            {"searchQuery":"lch sa","searchResults":70,"searchPage":1,"searchSort":"relevance","searchMode":"quality","activeContentTypeFilter":false,"activeConceptFilter":false,"activeDateFilter":false,"hasRelatedTopics":true,"hasRelatedLinks":false,"hasSuggestions":false}
        </script>

        <div class="search-layout ">

            <header class="search-layout__header">
                <h1 class="search-layout__title">Search Results</h1>
                <div class="search-head" data-trackable="search-head">
                    <form class="search-head__form" role="search" data-trackable="search-form">
                        <input type="hidden" name="sort" value="relevance">
                        <label class="search-head__form-label o-forms-field o-forms-field--optional">
                            <span class="search-head__form-title o-forms-title o-normalise-visually-hidden">
                                <span class="search-head__form-title-main o-forms-title__main">Query</span>
                            </span>
                            <span class="search-head__form-submit-wrapper o-forms-input o-forms-input--text o-forms-input--suffix">
                                <input required autocomplete="off" type="text" name="q" placeholder="please enter a search term" value="lch sa">
                                <button class="search-head__form-submit o-buttons o-buttons--primary o-buttons--big o-buttons-icon o-buttons-icon--search o-buttons-icon--icon-only">
                                    <span class="o-buttons-icon__label">Search</span>
                                </button>
                            </span>
                        </label>
                    </form>

                    <div class="search-head__notes">
                        Need some help searching? <a href="#help-tooltip" id="help-tooltip-target">View our hints and tips</a>.
                        <div id="help-tooltip" class="search-head__notes-tooltip">
                            <div class="search-tips">
                                <p>Hints and tips:</p>
                                <ul>
                                    <li>Put a phrase in double quotes to search for an exact match, E.G. <a href="?q=%22theresa+may%22">"Theresa May"</a>.</li>
                                    <li>Combine searches with operators (AND, OR, NOT) between words, E.G. <a href="?q=oil+AND+gas+NOT+europe">Oil AND gas NOT Europe</a>.</li>
                                    <li>Use parentheses to group operators, E.G. <a href="?q=elon+musk+AND+%28tesla+OR+spacex%29">Elon Musk AND (Tesla OR SpaceX)</a>.</li>
                                </ul>
                            </div>
                        </div>
                    </div>

                    <div class="search-head__active" data-trackable="active-filters">
                        <h2 class="search-head__active-heading o-normalise-visually-hidden">Active refinements</h2>





                    </div>
                </div>
            </header>

            <main class="search-layout__main" role="main" id="site-content">



                <div class="search-results o-teaser-collection" data-trackable="search-results">
                    <div class="search-results__heading">
                        <div class="search-results__heading-refine">
                                <div class="search-results__heading-sort">
                                    <div class="search-sort" data-trackable="sort-order">
                                        <div class="search-sort__label">
                                            Sort by
                                        </div>
                                        <div class="search-sort__buttons-group o-buttons-group">
                                                <a
                                                    class="o-buttons o-buttons--secondary search-sort__link js-sort-link"
                                                    aria-selected="true"
                                                    href="?q&#x3D;lch%20sa"
                                                    rel="nofollow"
                                                    aria-label="Relevance  (this option is currently selected)"
                                                    data-trackable="sort-item">
                                                    Relevance
                                                </a>
                                                <a
                                                    class="o-buttons o-buttons--secondary search-sort__link js-sort-link"
                                                    aria-selected="false"
                                                    href="?q&#x3D;lch%20sa&sort&#x3D;date"
                                                    rel="nofollow"

                                                    data-trackable="sort-item">
                                                    Date
                                                </a>
                                        </div>
                                    </div>
                                </div>
                                <div class="search-results__heading-refine-toggle">
                                    <a
                                        href="?q=lch sa&expandRefinements=true"
                                        aria-expanded="false"
                                        class="search-results__heading-refine-toggle__link" id="js-refinements-toggle">
                                        <span class="search-results__heading-refine-toggle__expanded-text">
                                            Fewer search options
                                        </span>
                                        <span class="search-results__heading-refine-toggle__collapsed-text">
                                            Refine your search
                                        </span>
                                    </a>
                                </div>
                        </div>
                        <div class="search-results__heading-title">
                            <h2 class="o-teaser-collection__heading o-teaser-collection__heading--half-width">
                                Viewing results 1‒25 of 70
                            </h2>
                        </div>
                    </div>

                    <div id="js-refinements" class="search-results__refine" hidden>
                        <dl>
                            <div class="search-results__refinements">
                                <div class="search-results__refinement">
                                    <dt class="search-refine__list-name">Date</dt>
                                    <dd class="search-refine__list-body" data-trackable="by-date">
                                        <ul class="search-refine__params search-refine__params--filters">
                                                <li class="search-refine__params-item" >
                                                    <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&sort&#x3D;relevance&dateRange&#x3D;now-7d&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                        Last 7 days
                                                    </a>
                                                </li>
                                                <li class="search-refine__params-item" >
                                                    <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&sort&#x3D;relevance&dateRange&#x3D;now-30d&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                        Last 30 days
                                                    </a>
                                                </li>
                                                <li class="search-refine__params-item" >
                                                    <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&sort&#x3D;relevance&dateRange&#x3D;now-1y&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                        Last year
                                                    </a>
                                                </li>
                                        </ul>

                                        <form class="search-refine__form search-refine__form--date">
                                            <input type="hidden" name="expandRefinements" value="true">
                                            <input type="hidden" name="q" value="lch sa">

                                            <label class="search-refine__form-label o-forms-field">
                                                <span class="search-refine__form-title o-forms-title">
                                                    <span class="search-refine__form-title-main o-forms-title__main">From</span>
                                                </span>
                                                <span class="search-refine__form-input-wrapper o-forms-input o-forms-input--text">
                                                    <input type="date" class="search-refine__form-input" name="dateFrom" id="dateFrom" value="" placeholder="dd/mm/yyyy">
                                                </span>
                                            </label>

                                            <label class="search-refine__form-label o-forms-field">
                                                <span class="search-refine__form-title o-forms-title">
                                                    <span class="search-refine__form-title-main o-forms-title__main">To</span>
                                                </span>
                                                <span class="search-refine__form-input-wrapper o-forms-input o-forms-input--text">
                                                    <input type="date" class="search-refine__form-input" name="dateTo" id="dateTo" value="" placeholder="dd/mm/yyyy">
                                                </span>
                                            </label>
                                            <br>

                                            <button class="search-refine__form-submit o-buttons o-buttons--primary" type="submit" data-trackable="date-submit">
                                                Apply
                                            </button>
                                        </form>

                                    </dd>
                                </div>

                                <div class="search-results__refinement">
                                    <dt class="search-refine__list-name">Type</dt>
                                    <dd class="search-refine__list-body" data-trackable="by-type">
                                        <ul class="search-refine__params search-refine__params--filters">
                                                <li class="search-refine__params-item" >
                                                    <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&sort&#x3D;relevance&contentType&#x3D;article&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                        Articles
                                                    </a>
                                                </li>
                                                <li class="search-refine__params-item" >
                                                    <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&sort&#x3D;relevance&contentType&#x3D;video&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                        Videos
                                                    </a>
                                                </li>
                                                <li class="search-refine__params-item" >
                                                    <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&sort&#x3D;relevance&contentType&#x3D;podcast&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                        Podcasts
                                                    </a>
                                                </li>
                                        </ul>
                                    </dd>
                                </div>
                            </div>

                            <div class="search-results__refinements">
                                    <div class="search-results__refinement">
                                        <dt class="search-refine__list-name">Topic</dt>
                                        <dd class="search-refine__list-body" data-trackable="by-Topic">
                                            <ul class="search-refine__params search-refine__params--facets">
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;c47f4dfc-6879-4e95-accf-ca8cbe6a1f69&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Companies&nbsp;<span class="search-refine__params-count">57</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;82645c31-4426-4ef5-99c9-9df6e0940c00&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            World&nbsp;<span class="search-refine__params-count">49</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;c91b1fad-1097-468b-be82-9a8ff717d54c&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Markets&nbsp;<span class="search-refine__params-count">49</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;4ffcf359-5e68-49b0-a266-af9c79e9b632&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Stock exchange consolidation&nbsp;<span class="search-refine__params-count">42</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;b062fd6b-01fa-4049-a1bc-092f9f43c307&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Financials&nbsp;<span class="search-refine__params-count">40</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;bdb031d9-5ddc-4927-b95b-30fe5087a394&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Equity exchanges&nbsp;<span class="search-refine__params-count">37</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;9d13b836-df76-4445-b599-10ae378680f0&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Mergers & Acquisitions&nbsp;<span class="search-refine__params-count">26</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;a829e170-bd47-44e2-b504-bafcd70f1d38&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Financial services&nbsp;<span class="search-refine__params-count">26</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;b7ea3c33-ea8c-432e-bb7e-e3bbc8fdc2bb&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Equities&nbsp;<span class="search-refine__params-count">26</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;a0a517eb-6b64-45ed-9ec3-c4404ca7e2b0&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Clearing and settlement&nbsp;<span class="search-refine__params-count">24</span>
                                                        </a>
                                                    </li>
                                            </ul>
                                        </dd>
                                    </div>
                                    <div class="search-results__refinement">
                                        <dt class="search-refine__list-name">Organisation</dt>
                                        <dd class="search-refine__list-body" data-trackable="by-Organisation">
                                            <ul class="search-refine__params search-refine__params--facets">
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;214c6ab8-1eea-48ad-99d2-ac55b0235e05&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            London Stock Exchange Group&nbsp;<span class="search-refine__params-count">41</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;69a384b2-ca58-4a1c-9a08-1eba88ea865d&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Clearnet SA&nbsp;<span class="search-refine__params-count">26</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;e9b9c404-bef4-4825-8a62-eb04e73bab80&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Deutsche Borse AG&nbsp;<span class="search-refine__params-count">26</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;1208d175-110b-4a0e-83d9-223bc87ef121&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            LCH.Clearnet Ltd&nbsp;<span class="search-refine__params-count">20</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;2b0f79dd-e9a7-4115-b573-77d35950119b&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            LCH.Clearnet Group Ltd&nbsp;<span class="search-refine__params-count">17</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;d276fab5-912f-40b7-808a-4cdda732d74e&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Euronext NV&nbsp;<span class="search-refine__params-count">15</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;80e05c2d-a888-41ca-8cdf-2fb135b67523&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            NYSE Euronext Inc&nbsp;<span class="search-refine__params-count">12</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;2571cbb7-ae30-4c72-b9f5-2dc081a02acf&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Intercontinental Exchange&nbsp;<span class="search-refine__params-count">10</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;016e34d3-2fcd-4263-a239-69ce06000789&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Lender Processing Services Inc&nbsp;<span class="search-refine__params-count">7</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;bb926f42-f246-4c84-9528-9f53b1f9acb4&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Eurex Clearing AG&nbsp;<span class="search-refine__params-count">7</span>
                                                        </a>
                                                    </li>
                                            </ul>
                                        </dd>
                                    </div>
                                    <div class="search-results__refinement">
                                        <dt class="search-refine__list-name">Location</dt>
                                        <dd class="search-refine__list-body" data-trackable="by-Location">
                                            <ul class="search-refine__params search-refine__params--facets">
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;6374b579-f67d-4970-a7cd-1ab5e41a551a&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Europe&nbsp;<span class="search-refine__params-count">39</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;6b683eff-56c3-43d9-acfc-7511d974fc01&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            UK&nbsp;<span class="search-refine__params-count">28</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;3e2eb1c1-7ecd-4600-8cbb-c02ba53ced4b&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            US&nbsp;<span class="search-refine__params-count">18</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;3a351c6d-ab15-42d7-bf94-4a5295be57f8&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            London&nbsp;<span class="search-refine__params-count">15</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;4327e5de-2361-40f9-bc70-47fb918cb1ba&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            England&nbsp;<span class="search-refine__params-count">15</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;52d8a898-27e1-4bca-9d7d-9bb5e6ce4706&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            France&nbsp;<span class="search-refine__params-count">13</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;c6a5c09e-ae56-304b-ae9a-d809927edd33&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Western Europe&nbsp;<span class="search-refine__params-count">11</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;6a0fcd79-11f6-43b3-9382-72e03c6871c5&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Paris&nbsp;<span class="search-refine__params-count">9</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;76ec50eb-1ebb-4931-8e34-8f601a16dcca&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Americas&nbsp;<span class="search-refine__params-count">7</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;4bfc6c35-6e2b-3dab-bc46-6b784a2505ce&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            North America&nbsp;<span class="search-refine__params-count">6</span>
                                                        </a>
                                                    </li>
                                            </ul>
                                        </dd>
                                    </div>
                                    <div class="search-results__refinement">
                                        <dt class="search-refine__list-name">Genre</dt>
                                        <dd class="search-refine__list-body" data-trackable="by-Genre">
                                            <ul class="search-refine__params search-refine__params--facets">
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;a579350c-61ce-4c00-97ca-ddaa2e0cacf6&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            News&nbsp;<span class="search-refine__params-count">36</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;6da31a37-691f-4908-896f-2829ebe2309e&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Opinion&nbsp;<span class="search-refine__params-count">16</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;7d62ed08-5d3b-4434-a03e-ca31765affc2&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Market report&nbsp;<span class="search-refine__params-count">10</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;06610896-2754-4847-a48f-9363d18a8c5e&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Feature&nbsp;<span class="search-refine__params-count">1</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;8a2cd02a-a97d-40be-b431-07ab2b26fe23&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            News in-depth&nbsp;<span class="search-refine__params-count">1</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;8c04f8e2-4048-4a2a-9fd2-706ed511c835&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Newsletter&nbsp;<span class="search-refine__params-count">1</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;aa08a644-46db-4753-ba06-f3608f369cbb&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Interview&nbsp;<span class="search-refine__params-count">1</span>
                                                        </a>
                                                    </li>
                                            </ul>
                                        </dd>
                                    </div>
                                    <div class="search-results__refinement">
                                        <dt class="search-refine__list-name">Author</dt>
                                        <dd class="search-refine__list-body" data-trackable="by-Author">
                                            <ul class="search-refine__params search-refine__params--facets">
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;e83182d4-c40e-4eb8-943c-3dc2e0c233a0&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Philip Stafford&nbsp;<span class="search-refine__params-count">22</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;3cb20da6-fefe-4a08-ace7-0cedd0ba5f4d&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Joseph Cotterill&nbsp;<span class="search-refine__params-count">3</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;527ccb94-eeb0-4ea1-ac74-98b175a01a08&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            James Shotter&nbsp;<span class="search-refine__params-count">3</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;de7d3273-899d-47b4-9bbd-fa9ffaaf59c2&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Mehreen Khan&nbsp;<span class="search-refine__params-count">2</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;03bd0b5a-12fe-4c19-85e9-650d3f1afc90&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Patrick Jenkins&nbsp;<span class="search-refine__params-count">1</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;1ba0db85-8600-470a-8222-bab11f47b119&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Sarah Provan&nbsp;<span class="search-refine__params-count">1</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;1e067c0b-7150-46c8-b330-f146bef98bb6&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Lisa Pollack&nbsp;<span class="search-refine__params-count">1</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;48be9dcf-7558-4b4c-9e39-255e8d8acf1d&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Hannah Murphy&nbsp;<span class="search-refine__params-count">1</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;4d03ee0c-122c-4a29-a7d1-d51f36600cca&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Richard Blackden&nbsp;<span class="search-refine__params-count">1</span>
                                                        </a>
                                                    </li>
                                                    <li class="search-refine__params-item" >
                                                        <a class="search-refine__params-link search-refine__params-link--unselected" href="?q&#x3D;lch%20sa&concept&#x3D;589ffc47-46b8-3390-a5b7-5b8ef2f2a8c9&sort&#x3D;relevance&expandRefinements&#x3D;true" rel="nofollow"  data-trackable="params-item">
                                                            Gwen Robinson&nbsp;<span class="search-refine__params-count">1</span>
                                                        </a>
                                                    </li>
                                            </ul>
                                        </dd>
                                    </div>
                            </div>
                        </dl>
                    </div>

                        <div class="search-results__related">
                            <div class="search-related" data-trackable="search-related">
                                <h3 class="search-related__heading o-normalise-visually-hidden">
                                    Related links
                                </h3>
                                <div class="search-related__container">
                                        <p class="search-related__list-heading">
                                            Related Topics
                                        </p>
                                        <ul class="search-related__list search-related__list--topics">
                                                <li class="search-related__list-item">
                                                    <a class="search-related__list-link" href="https://www.ft.com/stream/3da687f2-11a7-4e28-a18d-17b77aa18460" data-trackable="suggested-topic">
                                                        LCH.Clearnet SA
                                                    </a>
                                                </li>
                                        </ul>
                                </div>
                            </div>
                        </div>

                        <ul class="search-results__list" data-trackable="search-results-list">
                                <li class="search-results__list-item" data-trackable="item-0">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image o-teaser--opinion js-teaser" data-id="936e463e-2492-11e9-b329-c7e6ceb5ffdf"><div class="o-teaser__content"><div class="o-teaser__meta"><span class="o-teaser__tag-prefix">Lex</span><a class="o-teaser__tag" data-trackable="teaser-tag" href="/stream/214c6ab8-1eea-48ad-99d2-ac55b0235e05" aria-label="Category: London Stock Exchange Group">London Stock Exchange Group</a></div><div class="o-teaser__heading"><a href="/content/936e463e-2492-11e9-b329-c7e6ceb5ffdf" data-trackable="heading-link" class="js-teaser-heading-link">LSE/Brexit: clear and present danger</a><span> <span class="o-labels o-labels--premium o-labels--content-premium" aria-label="Premium content">Premium</span></span></div><p class="o-teaser__standfirst"><a href="/content/936e463e-2492-11e9-b329-c7e6ceb5ffdf" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...settled post-Brexit within the eurozone.

                                    LSE bought Borsa Italiana in 2007. <mark class="search-item__highlight">LCH</mark>, its clearing arm, has a Paris operation. Business could be diverted there. But much of the collateral held by <mark class="search-item__highlight">LCH</mark>...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2019-01-30T15:31:43+0000">January 30, 2019</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="936e463e-2492-11e9-b329-c7e6ceb5ffdf"
                                        data-myft-ui="saved"
                                        action="/myft/save/936e463e-2492-11e9-b329-c7e6ceb5ffdf"
                                        data-js-action="/__myft/api/core/saved/content/936e463e-2492-11e9-b329-c7e6ceb5ffdf?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="936e463e-2492-11e9-b329-c7e6ceb5ffdf" 
                                                aria-label="Save LSE/Brexit: clear and present danger to myFT for later"
                                                title="Save LSE/Brexit: clear and present danger to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/936e463e-2492-11e9-b329-c7e6ceb5ffdf" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2Fb71001e8-24a3-11e9-8ce6-5db4543da632?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-1">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="6fdeda50-245e-11e9-b329-c7e6ceb5ffdf"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/ft-trading-room/clearing-settlement" aria-label="Category: Clearing and settlement">Clearing and settlement</a></div><div class="o-teaser__heading"><a href="/content/6fdeda50-245e-11e9-b329-c7e6ceb5ffdf" data-trackable="heading-link" class="js-teaser-heading-link">LSE buys stake in Euroclear as rules reshape markets</a></div><p class="o-teaser__standfirst"><a href="/content/6fdeda50-245e-11e9-b329-c7e6ceb5ffdf" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...equity exchange and <mark class="search-item__highlight">LCH</mark>, the world’s largest swaps clearing house, means the LSE has plenty at stake.

                                    “It’s really about having a say in a key supplier, as most of the $160bn in physical collateral held...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2019-01-30T07:53:08+0000">January 30, 2019</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="6fdeda50-245e-11e9-b329-c7e6ceb5ffdf"
                                        data-myft-ui="saved"
                                        action="/myft/save/6fdeda50-245e-11e9-b329-c7e6ceb5ffdf"
                                        data-js-action="/__myft/api/core/saved/content/6fdeda50-245e-11e9-b329-c7e6ceb5ffdf?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="6fdeda50-245e-11e9-b329-c7e6ceb5ffdf" 
                                                aria-label="Save LSE buys stake in Euroclear as rules reshape markets to myFT for later"
                                                title="Save LSE buys stake in Euroclear as rules reshape markets to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/6fdeda50-245e-11e9-b329-c7e6ceb5ffdf" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2F1541c992-2461-11e9-b329-c7e6ceb5ffdf?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-2">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image o-teaser--opinion js-teaser" data-id="0fcfff86-6e5b-11e8-92d3-6c13e5c92914"><div class="o-teaser__content"><div class="o-teaser__meta"><span class="o-teaser__tag-prefix">Lex</span><a class="o-teaser__tag" data-trackable="teaser-tag" href="/stream/45c34a4b-8c55-4535-b308-fb4bf52f2833" aria-label="Category: UK banks">UK banks</a></div><div class="o-teaser__heading"><a href="/content/0fcfff86-6e5b-11e8-92d3-6c13e5c92914" data-trackable="heading-link" class="js-teaser-heading-link">Brexit/derivatives: compression injury</a><span> <span class="o-labels o-labels--premium o-labels--content-premium" aria-label="Premium content">Premium</span></span></div><p class="o-teaser__standfirst"><a href="/content/0fcfff86-6e5b-11e8-92d3-6c13e5c92914" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...value of interest rate swaps cleared from €1tn to €7tn in the year to date.

                                    The figure for incumbent <mark class="search-item__highlight">LCH</mark> is a hefty €100tn. Normally the clearer, a unit of the London Stock Exchange, could expect to...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2018-06-12T22:28:08+0000">June 12, 2018</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="0fcfff86-6e5b-11e8-92d3-6c13e5c92914"
                                        data-myft-ui="saved"
                                        action="/myft/save/0fcfff86-6e5b-11e8-92d3-6c13e5c92914"
                                        data-js-action="/__myft/api/core/saved/content/0fcfff86-6e5b-11e8-92d3-6c13e5c92914?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="0fcfff86-6e5b-11e8-92d3-6c13e5c92914" 
                                                aria-label="Save Brexit/derivatives: compression injury to myFT for later"
                                                title="Save Brexit/derivatives: compression injury to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/0fcfff86-6e5b-11e8-92d3-6c13e5c92914" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2Fcf218880-6e76-11e8-92d3-6c13e5c92914?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-3">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="dfa2bd4e-7c3e-11e7-9108-edda0bcbc928"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/ft-trading-room/clearing-settlement" aria-label="Category: Clearing and settlement">Clearing and settlement</a></div><div class="o-teaser__heading"><a href="/content/dfa2bd4e-7c3e-11e7-9108-edda0bcbc928" data-trackable="heading-link" class="js-teaser-heading-link">Euronext strikes derivatives clearing deal with LSE</a></div><p class="o-teaser__standfirst"><a href="/content/dfa2bd4e-7c3e-11e7-9108-edda0bcbc928" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...Euronext has ended a long-running dispute with rival London Stock Exchange Group over derivatives clearing, finalising a deal that keeps the exchanges operator’s business at <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>, the LSE’s French...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2017-08-08T16:06:47+0000">August 8, 2017</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="dfa2bd4e-7c3e-11e7-9108-edda0bcbc928"
                                        data-myft-ui="saved"
                                        action="/myft/save/dfa2bd4e-7c3e-11e7-9108-edda0bcbc928"
                                        data-js-action="/__myft/api/core/saved/content/dfa2bd4e-7c3e-11e7-9108-edda0bcbc928?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="dfa2bd4e-7c3e-11e7-9108-edda0bcbc928" 
                                                aria-label="Save Euronext strikes derivatives clearing deal with LSE to myFT for later"
                                                title="Save Euronext strikes derivatives clearing deal with LSE to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/dfa2bd4e-7c3e-11e7-9108-edda0bcbc928" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2Fd8ef8628-6021-11e7-91a7-502f7ee26895?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-4">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small js-teaser" data-id="b526c6a9-96dc-30d5-9e83-1e35c355c07e"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/companies" aria-label="Category: Companies">Companies</a></div><div class="o-teaser__heading"><a href="/content/b526c6a9-96dc-30d5-9e83-1e35c355c07e" data-trackable="heading-link" class="js-teaser-heading-link"><span>Euronext agrees new derivatives clearing deal with <mark class="search-item__highlight">LCH</mark></span></a></div><p class="o-teaser__standfirst"><a href="/content/b526c6a9-96dc-30d5-9e83-1e35c355c07e" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...commodities business from the unit to rival Intercontinental Exchange. Euronext supplies around half its business and has long been keen on the asset.

                                    “<mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark> is a core strategic asset and neither <mark class="search-item__highlight">LCH</mark>...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2017-08-08T07:17:59+0000">August 8, 2017</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="b526c6a9-96dc-30d5-9e83-1e35c355c07e"
                                        data-myft-ui="saved"
                                        action="/myft/save/b526c6a9-96dc-30d5-9e83-1e35c355c07e"
                                        data-js-action="/__myft/api/core/saved/content/b526c6a9-96dc-30d5-9e83-1e35c355c07e?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="b526c6a9-96dc-30d5-9e83-1e35c355c07e" 
                                                aria-label="Save Euronext agrees new derivatives clearing deal with LCH to myFT for later"
                                                title="Save Euronext agrees new derivatives clearing deal with LCH to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-5">
                                    <div class="search-item">
                                            <div class="search-item__advert">
                                                <div
                                                    class="o-ads o-ads--center o-ads--reserve-250"
                                                    data-o-ads-name="search-results-top"
                                                    data-o-ads-targeting="pos=top;"
                                                    data-o-ads-formats-small="MediumRectangle"
                                                    data-o-ads-formats-medium="false"
                                                    data-o-ads-formats-large="false"
                                                    data-o-ads-formats-extra="false"
                                                    aria-hidden="true">
                                                </div>
                                            </div>
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small js-teaser" data-id="469dc936-a805-3f19-95ed-db5d2fa12def"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/markets" aria-label="Category: Markets">Markets</a></div><div class="o-teaser__heading"><a href="/content/469dc936-a805-3f19-95ed-db5d2fa12def" data-trackable="heading-link" class="js-teaser-heading-link">Euronext and Intercontinental Exchange sign clearing services pact</a></div><p class="o-teaser__standfirst"><a href="/content/469dc936-a805-3f19-95ed-db5d2fa12def" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...relationship with LCH.Euronext accounts for nearly half of <mark class="search-item__highlight">LCH</mark> France’s business and has a contract with the clearing house that expires at the end of 2018.

                                    It had hoped to pick up <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>, which was...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2017-04-03T06:05:46+0000">April 3, 2017</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="469dc936-a805-3f19-95ed-db5d2fa12def"
                                        data-myft-ui="saved"
                                        action="/myft/save/469dc936-a805-3f19-95ed-db5d2fa12def"
                                        data-js-action="/__myft/api/core/saved/content/469dc936-a805-3f19-95ed-db5d2fa12def?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="469dc936-a805-3f19-95ed-db5d2fa12def" 
                                                aria-label="Save Euronext and Intercontinental Exchange sign clearing services pact to myFT for later"
                                                title="Save Euronext and Intercontinental Exchange sign clearing services pact to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-6">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image o-teaser--opinion js-teaser" data-id="865feaf0-1491-11e7-b0c1-37e417ee6c76"><div class="o-teaser__content"><div class="o-teaser__meta"><span class="o-teaser__tag-prefix">FT Trading Room</span><a class="o-teaser__tag" data-trackable="teaser-tag" href="/matthew-vincent" aria-label="Category: Matthew Vincent">Matthew Vincent</a></div><div class="o-teaser__heading"><a href="/content/865feaf0-1491-11e7-b0c1-37e417ee6c76" data-trackable="heading-link" class="js-teaser-heading-link">LSE free to move in its own clear direction</a></div><p class="o-teaser__standfirst"><a href="/content/865feaf0-1491-11e7-b0c1-37e417ee6c76" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...house, <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>, and MTS was less than one-tenth of the size, Mr Rolet’s refusal irked Deutsche Börse and perplexed shareholders. It looked like a convenient excuse for walking away from the deal, rather...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2017-03-29T18:37:24+0000">March 29, 2017</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="865feaf0-1491-11e7-b0c1-37e417ee6c76"
                                        data-myft-ui="saved"
                                        action="/myft/save/865feaf0-1491-11e7-b0c1-37e417ee6c76"
                                        data-js-action="/__myft/api/core/saved/content/865feaf0-1491-11e7-b0c1-37e417ee6c76?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="865feaf0-1491-11e7-b0c1-37e417ee6c76" 
                                                aria-label="Save LSE free to move in its own clear direction to myFT for later"
                                                title="Save LSE free to move in its own clear direction to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/865feaf0-1491-11e7-b0c1-37e417ee6c76" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2F4eb18a3a-ffe9-11e6-8d8e-a5e3738f9ae4?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-7">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="2ae1c513-40cf-32af-8eab-375e47b57f4f"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/stock_exchange_consolidation" aria-label="Category: Stock exchange consolidation">Stock exchange consolidation</a></div><div class="o-teaser__heading"><a href="/content/2ae1c513-40cf-32af-8eab-375e47b57f4f" data-trackable="heading-link" class="js-teaser-heading-link"><span>LSE enters into sale talks with Euronext over <mark class="search-item__highlight">LCH</mark></span></a></div><p class="o-teaser__standfirst"><a href="/content/2ae1c513-40cf-32af-8eab-375e47b57f4f" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...£20bn merger with Deutsche Borse.

                                    The LSE has offered to sell <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>, formerly known as Clearnet, to allay concerns from Brussels’ antitrust officials that the German deal could curb competition in repo...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2016-12-20T07:22:37+0000">December 20, 2016</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="2ae1c513-40cf-32af-8eab-375e47b57f4f"
                                        data-myft-ui="saved"
                                        action="/myft/save/2ae1c513-40cf-32af-8eab-375e47b57f4f"
                                        data-js-action="/__myft/api/core/saved/content/2ae1c513-40cf-32af-8eab-375e47b57f4f?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="2ae1c513-40cf-32af-8eab-375e47b57f4f" 
                                                aria-label="Save LSE enters into sale talks with Euronext over LCH to myFT for later"
                                                title="Save LSE enters into sale talks with Euronext over LCH to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/2ae1c513-40cf-32af-8eab-375e47b57f4f" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fwww.ft.com%2Ffastft%2Ffiles%2F2016%2F12%2FMAS_FTfm_LSE_1C54EEE7DE594D52-A4DB86FB1D2EE6D4_20161104181322.jpg?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-8">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="2e47acc6-c68b-11e6-8f29-9445cac8966f"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/ft-trading-room/clearing-settlement" aria-label="Category: Clearing and settlement">Clearing and settlement</a></div><div class="o-teaser__heading"><a href="/content/2e47acc6-c68b-11e6-8f29-9445cac8966f" data-trackable="heading-link" class="js-teaser-heading-link">Euronext in talks to buy LSE clearing business</a></div><p class="o-teaser__standfirst"><a href="/content/2e47acc6-c68b-11e6-8f29-9445cac8966f" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...with Deutsche Börse.

                                    LSE offered to sell <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>, the Paris-based subsidiary formerly known as Clearnet, three months ago to allay concerns from Brussels’ antitrust officials that its German merger...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2016-12-20T08:50:39+0000">December 20, 2016</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="2e47acc6-c68b-11e6-8f29-9445cac8966f"
                                        data-myft-ui="saved"
                                        action="/myft/save/2e47acc6-c68b-11e6-8f29-9445cac8966f"
                                        data-js-action="/__myft/api/core/saved/content/2e47acc6-c68b-11e6-8f29-9445cac8966f?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="2e47acc6-c68b-11e6-8f29-9445cac8966f" 
                                                aria-label="Save Euronext in talks to buy LSE clearing business to myFT for later"
                                                title="Save Euronext in talks to buy LSE clearing business to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/2e47acc6-c68b-11e6-8f29-9445cac8966f" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2F7e2ea132-c68f-11e6-9043-7e34c07b46ef?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-9">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small js-teaser" data-id="4038057b-8472-3119-ab97-844501d2ba9f"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/stock_exchange_consolidation" aria-label="Category: Stock exchange consolidation">Stock exchange consolidation</a></div><div class="o-teaser__heading"><a href="/content/4038057b-8472-3119-ab97-844501d2ba9f" data-trackable="heading-link" class="js-teaser-heading-link">LSE, Deutsche Börse offer sale of French clearing arm as antitrust remedy</a></div><p class="o-teaser__standfirst"><a href="/content/4038057b-8472-3119-ab97-844501d2ba9f" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...believe that the vast majority of the concerns that are coming out of Brussels will, in fact, be addressed by [the sale of <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>],” he said.

                                    <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark> clears derivatives, equities and repo trades, although...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2017-02-06T23:49:32+0000">February 6, 2017</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="4038057b-8472-3119-ab97-844501d2ba9f"
                                        data-myft-ui="saved"
                                        action="/myft/save/4038057b-8472-3119-ab97-844501d2ba9f"
                                        data-js-action="/__myft/api/core/saved/content/4038057b-8472-3119-ab97-844501d2ba9f?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="4038057b-8472-3119-ab97-844501d2ba9f" 
                                                aria-label="Save LSE, Deutsche Börse offer sale of French clearing arm as antitrust remedy to myFT for later"
                                                title="Save LSE, Deutsche Börse offer sale of French clearing arm as antitrust remedy to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-10">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small js-teaser" data-id="e68e7e0c-e77d-3239-80c2-908b2752753a"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/mergers-acquisitions" aria-label="Category: Mergers & Acquisitions">Mergers & Acquisitions</a></div><div class="o-teaser__heading"><a href="/content/e68e7e0c-e77d-3239-80c2-908b2752753a" data-trackable="heading-link" class="js-teaser-heading-link">Euronext bids €510m for LSE’s French clearing arm</a></div><p class="o-teaser__standfirst"><a href="/content/e68e7e0c-e77d-3239-80c2-908b2752753a" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...been the subject of concern from EU’s antitrust officials over competition in repo and equities clearing.

                                    Euronext’s acquisition of <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>, formerly known as Clearnet, will also be the subject of a...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2017-01-03T07:15:24+0000">January 3, 2017</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="e68e7e0c-e77d-3239-80c2-908b2752753a"
                                        data-myft-ui="saved"
                                        action="/myft/save/e68e7e0c-e77d-3239-80c2-908b2752753a"
                                        data-js-action="/__myft/api/core/saved/content/e68e7e0c-e77d-3239-80c2-908b2752753a?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="e68e7e0c-e77d-3239-80c2-908b2752753a" 
                                                aria-label="Save Euronext bids €510m for LSE’s French clearing arm to myFT for later"
                                                title="Save Euronext bids €510m for LSE’s French clearing arm to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-11">
                                    <div class="search-item">
                                            <div class="search-item__advert">
                                                <div
                                                    class="o-ads o-ads--center o-ads--reserve-250"
                                                    data-o-ads-name="search-results-mid"
                                                    data-o-ads-targeting="pos=mid;"
                                                    data-o-ads-formats-small="MediumRectangle"
                                                    data-o-ads-formats-medium="false"
                                                    data-o-ads-formats-large="false"
                                                    data-o-ads-formats-extra="false"
                                                    aria-hidden="true">
                                                </div>
                                            </div>
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small js-teaser" data-id="29f802b9-f8aa-38da-8bc7-166bd1033f2d"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/mergers-acquisitions" aria-label="Category: Mergers & Acquisitions">Mergers & Acquisitions</a></div><div class="o-teaser__heading"><a href="/content/29f802b9-f8aa-38da-8bc7-166bd1033f2d" data-trackable="heading-link" class="js-teaser-heading-link">Deutsche Börse CEO talked to Merkel adviser about LSE tie up before share purchase, Spiegel reports</a></div><p class="o-teaser__standfirst"><a href="/content/29f802b9-f8aa-38da-8bc7-166bd1033f2d" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...concerns, according to two people who have viewed the document.

                                    The two have offered to sell the LSE’s French clearing arm <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark> and the LSE already has a buyer, having agreed a €510m deal with Euronext...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2017-02-10T17:50:41+0000">February 10, 2017</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="29f802b9-f8aa-38da-8bc7-166bd1033f2d"
                                        data-myft-ui="saved"
                                        action="/myft/save/29f802b9-f8aa-38da-8bc7-166bd1033f2d"
                                        data-js-action="/__myft/api/core/saved/content/29f802b9-f8aa-38da-8bc7-166bd1033f2d?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="29f802b9-f8aa-38da-8bc7-166bd1033f2d" 
                                                aria-label="Save Deutsche Börse CEO talked to Merkel adviser about LSE tie up before share purchase, Spiegel reports to myFT for later"
                                                title="Save Deutsche Börse CEO talked to Merkel adviser about LSE tie up before share purchase, Spiegel reports to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-12">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="158e2164-c849-11e6-9043-7e34c07b46ef"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/ft-trading-room/clearing-settlement" aria-label="Category: Clearing and settlement">Clearing and settlement</a></div><div class="o-teaser__heading"><a href="/content/158e2164-c849-11e6-9043-7e34c07b46ef" data-trackable="heading-link" class="js-teaser-heading-link">LSE set to sell French clearing arm to Euronext</a></div><p class="o-teaser__standfirst"><a href="/content/158e2164-c849-11e6-9043-7e34c07b46ef" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...now been resolved and a formal statement is expected to come this week, two people briefed on the talks said.

                                    The LSE and Euronext declined to comment.

                                    LSE offered to sell <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>, the Paris-based...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2016-12-27T19:31:43+0000">December 27, 2016</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="158e2164-c849-11e6-9043-7e34c07b46ef"
                                        data-myft-ui="saved"
                                        action="/myft/save/158e2164-c849-11e6-9043-7e34c07b46ef"
                                        data-js-action="/__myft/api/core/saved/content/158e2164-c849-11e6-9043-7e34c07b46ef?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="158e2164-c849-11e6-9043-7e34c07b46ef" 
                                                aria-label="Save LSE set to sell French clearing arm to Euronext to myFT for later"
                                                title="Save LSE set to sell French clearing arm to Euronext to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/158e2164-c849-11e6-9043-7e34c07b46ef" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2Fbfc3279c-cc63-11e6-864f-20dcb35cede2?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-13">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="1f9990ae-ec79-11e6-ba01-119a44939bb6"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/stock_exchange_consolidation" aria-label="Category: Stock exchange consolidation">Stock exchange consolidation</a></div><div class="o-teaser__heading"><a href="/content/1f9990ae-ec79-11e6-ba01-119a44939bb6" data-trackable="heading-link" class="js-teaser-heading-link">D Börse and LSE make narrow offer to Brussels</a></div><p class="o-teaser__standfirst"><a href="/content/1f9990ae-ec79-11e6-ba01-119a44939bb6" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...<mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>, a process that could pave the way to regulatory approval for the deal by April.

                                    Clearance from Brussels is just one of many hurdles the exchanges need to surmount in their attempt to create...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2017-02-07T18:01:38+0000">February 7, 2017</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="1f9990ae-ec79-11e6-ba01-119a44939bb6"
                                        data-myft-ui="saved"
                                        action="/myft/save/1f9990ae-ec79-11e6-ba01-119a44939bb6"
                                        data-js-action="/__myft/api/core/saved/content/1f9990ae-ec79-11e6-ba01-119a44939bb6?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="1f9990ae-ec79-11e6-ba01-119a44939bb6" 
                                                aria-label="Save D Börse and LSE make narrow offer to Brussels to myFT for later"
                                                title="Save D Börse and LSE make narrow offer to Brussels to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/1f9990ae-ec79-11e6-ba01-119a44939bb6" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2F1708a7c2-c6d6-11e6-9043-7e34c07b46ef?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-14">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="56e01ad0-fc6c-11e6-96f8-3700c5664d30"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/stock_exchange_consolidation" aria-label="Category: Stock exchange consolidation">Stock exchange consolidation</a></div><div class="o-teaser__heading"><a href="/content/56e01ad0-fc6c-11e6-96f8-3700c5664d30" data-trackable="heading-link" class="js-teaser-heading-link">LSE and Deutsche Börse’s €29bn deal in jeopardy</a></div><p class="o-teaser__standfirst"><a href="/content/56e01ad0-fc6c-11e6-96f8-3700c5664d30" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>..., France and the US. Average daily volumes usually exceed €100bn.

                                    In 2015, <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark> cleared €72.6tn across 15 European government markets. Most of the LSE’s fixed-income business is traded through MTS...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2017-02-27T07:50:38+0000">February 27, 2017</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="56e01ad0-fc6c-11e6-96f8-3700c5664d30"
                                        data-myft-ui="saved"
                                        action="/myft/save/56e01ad0-fc6c-11e6-96f8-3700c5664d30"
                                        data-js-action="/__myft/api/core/saved/content/56e01ad0-fc6c-11e6-96f8-3700c5664d30?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="56e01ad0-fc6c-11e6-96f8-3700c5664d30" 
                                                aria-label="Save LSE and Deutsche Börse’s €29bn deal in jeopardy to myFT for later"
                                                title="Save LSE and Deutsche Börse’s €29bn deal in jeopardy to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/56e01ad0-fc6c-11e6-96f8-3700c5664d30" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2F1708a7c2-c6d6-11e6-9043-7e34c07b46ef?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-15">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="8040102a-d196-11e6-9341-7393bb2e1b51"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/companies/financial-services" aria-label="Category: Financial services">Financial services</a></div><div class="o-teaser__heading"><a href="/content/8040102a-d196-11e6-9341-7393bb2e1b51" data-trackable="heading-link" class="js-teaser-heading-link">LSE sells French clearing unit to Euronext for €510m</a></div><p class="o-teaser__standfirst"><a href="/content/8040102a-d196-11e6-9341-7393bb2e1b51" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...Börse.

                                    The UK group offered to sell <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>, the Paris-based subsidiary formerly known as Clearnet, three months ago to address antitrust concerns raised by the European financial watchdog over its €24bn...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2017-01-03T10:25:29+0000">January 3, 2017</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="8040102a-d196-11e6-9341-7393bb2e1b51"
                                        data-myft-ui="saved"
                                        action="/myft/save/8040102a-d196-11e6-9341-7393bb2e1b51"
                                        data-js-action="/__myft/api/core/saved/content/8040102a-d196-11e6-9341-7393bb2e1b51?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="8040102a-d196-11e6-9341-7393bb2e1b51" 
                                                aria-label="Save LSE sells French clearing unit to Euronext for €510m to myFT for later"
                                                title="Save LSE sells French clearing unit to Euronext for €510m to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/8040102a-d196-11e6-9341-7393bb2e1b51" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2F1708a7c2-c6d6-11e6-9043-7e34c07b46ef?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-16">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="539550b4-e936-11e6-893c-082c54a7f539"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/companies/financial-services" aria-label="Category: Financial services">Financial services</a></div><div class="o-teaser__heading"><a href="/content/539550b4-e936-11e6-893c-082c54a7f539" data-trackable="heading-link" class="js-teaser-heading-link">Probe of Deutsche Börse chief focuses on LSE talks</a></div><p class="o-teaser__standfirst"><a href="/content/539550b4-e936-11e6-893c-082c54a7f539" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...European Commission. Any package is likely to include the sale of <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>, the Paris-based subsidiary formerly known as Clearnet, to allay EU concerns about dominance in European fixed-income markets...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2017-02-02T14:24:21+0000">February 2, 2017</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="539550b4-e936-11e6-893c-082c54a7f539"
                                        data-myft-ui="saved"
                                        action="/myft/save/539550b4-e936-11e6-893c-082c54a7f539"
                                        data-js-action="/__myft/api/core/saved/content/539550b4-e936-11e6-893c-082c54a7f539?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="539550b4-e936-11e6-893c-082c54a7f539" 
                                                aria-label="Save Probe of Deutsche Börse chief focuses on LSE talks to myFT for later"
                                                title="Save Probe of Deutsche Börse chief focuses on LSE talks to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/539550b4-e936-11e6-893c-082c54a7f539" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2F82a63c18-e951-11e6-893c-082c54a7f539?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-17">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="cade96cc-e89a-11e6-967b-c88452263daf"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/companies/financial-services" aria-label="Category: Financial services">Financial services</a></div><div class="o-teaser__heading"><a href="/content/cade96cc-e89a-11e6-967b-c88452263daf" data-trackable="heading-link" class="js-teaser-heading-link">Deutsche Börse chief probed over share purchase</a></div><p class="o-teaser__standfirst"><a href="/content/cade96cc-e89a-11e6-967b-c88452263daf" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...Europe and are due to make a decision by March 13. That deadline will probably be extended if, as expected, the two sides offer potential remedies to the European Commission.

                                    LSE has offered to sell <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2017-02-01T20:17:25+0000">February 1, 2017</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="cade96cc-e89a-11e6-967b-c88452263daf"
                                        data-myft-ui="saved"
                                        action="/myft/save/cade96cc-e89a-11e6-967b-c88452263daf"
                                        data-js-action="/__myft/api/core/saved/content/cade96cc-e89a-11e6-967b-c88452263daf?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="cade96cc-e89a-11e6-967b-c88452263daf" 
                                                aria-label="Save Deutsche Börse chief probed over share purchase to myFT for later"
                                                title="Save Deutsche Börse chief probed over share purchase to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/cade96cc-e89a-11e6-967b-c88452263daf" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2Fca1fb050-c774-11e6-9043-7e34c07b46ef?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-18">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small js-teaser" data-id="4824f423-c6c0-35e4-b975-c4a4ecf9ff39"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/mergers-acquisitions" aria-label="Category: Mergers & Acquisitions">Mergers & Acquisitions</a></div><div class="o-teaser__heading"><a href="/content/4824f423-c6c0-35e4-b975-c4a4ecf9ff39" data-trackable="heading-link" class="js-teaser-heading-link">Deutsche Börse CEO faces probe over stock purchase ahead of merger talks</a></div><p class="o-teaser__standfirst"><a href="/content/4824f423-c6c0-35e4-b975-c4a4ecf9ff39" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>..., the two sides offer potential remedies to the European Commission.

                                    LSE has offered to sell <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>, the Paris-based subsidiary formerly known as Clearnet, to allay concerns from Brussels antitrust...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2017-02-01T18:46:43+0000">February 1, 2017</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="4824f423-c6c0-35e4-b975-c4a4ecf9ff39"
                                        data-myft-ui="saved"
                                        action="/myft/save/4824f423-c6c0-35e4-b975-c4a4ecf9ff39"
                                        data-js-action="/__myft/api/core/saved/content/4824f423-c6c0-35e4-b975-c4a4ecf9ff39?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="4824f423-c6c0-35e4-b975-c4a4ecf9ff39" 
                                                aria-label="Save Deutsche Börse CEO faces probe over stock purchase ahead of merger talks to myFT for later"
                                                title="Save Deutsche Börse CEO faces probe over stock purchase ahead of merger talks to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-19">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="9c0019a8-bfea-11e6-9bca-2b93a6856354"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/stock_exchange_consolidation" aria-label="Category: Stock exchange consolidation">Stock exchange consolidation</a></div><div class="o-teaser__heading"><a href="/content/9c0019a8-bfea-11e6-9bca-2b93a6856354" data-trackable="heading-link" class="js-teaser-heading-link">Brussels to focus LSE-Deutsche Börse probe on clearing</a></div><p class="o-teaser__standfirst"><a href="/content/9c0019a8-bfea-11e6-9bca-2b93a6856354" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...in repo clearing. Euronext, the Paris-based exchanges operator that uses <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>, remains in contention to buy it, according to three people familiar with the talks. CME Group and Nasdaq have also looked...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2016-12-12T11:21:49+0000">December 12, 2016</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="9c0019a8-bfea-11e6-9bca-2b93a6856354"
                                        data-myft-ui="saved"
                                        action="/myft/save/9c0019a8-bfea-11e6-9bca-2b93a6856354"
                                        data-js-action="/__myft/api/core/saved/content/9c0019a8-bfea-11e6-9bca-2b93a6856354?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="9c0019a8-bfea-11e6-9bca-2b93a6856354" 
                                                aria-label="Save Brussels to focus LSE-Deutsche Börse probe on clearing to myFT for later"
                                                title="Save Brussels to focus LSE-Deutsche Börse probe on clearing to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/9c0019a8-bfea-11e6-9bca-2b93a6856354" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2F4a14f1be-c061-11e6-81c2-f57d90f6741a?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-20">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small js-teaser" data-id="d557d1ee-401b-11e7-82b6-896b95f30f58"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/due-diligence" aria-label="Category: Due Diligence">Due Diligence</a></div><div class="o-teaser__heading"><a href="/content/d557d1ee-401b-11e7-82b6-896b95f30f58" data-trackable="heading-link" class="js-teaser-heading-link">Glencore&#x27;s Glasenberg back on front foot with Bunge approach</a><span> <span class="o-labels o-labels--premium o-labels--content-premium" aria-label="Premium content">Premium</span></span></div><p class="o-teaser__standfirst"><a href="/content/d557d1ee-401b-11e7-82b6-896b95f30f58" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...that mean Euronext has given up on <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>? No, but there needs to be a willing seller and there isn’t one yet, Stéphane Boujnah, Euronext chief executive, told the FT. Even so, he had words worth heeding...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2017-05-24T04:03:08+0000">May 24, 2017</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="d557d1ee-401b-11e7-82b6-896b95f30f58"
                                        data-myft-ui="saved"
                                        action="/myft/save/d557d1ee-401b-11e7-82b6-896b95f30f58"
                                        data-js-action="/__myft/api/core/saved/content/d557d1ee-401b-11e7-82b6-896b95f30f58?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="d557d1ee-401b-11e7-82b6-896b95f30f58" 
                                                aria-label="Save Glencore&#x27;s Glasenberg back on front foot with Bunge approach to myFT for later"
                                                title="Save Glencore&#x27;s Glasenberg back on front foot with Bunge approach to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-21">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="06c0a330-c6b6-11e6-8f29-9445cac8966f"><div class="o-teaser__content"><div class="o-teaser__meta"><span class="o-teaser__tag-prefix">Interview</span><a class="o-teaser__tag" data-trackable="teaser-tag" href="/ft-trading-room" aria-label="Category: FT Trading Room">FT Trading Room</a></div><div class="o-teaser__heading"><a href="/content/06c0a330-c6b6-11e6-8f29-9445cac8966f" data-trackable="heading-link" class="js-teaser-heading-link">Kengeter resists calls on Deutsche Börse-LSE HQ</a></div><p class="o-teaser__standfirst"><a href="/content/06c0a330-c6b6-11e6-8f29-9445cac8966f" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...competition in trading, pre-trade, clearing, other activities actually works,” says Mr Kengeter.

                                    The LSE is in talks with rival Euronext to sell its French clearing arm, <mark class="search-item__highlight">LCH</mark> <mark class="search-item__highlight">SA</mark>, to offset concerns about...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2016-12-21T16:00:21+0000">December 21, 2016</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="06c0a330-c6b6-11e6-8f29-9445cac8966f"
                                        data-myft-ui="saved"
                                        action="/myft/save/06c0a330-c6b6-11e6-8f29-9445cac8966f"
                                        data-js-action="/__myft/api/core/saved/content/06c0a330-c6b6-11e6-8f29-9445cac8966f?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="06c0a330-c6b6-11e6-8f29-9445cac8966f" 
                                                aria-label="Save Kengeter resists calls on Deutsche Börse-LSE HQ to myFT for later"
                                                title="Save Kengeter resists calls on Deutsche Börse-LSE HQ to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/06c0a330-c6b6-11e6-8f29-9445cac8966f" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2Fca1fb050-c774-11e6-9043-7e34c07b46ef?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-22">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small js-teaser" data-id="6d258552-a962-31f9-b3e8-792feade23e7"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/stock_exchange_consolidation" aria-label="Category: Stock exchange consolidation">Stock exchange consolidation</a></div><div class="o-teaser__heading"><a href="/content/6d258552-a962-31f9-b3e8-792feade23e7" data-trackable="heading-link" class="js-teaser-heading-link">Europe’s clearing houses pass stress tests</a></div><p class="o-teaser__standfirst"><a href="/content/6d258552-a962-31f9-b3e8-792feade23e7" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...more than €150bn worth of default resources, and collectively have more than 900 clearing house members. The majority of those resources were held at just five main houses, operated by <mark class="search-item__highlight">LCH</mark> Group...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2016-04-29T16:09:59+0000">April 29, 2016</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="6d258552-a962-31f9-b3e8-792feade23e7"
                                        data-myft-ui="saved"
                                        action="/myft/save/6d258552-a962-31f9-b3e8-792feade23e7"
                                        data-js-action="/__myft/api/core/saved/content/6d258552-a962-31f9-b3e8-792feade23e7?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="6d258552-a962-31f9-b3e8-792feade23e7" 
                                                aria-label="Save Europe’s clearing houses pass stress tests to myFT for later"
                                                title="Save Europe’s clearing houses pass stress tests to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-23">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small js-teaser" data-id="d517108e-cf22-3d95-8537-0dc6c31d7744"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/central-banks" aria-label="Category: Central banks">Central banks</a></div><div class="o-teaser__heading"><a href="/content/d517108e-cf22-3d95-8537-0dc6c31d7744" data-trackable="heading-link" class="js-teaser-heading-link">LSE eyes spinoff of French clearing house</a></div><p class="o-teaser__standfirst"><a href="/content/d517108e-cf22-3d95-8537-0dc6c31d7744" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>...cent of <mark class="search-item__highlight">LCH</mark>, with the balance owned by a consortium of banks. Putting Clearnet <mark class="search-item__highlight">SA</mark> under the control of a Eurozone-based business would be an attempt to allay concerns about its regulation if the UK...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2016-03-02T11:50:09+0000">March 2, 2016</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="d517108e-cf22-3d95-8537-0dc6c31d7744"
                                        data-myft-ui="saved"
                                        action="/myft/save/d517108e-cf22-3d95-8537-0dc6c31d7744"
                                        data-js-action="/__myft/api/core/saved/content/d517108e-cf22-3d95-8537-0dc6c31d7744?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="d517108e-cf22-3d95-8537-0dc6c31d7744" 
                                                aria-label="Save LSE eyes spinoff of French clearing house to myFT for later"
                                                title="Save LSE eyes spinoff of French clearing house to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div></div>
                                            </div>
                                    </div>
                                </li>
                                <li class="search-results__list-item" data-trackable="item-24">
                                    <div class="search-item">
                                            <div class="search-item__teaser">
                                                <div class="o-teaser o-teaser--article o-teaser--small o-teaser--has-image js-teaser" data-id="31fe3dd6-e094-11e5-8d9b-e88a2a889797"><div class="o-teaser__content"><div class="o-teaser__meta"><a class="o-teaser__tag" data-trackable="teaser-tag" href="/ft-trading-room/clearing-settlement" aria-label="Category: Clearing and settlement">Clearing and settlement</a></div><div class="o-teaser__heading"><a href="/content/31fe3dd6-e094-11e5-8d9b-e88a2a889797" data-trackable="heading-link" class="js-teaser-heading-link">ICE move could include Borsa spin-off</a></div><p class="o-teaser__standfirst"><a href="/content/31fe3dd6-e094-11e5-8d9b-e88a2a889797" data-trackable="standfirst-link" tabindex="-1" class="js-teaser-standfirst-link"><span>..., which the LSE bought for €1.6bn in 2007. The latter comprises equities, derivatives trading and post-trade businesses.

                                    The LSE is already considering spinning off Paris-based Clearnet <mark class="search-item__highlight">SA</mark>, said those...</span></a></p><div class="o-teaser__timestamp"><time class="o-teaser__timestamp-date" datetime="2016-03-03T10:34:47+0000">March 3, 2016</time></div><div class="o-teaser__action"><span><form class="n-myft-ui n-myft-ui--save" method="GET"
                                        data-content-id="31fe3dd6-e094-11e5-8d9b-e88a2a889797"
                                        data-myft-ui="saved"
                                        action="/myft/save/31fe3dd6-e094-11e5-8d9b-e88a2a889797"
                                        data-js-action="/__myft/api/core/saved/content/31fe3dd6-e094-11e5-8d9b-e88a2a889797?method=put">
                                        <input
                                            data-myft-csrf-token
                                            value=""
                                            type="hidden"
                                            name="token">
                                        <div
                                            class="n-myft-ui__announcement o-normalise-visually-hidden"
                                            aria-live="assertive"
                                            data-pressed-text="Article saved in My FT."
                                            data-unpressed-text="Removed article from My FT."
                                        ></div>
                                        <button
                                            type="submit"
                                            class="n-myft-ui__button"
                                            data-trackable="save-for-later"
                                                data-alternate-label="Saved to myFT"
                                                aria-pressed="false"
                                                    data-text-variant="save-button-longer-copy"
                                                    data-alternate-text="Saved&nbsp;"
                                            data-content-id="31fe3dd6-e094-11e5-8d9b-e88a2a889797" 
                                                aria-label="Save ICE move could include Borsa spin-off to myFT for later"
                                                title="Save ICE move could include Borsa spin-off to myFT for later"
                                            >

                                                            <span class="save-button-longer-copy" data-variant-label>Save&nbsp;</span><span class="n-myft-ui__button--viewport-large" aria-hidden="true">to myFT</span>
                                            </button>
                                        </form></span></div></div><div class="o-teaser__image-container js-teaser-image-container"><div class="o-teaser__image-placeholder" style="padding-bottom:56.2500%;"><a href="/content/31fe3dd6-e094-11e5-8d9b-e88a2a889797" data-trackable="image-link" tabindex="-1" aria-hidden="true"><img class="o-teaser__image o-lazy-load" data-src="https://www.ft.com/__origami/service/image/v2/images/raw/http%3A%2F%2Fprod-upp-image-read.ft.com%2F1b85b7f0-da81-11e5-a72f-1e7744c66818?source=next&fit=scale-down&compression=best&width=240" alt=""/></a></div></div></div>
                                            </div>
                                    </div>
                                </li>
                        </ul>
                        <div class="search-results__pagination">
                            <div class="search-pagination o-buttons-pagination" data-trackable="search-pagination">
                                <a class="search-pagination__prev-page o-buttons o-buttons--secondary o-buttons-icon o-buttons-icon--arrow-left o-buttons--big o-buttons-icon--icon-only" disabled rel="nofollow" data-trackable="prev-page">
                                    <span class="o-buttons-icon__label">Previous page</span>
                                </a>
                                <span class="search-pagination__page">Page <strong>1</strong> of 3</span>
                                <a class="search-pagination__next-page o-buttons o-buttons--secondary o-buttons-icon o-buttons-icon--arrow-right o-buttons--big o-buttons-icon--icon-only" href="?q&#x3D;lch%20sa&page&#x3D;2&sort&#x3D;relevance" rel="nofollow" data-trackable="next-page">
                                    <span class="o-buttons-icon__label">Next page</span>
                                </a>
                            </div>
                        </div>
                </div>

            </main>

            <nav class="search-layout__sidebar" id="site-sidebar">
                            <div
                                class="concept-list"
                                data-trackable="concept-list">
                                    <h2 class="concept-list__title">
                                            Topics relevant to this search
                                    </h2>
                                <ul class="concept-list__list">
                                        <li class="concept-list__list-item">
                                            <a
                                                href="https://www.ft.com/stock_exchange_consolidation"
                                                data-trackable="concept"
                                                class="concept-list__concept">
                                                Stock exchange consolidation
                                            </a>
                                                <form
                                                    class="n-myft-ui n-myft-ui--follow "
                                                    method="GET"
                                                    data-myft-ui="follow"
                                                    data-concept-id="4ffcf359-5e68-49b0-a266-af9c79e9b632"

                                                            action="/myft/add/4ffcf359-5e68-49b0-a266-af9c79e9b632"
                                                            data-js-action="/__myft/api/core/followed/concept/4ffcf359-5e68-49b0-a266-af9c79e9b632?method=put"
                                                    >
                                                    <input
                                                        data-myft-csrf-token
                                                        value=""
                                                        type="hidden"
                                                        name="token">
                                                    <div
                                                        class="n-myft-ui__announcement o-normalise-visually-hidden"
                                                        aria-live="assertive"
                                                        data-pressed-text="Now following Stock exchange consolidation."
                                                        data-unpressed-text="No longer following Stock exchange consolidation."
                                                    ></div>
                                                    <button
                                                            aria-pressed="false"
                                                            aria-label="Add Stock exchange consolidation to myFT"
                                                            title="Add Stock exchange consolidation to myFT"
                                                            data-alternate-label="Remove Stock exchange consolidation from myFT"
                                                                    data-alternate-text="Added"
                                                        class="
                                                            n-myft-follow-button"
                                                        data-concept-id="4ffcf359-5e68-49b0-a266-af9c79e9b632" 
                                                        data-trackable="follow"
                                                        type="submit">Add to myFT</button>
                                                </form>
                                        </li>
                                        <li class="concept-list__list-item">
                                            <a
                                                href="https://www.ft.com/markets"
                                                data-trackable="concept"
                                                class="concept-list__concept">
                                                Markets
                                            </a>
                                                <form
                                                    class="n-myft-ui n-myft-ui--follow "
                                                    method="GET"
                                                    data-myft-ui="follow"
                                                    data-concept-id="c91b1fad-1097-468b-be82-9a8ff717d54c"

                                                            action="/myft/add/c91b1fad-1097-468b-be82-9a8ff717d54c"
                                                            data-js-action="/__myft/api/core/followed/concept/c91b1fad-1097-468b-be82-9a8ff717d54c?method=put"
                                                    >
                                                    <input
                                                        data-myft-csrf-token
                                                        value=""
                                                        type="hidden"
                                                        name="token">
                                                    <div
                                                        class="n-myft-ui__announcement o-normalise-visually-hidden"
                                                        aria-live="assertive"
                                                        data-pressed-text="Now following Markets."
                                                        data-unpressed-text="No longer following Markets."
                                                    ></div>
                                                    <button
                                                            aria-pressed="false"
                                                            aria-label="Add Markets to myFT"
                                                            title="Add Markets to myFT"
                                                            data-alternate-label="Remove Markets from myFT"
                                                                    data-alternate-text="Added"
                                                        class="
                                                            n-myft-follow-button"
                                                        data-concept-id="c91b1fad-1097-468b-be82-9a8ff717d54c" 
                                                        data-trackable="follow"
                                                        type="submit">Add to myFT</button>
                                                </form>
                                        </li>
                                        <li class="concept-list__list-item">
                                            <a
                                                href="https://www.ft.com/stream/214c6ab8-1eea-48ad-99d2-ac55b0235e05"
                                                data-trackable="concept"
                                                class="concept-list__concept">
                                                London Stock Exchange Group
                                            </a>
                                                <form
                                                    class="n-myft-ui n-myft-ui--follow "
                                                    method="GET"
                                                    data-myft-ui="follow"
                                                    data-concept-id="214c6ab8-1eea-48ad-99d2-ac55b0235e05"

                                                            action="/myft/add/214c6ab8-1eea-48ad-99d2-ac55b0235e05"
                                                            data-js-action="/__myft/api/core/followed/concept/214c6ab8-1eea-48ad-99d2-ac55b0235e05?method=put"
                                                    >
                                                    <input
                                                        data-myft-csrf-token
                                                        value=""
                                                        type="hidden"
                                                        name="token">
                                                    <div
                                                        class="n-myft-ui__announcement o-normalise-visually-hidden"
                                                        aria-live="assertive"
                                                        data-pressed-text="Now following London Stock Exchange Group."
                                                        data-unpressed-text="No longer following London Stock Exchange Group."
                                                    ></div>
                                                    <button
                                                            aria-pressed="false"
                                                            aria-label="Add London Stock Exchange Group to myFT"
                                                            title="Add London Stock Exchange Group to myFT"
                                                            data-alternate-label="Remove London Stock Exchange Group from myFT"
                                                                    data-alternate-text="Added"
                                                        class="
                                                            n-myft-follow-button"
                                                        data-concept-id="214c6ab8-1eea-48ad-99d2-ac55b0235e05" 
                                                        data-trackable="follow"
                                                        type="submit">Add to myFT</button>
                                                </form>
                                        </li>
                                        <li class="concept-list__list-item">
                                            <a
                                                href="https://www.ft.com/world/europe"
                                                data-trackable="concept"
                                                class="concept-list__concept">
                                                Europe
                                            </a>
                                                <form
                                                    class="n-myft-ui n-myft-ui--follow "
                                                    method="GET"
                                                    data-myft-ui="follow"
                                                    data-concept-id="6374b579-f67d-4970-a7cd-1ab5e41a551a"

                                                            action="/myft/add/6374b579-f67d-4970-a7cd-1ab5e41a551a"
                                                            data-js-action="/__myft/api/core/followed/concept/6374b579-f67d-4970-a7cd-1ab5e41a551a?method=put"
                                                    >
                                                    <input
                                                        data-myft-csrf-token
                                                        value=""
                                                        type="hidden"
                                                        name="token">
                                                    <div
                                                        class="n-myft-ui__announcement o-normalise-visually-hidden"
                                                        aria-live="assertive"
                                                        data-pressed-text="Now following Europe."
                                                        data-unpressed-text="No longer following Europe."
                                                    ></div>
                                                    <button
                                                            aria-pressed="false"
                                                            aria-label="Add Europe to myFT"
                                                            title="Add Europe to myFT"
                                                            data-alternate-label="Remove Europe from myFT"
                                                                    data-alternate-text="Added"
                                                        class="
                                                            n-myft-follow-button"
                                                        data-concept-id="6374b579-f67d-4970-a7cd-1ab5e41a551a" 
                                                        data-trackable="follow"
                                                        type="submit">Add to myFT</button>
                                                </form>
                                        </li>
                                        <li class="concept-list__list-item">
                                            <a
                                                href="https://www.ft.com/stream/69a384b2-ca58-4a1c-9a08-1eba88ea865d"
                                                data-trackable="concept"
                                                class="concept-list__concept">
                                                Clearnet SA
                                            </a>
                                                <form
                                                    class="n-myft-ui n-myft-ui--follow "
                                                    method="GET"
                                                    data-myft-ui="follow"
                                                    data-concept-id="69a384b2-ca58-4a1c-9a08-1eba88ea865d"

                                                            action="/myft/add/69a384b2-ca58-4a1c-9a08-1eba88ea865d"
                                                            data-js-action="/__myft/api/core/followed/concept/69a384b2-ca58-4a1c-9a08-1eba88ea865d?method=put"
                                                    >
                                                    <input
                                                        data-myft-csrf-token
                                                        value=""
                                                        type="hidden"
                                                        name="token">
                                                    <div
                                                        class="n-myft-ui__announcement o-normalise-visually-hidden"
                                                        aria-live="assertive"
                                                        data-pressed-text="Now following Clearnet SA."
                                                        data-unpressed-text="No longer following Clearnet SA."
                                                    ></div>
                                                    <button
                                                            aria-pressed="false"
                                                            aria-label="Add Clearnet SA to myFT"
                                                            title="Add Clearnet SA to myFT"
                                                            data-alternate-label="Remove Clearnet SA from myFT"
                                                                    data-alternate-text="Added"
                                                        class="
                                                            n-myft-follow-button"
                                                        data-concept-id="69a384b2-ca58-4a1c-9a08-1eba88ea865d" 
                                                        data-trackable="follow"
                                                        type="submit">Add to myFT</button>
                                                </form>
                                        </li>
                                </ul>
                            </div>
                    <div
                        class="o-ads o-ads--center o-ads--reserve-"
                        data-o-ads-name="search-refine-mid"
                        data-o-ads-targeting="pos=mid;"
                        data-o-ads-formats-small="false"
                        data-o-ads-formats-medium="MediumRectangle"
                        data-o-ads-formats-large="MediumRectangle,HalfPage,Responsive"
                        data-o-ads-formats-extra="MediumRectangle,HalfPage,Responsive"
                        aria-hidden="true">
                    </div>
            </nav>

        </div>
    </div></div></div><div class="n-layout__row n-layout__row--footer"><div class="n-layout__footer-before"><div class="n-feedback__container n-feedback--hidden"></div></div><footer id="site-footer" class="o-footer o-footer--theme-dark" data-o-component="o-footer"><div class="o-footer__container"><div class="o-footer__row"><h2 class="o-normalise-visually-hidden">Useful links</h2><nav class="o-footer__matrix" role="navigation" aria-label="Useful links"><div class="o-footer__matrix-group o-footer__matrix-group--1"><h3 class="o-footer__matrix-title">Support</h3><div class="o-footer__matrix-content" id="o-footer-section-0"><div class="o-footer__matrix-column"><a class="o-footer__matrix-link" href="/tour" data-trackable="View Site Tips">View Site Tips</a><a class="o-footer__matrix-link" href="https://help.ft.com" data-trackable="Help Centre">Help Centre</a><a class="o-footer__matrix-link" href="https://aboutus.ft.com/en-gb/contact-us/" data-trackable="Contact Us">Contact Us</a><a class="o-footer__matrix-link" href="https://www.ft.com/aboutus" data-trackable="About Us">About Us</a><a class="o-footer__matrix-link" href="/accessibility" data-trackable="Accessibility">Accessibility</a><a class="o-footer__matrix-link" href="/tour/myft" data-trackable="myFT Tour">myFT Tour</a><a class="o-footer__matrix-link" href="https://aboutus.ft.com/en-gb/careers/" data-trackable="Careers">Careers</a></div></div></div><div class="o-footer__matrix-group o-footer__matrix-group--1"><h3 class="o-footer__matrix-title">Legal & Privacy</h3><div class="o-footer__matrix-content" id="o-footer-section-1"><div class="o-footer__matrix-column"><a class="o-footer__matrix-link" href="https://help.ft.com/help/legal-privacy/terms-conditions/" data-trackable="Terms & Conditions">Terms & Conditions</a><a class="o-footer__matrix-link" href="https://help.ft.com/help/legal-privacy/privacy/" data-trackable="Privacy Policy">Privacy Policy</a><a class="o-footer__matrix-link" href="https://help.ft.com/help/legal-privacy/cookies/" data-trackable="Cookies">Cookies</a><a class="o-footer__matrix-link" href="https://help.ft.com/help/legal-privacy/copyright/copyright-policy/" data-trackable="Copyright">Copyright</a><a class="o-footer__matrix-link" href="https://help.ft.com/help/legal/slavery-statement/" data-trackable="Slavery Statement & Policies">Slavery Statement & Policies</a></div></div></div><div class="o-footer__matrix-group o-footer__matrix-group--2"><h3 class="o-footer__matrix-title" aria-controls="o-footer-section-2">Services</h3><div class="o-footer__matrix-content" id="o-footer-section-2"><div class="o-footer__matrix-column"><a class="o-footer__matrix-link" href="https://live.ft.com/" data-trackable="FT Live">FT Live</a><a class="o-footer__matrix-link" href="/securedrop" data-trackable="Share News Tips Securely" data-o-tracking-do-not-track="true">Share News Tips Securely</a><a class="o-footer__matrix-link" href="https://www.ft.com/products" data-trackable="Individual Subscriptions">Individual Subscriptions</a><a class="o-footer__matrix-link" href="https://enterprise.ft.com/en-gb/services/group-subscriptions/group-contact-us/?segmentId=383c7f63-abf4-b62d-cb33-4c278e6fdf61&cpccampaign=B2B_link_ft.com_footer" data-trackable="Group Subscriptions">Group Subscriptions</a><a class="o-footer__matrix-link" href="https://enterprise.ft.com/en-gb/services/republishing/" data-trackable="Republishing">Republishing</a><a class="o-footer__matrix-link" href="http://www.businessesforsale.com/ft2/notices" data-trackable="Contracts & Tenders">Contracts & Tenders</a></div><div class="o-footer__matrix-column"><a class="o-footer__matrix-link" href="https://www.exec-appointments.com/" data-trackable="Executive Job Search">Executive Job Search</a><a class="o-footer__matrix-link" href="http://fttoolkit.co.uk/" data-trackable="Advertise with the FT">Advertise with the FT</a><a class="o-footer__matrix-link" href="https://twitter.com/financialtimes" data-trackable="Follow the FT on Twitter">Follow the FT on Twitter</a><a class="o-footer__matrix-link" href="https://transact.ft.com/" data-trackable="FT Transact">FT Transact</a><a class="o-footer__matrix-link" href="https://enterprise.ft.com/en-gb/services/group-subscriptions/education/" data-trackable="Secondary Schools">Secondary Schools</a></div></div></div><div class="o-footer__matrix-group o-footer__matrix-group--2"><h3 class="o-footer__matrix-title" aria-controls="o-footer-section-3">Tools</h3><div class="o-footer__matrix-content" id="o-footer-section-3"><div class="o-footer__matrix-column"><a class="o-footer__matrix-link" href="https://markets.ft.com/data/portfolio/dashboard" data-trackable="Portfolio">Portfolio</a><a class="o-footer__matrix-link" href="https://www.ft.com/todaysnewspaper" data-trackable="Today&#x27;s Newspaper (ePaper)">Today&#x27;s Newspaper (ePaper)</a><a class="o-footer__matrix-link" href="https://markets.ft.com/data/alerts/" data-trackable="Alerts Hub">Alerts Hub</a><a class="o-footer__matrix-link" href="https://rankings.ft.com/businessschoolrankings/rankings" data-trackable="Business School Rankings">Business School Rankings</a></div><div class="o-footer__matrix-column"><a class="o-footer__matrix-link" href="https://kat.ft.com/" data-trackable="Enterprise Tools">Enterprise Tools</a><a class="o-footer__matrix-link" href="/news-feed" data-trackable="News feed">News feed</a><a class="o-footer__matrix-link" href="/newsletters" data-trackable="Newsletters">Newsletters</a><a class="o-footer__matrix-link" href="https://markets.ft.com/research/Markets/Currencies?segid=70113" data-trackable="Currency Converter">Currency Converter</a></div></div></div></nav></div><div class="o-footer__external-link o-footer__matrix-title"><a class="o-footer__more-from-ft o-footer__matrix-title" href="https://ft.com/more-from-ft-group" data-trackable="more-from-ft">More from the FT Group</a></div><div class="o-footer__copyright"><small>Markets data delayed by at least 15 minutes. © THE FINANCIAL TIMES LTD 2021. <abbr title="Financial Times" aria-label="F T">FT</abbr> and ‘Financial Times’ are trademarks of The Financial Times Ltd.<br/>The Financial Times and its journalism are subject to a self-regulation regime under the <a href="https://aboutus.ft.com/en-gb/ft-editorial-code/" aria-label="F T Editorial Code of Practice">FT Editorial Code of Practice</a>.</small></div></div><div class="o-footer__brand"><div class="o-footer__container"><div class="o-footer__brand-logo"></div></div></div></footer><div class="n-layout__footer-after"><script data-pixel-src="https://spoor-api.ft.com/px.gif?data=%7B%22category%22%3A%22page%22%2C%22action%22%3A%22view%22%2C%22system%22%3A%7B%22source%22%3A%22non-ctm%22%7D%2C%22context%22%3A%7B%22name%22%3A%22search-page%22%2C%22product%22%3A%22next%22%2C%22data%22%3A%7B%22source%22%3A%22%5BSOURCE%5D%22%7D%7D%7D">(function coreExperience() {
        if (/\bcore\b/.test(document.documentElement.className)) {
          var currentScript = document.scripts[document.scripts.length - 1];
          var img = new Image();
          img.src = currentScript.getAttribute('data-pixel-src');
        }
      })();</script><noscript><img src="https://spoor-api.ft.com/px.gif?data=%7B%22category%22%3A%22page%22%2C%22action%22%3A%22view%22%2C%22system%22%3A%7B%22source%22%3A%22non-ctm%22%7D%2C%22context%22%3A%7B%22name%22%3A%22search-page%22%2C%22product%22%3A%22next%22%2C%22data%22%3A%7B%22source%22%3A%22%5BSOURCE%5D%22%7D%7D%7D"/></noscript></div></div><div class="o-header__drawer" id="o-header-drawer" role="navigation" aria-label="Drawer menu" data-o-header-drawer="true" data-o-header-drawer--no-js="true" data-trackable="drawer" data-trackable-terminate="true"><div class="o-header__drawer-inner"><div class="o-header__drawer-tools"><button type="button" class="o-header__drawer-tools-close" title="Close drawer menu" aria-controls="o-header-drawer" data-trackable="close"><span class="o-header__visually-hidden">Close drawer menu</span></button><a class="o-header__drawer-tools-logo" href="/" data-trackable="logo"><span class="o-header__visually-hidden">Financial Times</span></a><p class="o-header__drawer-current-edition">International Edition</p></div><div class="o-header__drawer-search"><form class="o-header__drawer-search-form" action="/search" role="search" aria-label="Site search" data-n-topic-search="true" data-n-topic-search-categories="concepts,equities" data-n-topic-search-view-all="true"><label class="o-header__visually-hidden" for="o-header-drawer-search-term">Search the <abbr title="Financial Times">FT</abbr></label><input type="text" class="o-header__drawer-search-term" id="o-header-drawer-search-term" name="q" autoComplete="off" autoCorrect="off" autoCapitalize="off" spellcheck="false" placeholder="Search the FT" data-trackable="search-term" data-n-topic-search-input="true"/><button class="o-header__drawer-search-submit" type="submit" data-trackable="search-submit"><span class="o-header__visually-hidden">Search</span></button></form></div><nav class="o-header__drawer-menu o-header__drawer-menu--primary o-header__drawer-menu--border"><ul class="o-header__drawer-menu-list"><li class="o-header__drawer-menu-item" data-trackable="edition-switcher"><a class="o-header__drawer-menu-link" href="/?edition=uk" data-trackable="uk">Switch to UK Edition</a></li></ul><ul class="o-header__drawer-menu-list"><li class="o-header__drawer-menu-item o-header__drawer-menu-item--heading">Top sections</li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/" data-trackable="Home">Home</a></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/world" data-trackable="World">World</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-1" data-trackable="sub-level-toggle | World">Show more World</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-1" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/global-economy" data-trackable="Global Economy">Global Economy</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/uk" data-trackable="UK">UK</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/us" data-trackable="US">US</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/asia-pacific/china" data-trackable="China">China</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/africa" data-trackable="Africa">Africa</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/asia-pacific" data-trackable="Asia Pacific">Asia Pacific</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/emerging-markets" data-trackable="Emerging Markets">Emerging Markets</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/europe" data-trackable="Europe">Europe</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/americas" data-trackable="Americas">Americas</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/mideast" data-trackable="Middle East and North Africa">Middle East and North Africa</a></li></ul></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/world/us" data-trackable="US">US</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-2" data-trackable="sub-level-toggle | US">Show more US</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-2" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/us-economy" data-trackable="US Economy">US Economy</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/us" data-trackable="US Companies">US Companies</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/world/us/politics" data-trackable="US Politics & Policy">US Politics & Policy</a></li></ul></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/companies" data-trackable="Companies">Companies</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-3" data-trackable="sub-level-toggle | Companies">Show more Companies</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-3" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/energy" data-trackable="Energy">Energy</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/financials" data-trackable="Financials">Financials</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/health" data-trackable="Health">Health</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/industrials" data-trackable="Industrials">Industrials</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/media" data-trackable="Media">Media</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/professional-services" data-trackable="Professional Services">Professional Services</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/retail-consumer" data-trackable="Retail & Consumer">Retail & Consumer</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/technology" data-trackable="Tech Sector">Tech Sector</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/telecoms" data-trackable="Telecoms">Telecoms</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/companies/transport" data-trackable="Transport">Transport</a></li></ul></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/technology" data-trackable="Tech">Tech</a></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/markets" data-trackable="Markets">Markets</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-5" data-trackable="sub-level-toggle | Markets">Show more Markets</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-5" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/alphaville" data-trackable="Alphaville">Alphaville</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="https://markets.ft.com/data" data-trackable="Markets Data">Markets Data</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/capital-markets" data-trackable="Capital Markets">Capital Markets</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/commodities" data-trackable="Commodities">Commodities</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/currencies" data-trackable="Currencies">Currencies</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/equities" data-trackable="Equities">Equities</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/fund-management" data-trackable="Fund Management">Fund Management</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/ft-trading-room" data-trackable="Trading">Trading</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/moral-money" data-trackable="Moral Money">Moral Money</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="https://etf.ft.com" data-trackable="ETF Hub">ETF Hub</a></li></ul></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/climate-capital" data-trackable="Climate">Climate</a></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/opinion" data-trackable="Opinion">Opinion</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-7" data-trackable="sub-level-toggle | Opinion">Show more Opinion</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-7" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/columnists" data-trackable="Columnists">Columnists</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/ft-view" data-trackable="The FT View">The FT View</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/the-big-read" data-trackable="The Big Read">The Big Read</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/lex" data-trackable="Lex">Lex</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/obituaries" data-trackable="Obituaries">Obituaries</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/letters" data-trackable="Letters">Letters</a></li></ul></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/work-careers" data-trackable="Work & Careers">Work & Careers</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-8" data-trackable="sub-level-toggle | Work & Careers">Show more Work & Careers</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-8" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="https://rankings.ft.com/businessschoolrankings/rankings" data-trackable="Business School Rankings">Business School Rankings</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/business-education" data-trackable="Business Education">Business Education</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/entrepreneurship" data-trackable="Entrepreneurship">Entrepreneurship</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/recruitment" data-trackable="Recruitment">Recruitment</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/business-books" data-trackable="Business Books">Business Books</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/business-travel" data-trackable="Business Travel">Business Travel</a></li></ul></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/life-arts" data-trackable="Life & Arts">Life & Arts</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-9" data-trackable="sub-level-toggle | Life & Arts">Show more Life & Arts</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-9" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/arts" data-trackable="Arts">Arts</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/books" data-trackable="Books">Books</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/food-drink" data-trackable="Food & Drink">Food & Drink</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/magazine" data-trackable="FT Magazine">FT Magazine</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/house-home" data-trackable="House & Home">House & Home</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/style" data-trackable="Style">Style</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/travel" data-trackable="Travel">Travel</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/globetrotter" data-trackable="FT Globetrotter">FT Globetrotter</a></li></ul></li><li class="o-header__drawer-menu-item"><div class="o-header__drawer-menu-toggle-wrapper"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--parent" href="/personal-finance" data-trackable="Personal Finance">Personal Finance</a><button class="o-header__drawer-menu-toggle o-header__drawer-menu-toggle--unselected" aria-controls="o-header-drawer-child-10" data-trackable="sub-level-toggle | Personal Finance">Show more Personal Finance</button></div><ul class="o-header__drawer-menu-list o-header__drawer-menu-list--child" id="o-header-drawer-child-10" data-trackable="sub-level"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/personal-finance/mortgages" data-trackable="Property & Mortgages">Property & Mortgages</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/personal-finance/investments" data-trackable="Investments">Investments</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/personal-finance/pensions" data-trackable="Pensions">Pensions</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/personal-finance/tax" data-trackable="Tax">Tax</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/personal-finance/banking-savings" data-trackable="Banking & Savings">Banking & Savings</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/personal-finance/advice-commment" data-trackable="Advice & Comment">Advice & Comment</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--child" href="/next-act" data-trackable="Next Act">Next Act</a></li></ul></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/htsi" data-trackable="How to Spend It">How to Spend It</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/special-reports" data-trackable="Special Reports">Special Reports</a></li><li class="o-header__drawer-menu-item o-header__drawer-menu-item--heading">FT recommends</li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/graphics" data-trackable="Graphics">Graphics</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/lex" data-trackable="Lex">Lex</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/alphaville" data-trackable="Alphaville">Alphaville</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/lunch-with-the-ft" data-trackable="Lunch with the FT">Lunch with the FT</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/globetrotter" data-trackable="FT Globetrotter">FT Globetrotter</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="https://www.ft.com/tech-asia" data-trackable="#techAsia">#techAsia</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/moral-money" data-trackable="Moral Money">Moral Money</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/ftfm" data-trackable="FTfm">FTfm</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/newsletters" data-trackable="Newsletters">Newsletters</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/video" data-trackable="Video">Video</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="https://www.ft.com/podcasts" data-trackable="Podcasts">Podcasts</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected" href="/news-feed" data-trackable="News feed">News feed</a></li><li class="o-header__drawer-menu-item o-header__drawer-menu-item--divide"><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--secondary" href="/myft" data-trackable="myFT">myFT</a></li><li class="o-header__drawer-menu-item "><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--secondary" href="https://markets.ft.com/data/portfolio/dashboard" data-trackable="Portfolio">Portfolio</a></li><li class="o-header__drawer-menu-item "><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--secondary" href="https://www.ft.com/todaysnewspaper" data-trackable="Today&#x27;s Newspaper (ePaper)">Today&#x27;s Newspaper (ePaper)</a></li><li class="o-header__drawer-menu-item "><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--secondary" href="https://www.ft.com/crossword" data-trackable="Crossword">Crossword</a></li><li class="o-header__drawer-menu-item "><a class="o-header__drawer-menu-link o-header__drawer-menu-link--unselected o-header__drawer-menu-link--secondary" href="https://www.ft.com/tour/apps" data-trackable="Our Apps">Our Apps</a></li></ul></nav><nav class="o-header__drawer-menu o-header__drawer-menu--user" data-trackable="user-nav"><ul class="o-header__drawer-menu-list"><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link" href="https://help.ft.com" data-trackable="Help Centre">Help Centre</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link" href="/products?segmentId=d290332b-e8e8-d29b-2ff4-2019b13f008a" data-trackable="Subscribe">Subscribe</a></li><li class="o-header__drawer-menu-item"><a class="o-header__drawer-menu-link" href="/login?location=/search" data-trackable="Sign In">Sign In</a></li></ul></nav></div></div></div><script type="application/json" id="page-kit-app-context">{"appName":"search-page","product":"next","edition":"international","appVersion":"8877cff120fdb2086dd6aa207f963e792366ff7d","abTestState":"messageSlotTop:threeMonthsIntroPricePromo,messageSlotBottom:cookieConsentC,apprecommendedarticles:variant1","isProduction":true,"isUserLoggedIn":false,"pageKitVersion":"2.3.0"}</script><script type="application/json" id="page-kit-flags-embed">{"articleRelatedContent":true,"ads":true,"javascript":true,"oTracking":true,"articleComments":true,"frontPageHeaderMarketsData":true,"myFtApi":true,"articleShareButtons":true,"customStreamFragments":true,"frontPageMyFtSection":true,"myFtApiWrite":true,"adTargetingUserApi":true,"videoPlayerAdvertising":true,"streamMostRead":true,"serviceWorker":true,"openGraph":true,"googleConsumerSurvey":true,"nativeAds":true,"myFtDigestPromo":true,"regionalNews":true,"syndication":true,"myFTEmailDisableABAlloc":true,"streamRecommendedStoriesList":true,"myFtNoContentDigests":true,"swAssetCaching":true,"refererCohort":true,"newsletterSignupOnArticle":true,"myFTInstantNotifications":true,"myftPrioritiseTopics":true,"messageSlotTop":"threeMonthsIntroPricePromo","myftPrioritiseTopicsOnHomepage":true,"myftIndustryRecs":true,"qualtrics":true,"showEventPromo":true,"myftConsentSwitch":true,"moatAdsTraffic":true,"manageCookiesWhiteLabel":true,"fcmSwitch":true,"graphicsPage":true,"myftIndustryRecsFeed":true,"myftArticleInstantAlertsCta":true,"b2bLiveChat":true,"AdsPermutive":true,"syndication_promo":true,"topicTracker":true,"messageSlotBottom":"cookieConsentC","enableGTM":true,"myftUnreadFavicon":true,"myftNewNewLabel":true,"myftBoldNewArticles":true,"realUserMonitoringForPerformance":true,"pageAttentionTracking":true,"textCopyTracking":true,"enableIHSindexBasedTargetingScript":true,"enableBrandmetricsSurveyScript":true,"managePremiumCancellationJourney":true,"articleInstantAlertsPromo":true,"manageStepUpCancellationJourney":true,"manageStandardCancellationJourney":true,"coronavirusStats":true,"manageStepUpTransitionJourney":true,"manageDirectCancellationJourney":true,"privShowCCPALinkOnFooter":true,"useXDashForLiveBlogs":true,"customerCareLiveChat":true,"apprecommendedarticles":"variant1","liveBlogsV2NewPostButton":true,"optimizely":true,"accountSettingsBillingDetailsPaypal":true,"graphicSyndication":true,"showApplePay":true,"adsNativeLazyload":true,"adsMoatTimeout":true,"enableRavelinOnProd":true,"enableMarketingAnalyticsTags":true,"liteSubscriptionSmokeTest":true}</script></body></html>'''

In [29]:
xpath = '//*[@id="site-content"]/div/ul/li[5]/div/div/div/div/p//text()'

In [30]:
sel.xpath(xpath).extract()

[]

## Newspaper3k

Site newspaper3k cannot extract from: 
- modernreaders.com
- tradenews.com

In [236]:
url = 'https://www.thetradenews.com/eurex-hits%E2%80%AFesg-derivatives-daily-trading-record%E2%80%AF/'

article = Article(url)
article.download()
article.parse()
article.nlp()

title = article.title
text = article.text
summary = article.summary
date = article.publish_date

article

ArticleException: Article `download()` failed with HTTPSConnectionPool(host='www.thetradenews.com', port=443): Read timed out. (read timeout=7) on URL https://www.thetradenews.com/eurex-hits%E2%80%AFesg-derivatives-daily-trading-record%E2%80%AF/

## Google News 

In [165]:
from GoogleNews import GoogleNews

# Max and Min Date of Search
start_dt = (datetime.today() - timedelta(days=search_days)).strftime('%m/%d/%Y')
end_dt = datetime.today().strftime('%m/%d/%Y')

# Fetch News 
googlenews=GoogleNews(start=start_dt,end=end_dt, lang='en') # language english
googlenews.search(searchword)
result=googlenews.result()

# List of news urls 
urls = [news.get('link') for news in result]
title_list = [news.get('title') for news in result]
api_list = [news.get('media') for news in result]
date_list = [news.get('datetime').strftime('%Y/%m/%d') for news in result]
summary_list = [news.get('desc') for news in result]

In [168]:
api_list

["Barron's", '', 'The TRADE News', 'SoccerNurds', 'FinanceFeeds', '']

In [170]:
result[3]

{'title': 'Hardfacing by Plasma Transferred Arc Process Market Is ...',
 'media': 'SoccerNurds',
 'date': '8 uur geleden',
 'datetime': datetime.datetime(2021, 2, 22, 5, 49, 51, 320032),
 'desc': '... Avweld Australasia Pty Ltd (United States), Eurex (Austria), Arcraft Plasma Equipments Pvt Ltd. (India), Mackays (South Africa), Technoplus srl (Romania).',
 'link': 'https://soccernurds.com/uncategorized/1823319/hardfacing-by-plasma-transferred-arc-process-market-is-booming-worldwide-commersald-s-p-a-italy-f-w-gartner-thermal-spraying-united-states-oerlikon-metco-switzerland/',
 'img': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='}

## Article Verification

__Language Detection__

In [40]:
# Authenticate Azure text analysis client 
client = authenticate_azure_textanalytics_client()

In [65]:
documents = cpt_df.iloc[7:7, :].apply(lambda row: row.Article if len(row.Article)>25 else row.Title, axis=1)

In [69]:
row = cpt_df.iloc[7]
text = row.Article if len(row.Article)>25 else row.Title

In [74]:
documents = [text[:5120]]
response = client.detect_language(documents = documents)[0]
language = response.primary_language.name
language

'English'

__Check for searchword in title__

In [88]:
cpt_df[['NER', 'Topics']].head(1)

,NER,Topics
0,'BNP Paribas AM''BNP Paribas Inclusive Growth',"'revamps thematic fund', 'BNP Paribas', 'inequ..."


In [96]:
type(cpt_df.NER[0])

pandas.core.series.Series

In [124]:
cpt_df.head(2)

,API,Article,Article_Negativ,Article_Neutral,Article_Positv,Article_Sentiment,CPT,Date,Link,Summary,Title,Title_Negativ,Title_Neutral,Title_Positv,Title_Sentiment,Verified,Verified_Flag,Found_Keywords,NER,Topics
0,Google citywireselector.com,The strategy will be renamed as the BNP Pariba...,0.03,0.79,0.18,neutral,BNP Paribas,2021/02/23,http://citywireselector.com/news/bnp-paribas-a...,The strategy will be renamed as the BNP Pariba...,BNP Paribas AM revamps thematic fund to tackle...,0.01,0.95,0.04,neutral,LCH SA does not appear in the title.,False,-,'BNP Paribas AM''BNP Paribas Inclusive Growth',"'revamps thematic fund', 'BNP Paribas', 'inequ..."
1,Google,"Boudewijn Dierick, BNP Paribas’ head of ABS an...",0,0.99,0.01,neutral,BNP Paribas,None,https://www.globalcapital.com/article/b1qp0968...,"Boudewijn Dierick, BNP Paribas’ head of ABS an...",Head of ABS and covered bonds leaves BNP Paribas,0.00,0.99,0.01,neutral,LCH SA does not appear in the title.,False,-,"'ABS', 'BNP Paribas''Auxmoney Investments', 'B...","'covered bonds', 'Head of ABS', 'BNP Paribas''..."


In [116]:
# Authenticate connection to azure text analytics service
text_analytics_client = authenticate_azure_textanalytics_client()

def get_text_topics(client, text):
    """"""
    text = text[:5120]
    document = [text]

    try:
        response = text_analytics_client.extract_key_phrases(document)
        result = [doc for doc in response if not doc.is_error]
        topics = result[0].key_phrases
        topics = [topic.lower() for topic in topics]
    except: 
        topics = []
    topics = str(topics)[1:-1]
    return topics

cpt_df['Title_Topics'] = cpt_df.Title.apply(get_text_topics)

In [117]:
cpt_df['Title_Topics'] = cpt_df.Title.apply(get_text_topics)

TypeError: get_text_topics() missing 1 required positional argument: 'text'

In [123]:
text = 'BNP Paribas AM revamps thematic fund to tackle inequalities'
document = [text]

response = text_analytics_client.extract_key_phrases(document)
result = [doc for doc in response if not doc.is_error]
topics = result[0].key_phrases
topics = [topic.lower() for topic in topics]
topics #= str(topics)[1:-1]

['revamps thematic fund', 'bnp paribas', 'inequalities']

In [122]:
result

[ExtractKeyPhrasesResult(id=0, key_phrases=['revamps thematic fund', 'BNP Paribas', 'inequalities'], warnings=[], statistics=None, is_error=False)]

__Functions__

In [81]:
def verify_article(row, searchword):
    """"""
    
    def language_detection_example(text, client):
        try:
            documents = [text[:5120]]
            response = client.detect_language(documents = documents)[0]
            language = response.primary_language.name
            return language
        except Exception as err:
            return 'Unknown'
        
    verified = '' 

    #Check Language of Article
    text = row.Article if len(row.Article)>25 else row.Title
    language = language_detection_example(row.Article, client)
    if (language != 'English'):
        verified += 'Detected language is ' + language + '. '

    # Check if Article is actually about searchword
    if (searchword not in row.Title):
        verified += searchword + ' does not appear in the title.'
    
    return verified 

In [85]:
def verify_articles(dff, searchword):
    """"""
    # Authenticate Azure text analysis client 
    ta_client = authenticate_azure_textanalytics_client()
    
    def get_text_topics(client, text):
        """"""
        text = text[:5120]
        document = [text]

        try:
            response = text_analytics_client.extract_key_phrases(document)
            result = [doc for doc in response if not doc.is_error]
            topics = result[0].key_phrases
            topics = [topic.lower() for topic in topics]
        except: 
            topics = []
        topics = str(topics)[1:-1]
        return topics
    def language_detection_example(text, client):
        try:
            documents = [text[:5120]]
            response = client.detect_language(documents = documents)[0]
            language = response.primary_language.name
            return language
        except Exception as err:
            return 'Unknown'
        
    verified_col = []
    for row in dff.itertuples():
        verified = '' 

        #Check Language of Article
        text = row.Article if len(row.Article)>25 else row.Title
        language = language_detection_example(row.Article, ta_client)
        if (language != 'English'):
            verified += 'Detected language is ' + language + '. '

        # Check if Article is actually about searchword
        title_topic_list = get_text_topics(ta_client, row.Title)
        if (searchword.lower() not in title_topic_list):
            verified += searchword + ' is not subject of the title.'

        # Is verified
        verified_col.append(verified)

    dff['Verified'] = verified_col
    dff['Verified_Flag'] = 'True' if (len(verified_col)<2) else 'False'
    
    return dff 
#dff = verify_articles(dff, searchword)

## Topic Search

In [120]:
def add_topic_search(dff):
    """"""
    # Authenticate Azure text analysis client 
    text_analytics_client = authenticate_azure_textanalytics_client()
    
    def get_article_topics(article):
        """"""
        article = article[:5120]
        document = [article]

        try:
            response = text_analytics_client.extract_key_phrases(document)
            result = [doc for doc in response if not doc.is_error]
            topics = result[0].key_phrases
        except: 
            topics = []
        topics = str(topics)[1:-1]
        return topics
    
    # Add Ner column 
    dff['Topics'] = dff.Article.apply(get_article_topics)
    
    return dff 

In [123]:
def get_article_topics(article):
    """"""
    article = article[:5120]
    document = [article]
    
    try:
        response = text_analytics_client.extract_key_phrases(document)
        result = [doc for doc in response if not doc.is_error]
        topics = result[0].key_phrases
    except: 
        topics = []
    topics = str(topics)[1:-1]
    return topics

In [9]:
dff = add_topic_search(dff)
dff.head()

,API,Link,Date,Title,Article,Summary,Title_Sentiment,Title_Positv,Title_Neutral,Title_Negativ,Article_Sentiment,Article_Positv,Article_Neutral,Article_Negativ,Verified,Verified_Flag,Found_Keywords,NER,Topics
0,Bing,https://www.leaprate.com/forex/market-news/for...,2021-02-12,"Forex Weekly Look Back: StoneX revenue up 1,20...",Multi-asset broker Exness will be taking part ...,,neutral,0.00,1.00,0.00,neutral,0.02,0.97,0.01,EUREX does not appear in the title.,False,,Bing,"'iFEX virtual expo', 'financial virtual expo',..."
1,Bing,https://www.modernreaders.com/news/2021/02/12/...,2021-02-12,Contrasting Deutsche Börse (DBOEY) and Its Rivals,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,Analyst RatingsGet Deutsche Börse alerts:This ...,neutral,0.04,0.78,0.18,mixed,0.22,0.23,0.55,EUREX does not appear in the title.,False,,Bing,"'Deutsche Börse rivals', 'Deutsche Börse Compe..."
2,Bing,https://theenterpriseleader.com/2021/02/12/deu...,2021-02-12,Deutsche Börse (DBOEY) versus Its Peers Head t...,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,DividendsGet Deutsche Börse alerts:Deutsche Bö...,neutral,0.01,0.96,0.03,mixed,0.29,0.19,0.52,EUREX does not appear in the title.,False,,Bing,"'Deutsche Börse peers', 'Deutsche Börse Compet..."
3,Bing,https://theenterpriseleader.com/2021/02/12/fin...,2021-02-12,Financial Comparison: Deutsche Börse (DBOEY) a...,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,DividendsGet Deutsche Börse alerts:Deutsche Bö...,neutral,0.06,0.89,0.05,mixed,0.31,0.20,0.49,EUREX does not appear in the title.,False,,Bing,"'Deutsche Börse peers', 'Deutsche Börse Compet..."
4,Bing,https://apnews.com/press-release/pr-newswire/t...,2021-02-12,Fenics GO named OTC Trading Platform of the Year,"LONDON, Feb. 11, 2021 /PRNewswire/ -- Fenics G...","LONDON, Feb. 11, 2021 /PRNewswire/ -- Fenics G...",neutral,0.01,0.98,0.01,mixed,0.26,0.09,0.65,Detected language is Unknown. EUREX does not a...,False,,Bing,"'Fenics division of BGC Partners', 'resiliency..."


## NER Extraction

https://azuresdkdocs.blob.core.windows.net/$web/python/azure-ai-textanalytics/1.0.0b1/index.html

In [9]:
# Authenticate connection to azure text analytics service
text_analytics_client = authenticate_azure_textanalytics_client()

In [76]:
def add_ner_search(dff):
    """"""
    
    # Authenticate connection to azure text analytics service
    text_analytics_client = authenticate_azure_textanalytics_client()
    
    # NER Search
    def get_ner_string(article):
        """"""
        article = article[:5120] # Max length for article # should be split in multisearch
        document = [article]

        response = text_analytics_client.recognize_entities(document , language="en")
        result = [doc for doc in response if not doc.is_error]

        ner_entities = []
        ner_categories, ner_scores = [], []

        for entity in result[0].entities:
            if (entity.category in ['Organization']):#,'Skill']):
                ner_entities.append(entity.text) 
                ner_categories.append(entity.category)
                #ner_scores += entity.confidence_score

        return str(list(set(ner_entities)))[1:-1]
    
    # Add Ner column 
    dff['NER'] = dff.Article.apply(get_ner_string)
    
    return dff 
df['NER'] = add_ner_search(df)

,API,Link,Date,Title,Article,Summary,Title_Sentiment,Title_Positv,Title_Neutral,Title_Negativ,Article_Sentiment,Article_Positv,Article_Neutral,Article_Negativ,Found_Keywords,NER
0,Bing,https://www.leaprate.com/forex/market-news/for...,2021-02-12,"Forex Weekly Look Back: StoneX revenue up 1,20...",Multi-asset broker Exness will be taking part ...,,neutral,0.00,1.00,0.00,neutral,0.02,0.97,0.01,,'Exness'
1,Bing,https://www.modernreaders.com/news/2021/02/12/...,2021-02-12,Contrasting Deutsche Börse (DBOEY) and Its Rivals,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,Analyst RatingsGet Deutsche Börse alerts:This ...,neutral,0.04,0.78,0.18,mixed,0.22,0.23,0.55,,"'Xetra', 'GSF', 'EEX', 'MarketBeat.com', 'Qont..."
2,Bing,https://theenterpriseleader.com/2021/02/12/deu...,2021-02-12,Deutsche Börse (DBOEY) versus Its Peers Head t...,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,DividendsGet Deutsche Börse alerts:Deutsche Bö...,neutral,0.01,0.96,0.03,mixed,0.29,0.19,0.52,,"'Xetra', 'GSF', 'EEX', 'MarketBeat.com', 'Qont..."
3,Bing,https://theenterpriseleader.com/2021/02/12/fin...,2021-02-12,Financial Comparison: Deutsche Börse (DBOEY) a...,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,DividendsGet Deutsche Börse alerts:Deutsche Bö...,neutral,0.06,0.89,0.05,mixed,0.31,0.20,0.49,,"'Deutsche Börse', 'MarketBeat', 'Data', 'Clear..."
4,Bing,https://apnews.com/press-release/pr-newswire/t...,2021-02-12,Fenics GO named OTC Trading Platform of the Year,"LONDON, Feb. 11, 2021 /PRNewswire/ -- Fenics G...","LONDON, Feb. 11, 2021 /PRNewswire/ -- Fenics G...",neutral,0.01,0.98,0.01,mixed,0.26,0.09,0.65,,"'Citadel Securities', 'BGC Trader', 'BGC Group..."
5,Bing,https://www.modernreaders.com/news/2021/02/12/...,2021-02-12,Deutsche Börse (DBOEY) and The Competition Cri...,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,Given Deutsche Börse’s peers stronger consensu...,negative,0.00,0.01,0.99,mixed,0.33,0.21,0.46,,"'Investment', 'Xetra', 'GSF', 'EEX', 'Qontigo'..."
6,Bing,https://www.financemagnates.com/institutional-...,2021-02-10,Clearstream Reports €15.4 Trillion in Assets U...,Settlement volumes at Deutsche Boerse AG’s Cle...,Settlement volumes at Deutsche Boerse AG’s Cle...,neutral,0.01,0.89,0.10,mixed,0.27,0.11,0.62,,"'Xetra', 'Deutsche Börse Group', 'Deutsche Boe..."
7,Bing,https://www.vindobona.org/article/vienna-stock...,2021-02-09,Vienna Stock Exchange: New Calculation Technol...,The Vienna Stock Exchange (Wiener Börse AG) ha...,,neutral,0.01,0.99,0.00,positive,0.50,0.48,0.02,,"'Wiener Börse AG', 'Vienna Stock Exchange'"
8,Bing,https://www.leaprate.com/financial-services/ex...,2021-02-08,Eurex traded contracts fall 13%MoM in January ...,Eurex Repo posted drops 25.2% MoM rise in GC P...,,neutral,0.00,0.96,0.04,neutral,0.01,0.87,0.12,,"'Eurex Repo', 'GC'"


In [12]:
for row in dfc.itertuples():
    article = row.Article
    print('x')

    article = article[:5120] # Max length for article # should be split in multisearch
    document = [article]

    response = text_analytics_client.recognize_entities(document , language="en")
    result = [doc for doc in response if not doc.is_error]

    ner_entities = []
    ner_categories, ner_scores = [], []

    for entity in result[0].entities:
        if (entity.category in ['Organization']):#,'Skill']):
            ner_entities.append(entity.text) 
            ner_categories.append(entity.category)
            #ner_scores += entity.confidence_score

    print(str(list(set(ner_entities)))[1:-1])

x
'Eurex'
x


IndexError: list index out of range

In [57]:
documents = [
    "Microsoft was founded by Bill Gates and Paul Allen. \
     Redmond is a city in King County, Washington, United States, located 15 miles east of Seattle.",
    "Jeff bought three dozen eggs because there was a 50% discount."
]
documents = [df.Article[1]]

response = text_analytics_client.recognize_entities(documents, language="en")
result = [doc for doc in response if not doc.is_error]

for doc in result:
    print('XXX')
    for entity in doc.entities:
        print("Entity: ", entity.text, "\tCategory: ", entity.category,
              "\tConfidence Score: ", entity.confidence_score)

XXX
Entity:  Deutsche Börse 	Category:  Organization 	Confidence Score:  0.98
Entity:  one 	Category:  Quantity 	Confidence Score:  0.8
Entity:  23 	Category:  Quantity 	Confidence Score:  0.8
Entity:  Security 	Category:  Skill 	Confidence Score:  0.8
Entity:  commodity 	Category:  Skill 	Confidence Score:  0.8
Entity:  Deutsche Börse 	Category:  Organization 	Confidence Score:  0.97
Entity:  businesses 	Category:  Skill 	Confidence Score:  0.8
Entity:  analyst 	Category:  PersonType 	Confidence Score:  0.86
Entity:  analyst 	Category:  Skill 	Confidence Score:  0.8
Entity:  valuation 	Category:  Skill 	Confidence Score:  0.8
Entity:  Analyst 	Category:  Skill 	Confidence Score:  0.8
Entity:  Deutsche Börse 	Category:  Organization 	Confidence Score:  0.95
Entity:  Deutsche Börse 	Category:  Organization 	Confidence Score:  0.96
Entity:  MarketBeat.com 	Category:  Organization 	Confidence Score:  0.9
Entity:  Deutsche Börse 	Category:  Organization 	Confidence Score:  0.78
Entity:  0 

Entity:  Deutsche Börse 	Category:  Organization 	Confidence Score:  0.95
Entity:  MarketBeat 	Category:  Organization 	Confidence Score:  0.65
Entity:  MarketBeat.com 	Category:  Organization 	Confidence Score:  0.9
Entity:  daily 	Category:  DateTime 	Confidence Score:  0.8


## Keyword Search

In [14]:
from fuzzysearch import find_near_matches

In [20]:
find_near_matches('PATTERN', '---PATERN--- PATTERN pattern', max_l_dist=1)

[Match(start=3, end=9, dist=1, matched='PATERN'),
 Match(start=13, end=20, dist=0, matched='PATTERN')]

In [42]:
def find_keywords(article):
    """"""
    from fuzzysearch import find_near_matches
    keywords = ['fraud', 'claim', 'incident', 'bankruptcy', 'fine', 'bankrupt',
                'Fraud', 'Claim', 'Incident', 'Bankruptcy', 'Fine', 'Bankrupt']
    words = ''
    for keyword in keywords:
        count = len(find_near_matches(keyword, article, max_l_dist=0))
        if(count>0):
            words += str(count)+'x '+keyword+', '
    if(len(words)>1):
        words = words[:-2]
    return words

In [39]:
len( find_near_matches('fin', 'fineland', max_l_dist=0))

1

## Message: Overall Sentiment

In [5]:
# Overall Sentiment  
sentiment_list = [*dff.Title_Sentiment] + [*dff.Article_Sentiment]
sentiment = max(set(sentiment_list), key = sentiment_list.count) 
message += 'The overall sentiment of the articles was '+sentiment + '.\n\n'
# Mean sentiment
def str_mean(x):
    return str(round(x.mean(),2))

message += 'Mean Sentiment: Title ('\
                            + str_mean(dff.Title_Positv)+', '\
                            + str_mean(dff.Title_Neutral)+', '\
                            + str_mean(dff.Title_Negativ)+'); '\
                        +'Article ('+str_mean(dff.Article_Positv)+', '\
                                    +str_mean(dff.Article_Positv)+', '\
                                    +str_mean(dff.Article_Negativ)+').+\n\n'
# Negativ Articles
# Articles with a negativ score >0.33
neg_th =  0.33
neg_df = dff[(dff.Title_Negativ>neg_th)|(dff.Title_Negativ>neg_th)]
neg_count=len(neg_df) 
if(neg_count==1):
    message += 'There is '+str(neg_count) + ' articles with a negativ sentiment:\n'
elif(neg_count>1):
    message += 'There are '+str(neg_count) + ' articles with a negativ sentiment:\n'
for row in neg_df.itertuples():
     message += '\t'+row.Title + ' ['+row.Title_Sentiment+' title ('\
                                        +str(row.Title_Positv)+', '\
                                        +str(row.Title_Neutral)+', '\
                                        +str(row.Title_Negativ)+'); '\
                                    +row.Article_Sentiment+' article ('\
                                        +str(row.Article_Positv)+', '\
                                        +str(row.Article_Neutral)+', '\
                                        +str(row.Article_Negativ)+')]+\n\n'

message

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [97]:
# # Overall Sentiment  
# message=''
# # Overall Sentiment  
# sentiment_list = [*df.Title_Sentiment] + [*df.Article_Sentiment]
# sentiment = max(set(sentiment_list), key = sentiment_list.count) 
# message += 'The overall sentiment of the articles was '+sentiment + '.\n\n'
# # Mean sentiment
# def str_mean(x):
#     return str(round(x.mean(),2))
    
# message += 'Mean Sentiment: Title ('\
#                             + str_mean(df.Title_Positv)+', '\
#                             + str_mean(df.Title_Neutral)+', '\
#                             + str_mean(df.Title_Negativ)+'); '\
#                         +'Article ('+str_mean(df.Article_Positv)+', '\
#                                     +str_mean(df.Article_Positv)+', '\
#                                     +str_mean(df.Article_Negativ)+').+\n\n'
# # Negativ Articles
# # Articles with a negativ score >0.33
# neg_th =  0.33
# neg_df = df[(df.Title_Negativ>neg_th)|(df.Title_Negativ>neg_th)]
# neg_count=len(neg_df) 
# if(neg_count==1):
#     message += 'There is '+str(neg_count) + ' articles with a negativ sentiment:\n'
# elif(neg_count>1):
#     message += 'There are '+str(neg_count) + ' articles with a negativ sentiment:\n'
# for row in neg_df.itertuples():
#      message += '\t'+row.Title + ' ['+row.Title_Sentiment+' title ('\
#                                         +str(row.Title_Positv)+', '\
#                                         +str(row.Title_Neutral)+', '\
#                                         +str(row.Title_Negativ)+'); '\
#                                     +row.Article_Sentiment+' article ('\
#                                         +str(row.Article_Positv)+', '\
#                                         +str(row.Article_Neutral)+', '\
#                                         +str(row.Article_Negativ)+')]+\n\n'



# # import pprint
# pp.pprint(message)

('The overall sentiment of the articles was neutral.\n'
 '\n'
 'Mean Sentiment: Title (0.09, 0.85, 0.06); Article (0.22, 0.22, 0.27).+\n'
 '\n'
 'There is 1 articles with a negativ sentiment:\n'
 '\tCritical Contrast: Deutsche Börse (DBOEY) and Its Rivals [neutral title '
 '(0.07, 0.58, 0.35); mixed article (0.33, 0.25, 0.42)]+\n'
 '\n')


## FT News

### Test ft

In [28]:
import requests
from pprint import pprint

apikey = '59cbaf20e3e06d3565778e7b7e93393d34f24330818d4de7f89b27a1'
timestamp = "2021-01-01T00:00:00.000Z"
#
query_url = f"https://api.ft.com/content/notifications?apiKey={apikey}&since={timestamp}"

r = requests.get(query_url)
pprint(r.json())

{'links': [{'href': 'https://api.ft.com/content/notifications?page=MjAyMTAxMDQyMzIxMjYuNzc0IzAjdXBwLXByb2QtZGVsaXZlcnktZXU%3D&since=2021-01-04T23%3A21%3A26.774Z',
            'rel': 'next'}],
 'notifications': [{'apiUrl': 'https://api.ft.com/content/6ed46c9b-ce04-48c3-aa38-68efbc4eebb9',
                    'id': 'http://www.ft.com/thing/6ed46c9b-ce04-48c3-aa38-68efbc4eebb9',
                    'type': 'http://www.ft.com/thing/ThingChangeType/UPDATE'},
                   {'apiUrl': 'https://api.ft.com/content/deb46ed1-b6c3-4e57-9054-1dca16dadad0',
                    'id': 'http://www.ft.com/thing/deb46ed1-b6c3-4e57-9054-1dca16dadad0',
                    'type': 'http://www.ft.com/thing/ThingChangeType/UPDATE'},
                   {'apiUrl': 'https://api.ft.com/content/d9f91804-105c-4e17-b88a-3d1017a6ba0e',
                    'id': 'http://www.ft.com/thing/d9f91804-105c-4e17-b88a-3d1017a6ba0e',
                    'type': 'http://www.ft.com/thing/ThingChangeType/UPDATE'},
          

                   {'apiUrl': 'https://api.ft.com/content/a6275216-bce4-4a5a-a582-3a99a6042732',
                    'id': 'http://www.ft.com/thing/a6275216-bce4-4a5a-a582-3a99a6042732',
                    'type': 'http://www.ft.com/thing/ThingChangeType/UPDATE'},
                   {'apiUrl': 'https://api.ft.com/content/5c2d1a1e-f783-43bd-a7ab-bb34f663309d',
                    'id': 'http://www.ft.com/thing/5c2d1a1e-f783-43bd-a7ab-bb34f663309d',
                    'type': 'http://www.ft.com/thing/ThingChangeType/UPDATE'},
                   {'apiUrl': 'https://api.ft.com/content/644035a3-fc42-4720-b414-8a9c3457799a',
                    'id': 'http://www.ft.com/thing/644035a3-fc42-4720-b414-8a9c3457799a',
                    'type': 'http://www.ft.com/thing/ThingChangeType/UPDATE'},
                   {'apiUrl': 'https://api.ft.com/content/15ddd567-9a78-472c-959b-0f54dafbbbd1',
                    'id': 'http://www.ft.com/thing/15ddd567-9a78-472c-959b-0f54dafbbbd1',
               

In [35]:
apikey = '59cbaf20e3e06d3565778e7b7e93393d34f24330818d4de7f89b27a1'
timestamp = "2021-01-01T00:00:00.000Z"
content='0be503b0-ff9e-11de-921f-00144feabdc0'
#
query_url = f"https://api.ft.com/content/{content}&apiKey={apikey}"

r = requests.get(query_url)
pprint(r.json())

{'error': 'Access to this API has been disallowed'}


In [32]:
apikey = '59cbaf20e3e06d3565778e7b7e93393d34f24330818d4de7f89b27a1'
timestamp = "2021-01-01T00:00:00.000Z"
#
query_url = f"https://api.ft.com/content/7348edd8-0403-11de-845b-000077b07658?apiKey={apikey}"

r = requests.get(query_url)
pprint(r.json())

{'error': 'Access to this API has been disallowed'}


In [76]:
apikey = '59cbaf20e3e06d3565778e7b7e93393d34f24330818d4de7f89b27a1'

query_url = f"https://api.ft.com/content/items/v1/0be503b0-ff9e-11de-921f-00144feabdc0?apiKey={apikey}"

r = requests.get(query_url)
pprint(r.json())

{'error': 'Access to this API has been disallowed'}


In [73]:
#  Retrieve all published pages
apikey = '59cbaf20e3e06d3565778e7b7e93393d34f24330818d4de7f89b27a1'

query_url = f"https://api.ft.com/site/v1/pages?apiKey={apikey}"

r = requests.get(query_url)
pprint(r.json())

JSONDecodeError: Extra data: line 1 column 5 (char 4)

### Test Rapid API

- https://rapidapi.com/blog/rapidapi-featured-news-apis/

In [ ]:
import requests

url = "https://financialtimesmikilior1v1.p.rapidapi.com/getAspectsList"

payload = "apiKey=%3CREQUIRED%3E"
headers = {
    'content-type': "application/x-www-form-urlencoded",
    'x-rapidapi-key': "59cbaf20e3e06d3565778e7b7e93393d34f24330818d4de7f89b27a1",
    'x-rapidapi-host': "FinancialTimesmikilior1V1.p.rapidapi.com"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

### Test Currents

- https://currentsapi.services/en/profile
- https://towardsdatascience.com/scraping-news-and-articles-from-public-apis-with-python-be84521d85b9

In [44]:
# https://currentsapi.services/en/docs/search
apikey = 'Zm0T2sOpC2G9LInaOGnXBdDwU29-U__Z_rXixLa-dJQdaUb2'
category = "business"
language = 'en'  # Mapping from Language to Code, e.g.: "English": "en"
country = "CA"  # Mapping from Country to Code, e.g.: "Canada": "CA",
keywords = "bitcoin"
t = "1"  # 1 for news, 2 for article and 3 for discussion content
domain = "financialpost.com"  # website primary domain name (without www or blog prefix)
start_date = "2020-06-01T14:30"  # YYYY-MM-DDTHH:MM:SS+00:00
query_url = f"https://api.currentsapi.services/v1/search?" \
            f"apiKey={apikey}" \
            f"&language={language}" \
            f"&category={category}" \
            f"&country={country}" \
            f"&type={t}" \
            f"&domain={domain}" \
            f"&keywords={keywords}" \
            f"&start_date={start_date}"

r = requests.get(query_url)
pprint(r.json())

{'news': [{'author': 'Reuters',
           'category': ['business'],
           'description': 'Digital currency Bitcoin extended its record '
                          'smashing rally on Saturday, beginning the year with '
                          'a surge over $30,000 for the first time, with ever '
                          'more traders and investors betting that it is on '
                          'its way to becoming a mainstream payment method. '
                          'Read More...',
           'id': '7d1e2c4e-886f-466f-b3f2-77c107dfe3e2',
           'image': 'None',
           'language': 'en',
           'published': '2021-01-02 15:17:04 +0000',
           'title': 'Bitcoin rallies above $30,000 for first time',
           'url': 'https://financialpost.com/pmn/business-pmn/bitcoin-rallies-above-30000-for-first-time'}],
 'page': 1,
 'status': 'ok'}


In [69]:
# https://currentsapi.services/en/docs/search
apikey = 'Zm0T2sOpC2G9LInaOGnXBdDwU29-U__Z_rXixLa-dJQdaUb2'
language = 'en'  # Mapping from Language to Code, e.g.: "English": "en"
keywords = "nasdaq"
domain = "ft.com"  # website primary domain name (without www or blog prefix)
t = "2"  # 1 for news, 2 for article and 3 for discussion content
start_date = "2021-01-01T00:00"  # YYYY-MM-DDTHH:MM:SS+00:00
query_url = f"https://api.currentsapi.services/v1/search?" \
            f"apiKey={apikey}" \
            f"&language={language}" \
            f"&type={t}" \
            f"&domain={domain}" \
            f"&keywords={keywords}" \
            f"&start_date={start_date}"

r = requests.get(query_url)
pprint(r.json())

{'news': [], 'page': 1, 'status': 'ok'}


In [65]:
query_url

'https://api.currentsapi.services/v1/search?apiKey=Zm0T2sOpC2G9LInaOGnXBdDwU29-U__Z_rXixLa-dJQdaUb2&language=en&type=2&keywords="LCH SA"&start_date=2021-01-01T00:00'

## Bing Websearch

In [194]:
bing_client = get_newssearch_client()
    
# Fetch News
results = bing_client.news.search(query=searchword,
        #market="en-uk", NOT WORKING 
        freshness="Week",
        #setLang:"en", NOT WORKING
        sort_by="Date"
    )

In [228]:
urls = [article.get('url') for article in results.as_dict().get('value')]
title_list = [article.get('name') for article in results.as_dict().get('value')]
summary_list = [article.get('description') for article in results.as_dict().get('value')]
date_list = [article.get('date_published') for article in results.as_dict().get('value')]
date_list = [datetime.strptime(date[:10], '%Y-%m-%d').strftime('%Y/%m/%d') for date in date_list]
media_list = [article.get('provider')[0].get('name') for article in results.as_dict().get('value')]

### Final Function

In [126]:
def get_newssearch_client():
    """"""
    # Import Libraries
    from azure.cognitiveservices.search.newssearch import NewsSearchClient
    from msrest.authentication import CognitiveServicesCredentials
    
    # Credentials
    subscription_key = "5521444b482f4dd6a800e85fdfb1c636"
    ENDPOINT = "https://api.bing.microsoft.com/v7.0/search?q=microsoft+devices"
    
    # Client
    bing_client = NewsSearchClient(
            endpoint=ENDPOINT,
            credentials=CognitiveServicesCredentials(subscription_key)
        )
    
    return bing_client

def add_bing_news(dff, searchword):
    """"""
    # Authenticate connection to azure bing news search service
    bing_client = get_newssearch_client()
    
    # Fetch News
    news_result = bing_client.news.search(query=searchword,
            market="en-us", #NOT WORKING 
            freshness="Week",
            #setLang:"en", NOT WORKING
            sort_by="Date"
        )
    
    # Add Bing news to df  
    dff =  bing_news_to_df(news_result.as_dict().get("value"), dff)
    
    return dff

In [144]:
# Import Libraries
from azure.cognitiveservices.search.newssearch import NewsSearchClient
from msrest.authentication import CognitiveServicesCredentials

# Credentials
subscription_key = "5521444b482f4dd6a800e85fdfb1c636"
ENDPOINT = "https://api.bing.microsoft.com/v7.0/search?q=microsoft+devices"
ENDPOINT = "https://api.bing.microsoft.com/v7.0/search?"


# Client
bing_client = NewsSearchClient(
        endpoint=ENDPOINT,
        credentials=CognitiveServicesCredentials(subscription_key)
    )

In [145]:
# Fetch News
news_result = bing_client.news.search(query='LCH SA',
        market="en-us", #NOT WORKING 
        freshness="Week",
        #setLang:"en", NOT WORKING
        sort_by="Date"
    )
news_result.as_dict().get("value")

[{'_type': 'NewsArticle',
  'name': 'La Crescent-Hokah hockey slips by Fairmont Area, 6-4',
  'url': 'https://www.fairmontsentinel.com/sports/local-sports/2021/02/13/la-crescent-hokah-hockey-slips-by-fairmont-area-6-4/',
  'image': {'_type': 'ImageObject',
   'thumbnail': {'_type': 'ImageObject',
    'content_url': 'https://www.bing.com/th?id=OVFT.Jsq9Et5Dw6-aKdhtm-QAWS&pid=News',
    'width': 264,
    'height': 500}},
  'description': 'Three second-period goals pushed the Lancers past the Cardinals. La Crescent-Hokah beat Fairmont Area 6-4 in nonconference boys hockey action Friday night in La Crescent. The Cardinals jumped ahead in the first period when Brock Lutterman scored off an assist from Alec Reutzel at the 13:00 juncture on a power-play opportunity,',
  'provider': [{'_type': 'Organization',
    'name': 'Fairmont Sentinel',
    'image': {'_type': 'ImageObject',
     'thumbnail': {'_type': 'ImageObject',
      'content_url': 'https://www.bing.com/th?id=AR_7f62abeff8d00ddc36dcd

In [133]:
news_result.as_dict().get("value")

[]

### Azure API

In [23]:
import requests

In [157]:
# Archive

# https://docs.microsoft.com/en-us/azure/cognitive-services/bing-web-search/quickstarts/client-libraries?pivots=programming-language-python

# # Import required modules.
# from azure.cognitiveservices.search.websearch import WebSearchClient
# from azure.cognitiveservices.search.websearch.models import SafeSearch
# from msrest.authentication import CognitiveServicesCredentials

- https://docs.microsoft.com/en-us/azure/cognitive-services/bing-web-search/quickstarts/python

In [ ]:
# # Working Copy
# subscription_key = "5521444b482f4dd6a800e85fdfb1c636"
# assert subscription_key

# search_url = "https://api.bing.microsoft.com/v7.0/search"

# search_term = "EUREX"

# # Search
# headers = {"Ocp-Apim-Subscription-Key": subscription_key}
# params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
# response = requests.get(search_url, headers=headers, params=params)
# response.raise_for_status()
# search_results = response.json()
# search_results

In [138]:
subscription_key = "5521444b482f4dd6a800e85fdfb1c636"
assert subscription_key

search_url = "https://api.bing.microsoft.com/v7.0/news/search"

search_term = "EUREX"
search_term = "LCH SA"

In [143]:
# Search
headers = {"Ocp-Apim-Subscription-Key": subscription_key}
params = {"q": search_term, "textDecorations": True, 
          #"textFormat": "HTML",
          #"freshness":"week",
          "responseFilter":"News",
          "setLang":"en"
         }
response = requests.get(search_url, headers=headers, params=params)
response.raise_for_status()
search_results = response.json()
search_results

{'_type': 'News',
 'readLink': 'https://api.bing.microsoft.com/api/v7/news/search?q=LCH+SA&setLang=en',
 'queryContext': {'originalQuery': 'LCH SA', 'adultIntent': False},
 'totalEstimatedMatches': 21,
 'sort': [{'name': 'Beste overeenkomst',
   'id': 'relevance',
   'isSelected': True,
   'url': 'https://api.bing.microsoft.com/api/v7/news/search?q=LCH+SA&setLang=en&setLang=en'},
  {'name': 'Meest recent',
   'id': 'date',
   'isSelected': False,
   'url': 'https://api.bing.microsoft.com/api/v7/news/search?q=LCH+SA&setLang=en&sortby=date&setLang=en'}],
 'value': [{'name': "Jérémy Lauzon a toujours eu l'ADN des Bruins",
   'url': 'https://www.tvasports.ca/2021/02/16/jeremy-lauzon-a-toujours-eu-ladn-des-bruins',
   'description': "C'est le cas de le dire, les dernières semaines ont été plutôt mouvementées pour André Ruel. D’abord, l’ancien entraîneur-chef dans la Ligue de hockey junior majeur du Québec devenu agent de joueurs a",
   'provider': [{'_type': 'Organization', 'name': 'TVA Spo

In [26]:
search_results.get("value")[0]

{'name': 'Matthew Stafford avait <b>sa</b> liste d’équipes',
 'url': 'https://www.tvasports.ca/2021/02/14/matthew-stafford-avait-sa-liste-dequipes-1',
 'image': {'thumbnail': {'contentUrl': 'https://www.bing.com/th?id=OVFT.9robraYI5-1eVXTjr3ytmC&pid=News',
   'width': 700,
   'height': 393}},
 'description': 'Le quart-arrière Matthew Stafford n’a pas confirmé qu’il ne voulait rien savoir d’être échangé aux Patriots de la Nouvelle-Angleterre, mais c’est comme s’il avait m',
 'about': [{'readLink': 'https://api.bing.microsoft.com/api/v7/entities/fb242e14-182b-68a7-12da-68d8456f512f?setLang=en',
   'name': 'Matthew Stafford'}],
 'provider': [{'_type': 'Organization', 'name': 'TVA Sports'}],
 'datePublished': '2021-02-14T20:45:00.0000000Z'}

In [141]:
# Extract articles 
from newspaper import Article
for result in search_results.get("value")[1:]:
    print(result.get("name"))
    print(result.get("url"))
    print(result.get("datePublished"))
    print()
    # Download Article
    article = Article(result.get("url"))
    article.download()
    article.parse()
    article.nlp()

    title = article.title
    text = article.text
    summary = article.summary
    date = article.publish_date

Legazpi City Hospital placed on lockdown due to Covid-19
https://www.pna.gov.ph/articles/1129633
2021-02-04T10:09:00.0000000Z

La valeur de Marc-André Fleury est à la hausse
https://www.tvasports.ca/2021/02/15/la-valeur-de-marc-andre-fleury-est-a-la-hausse
2021-02-15T23:06:00.0000000Z

La Crescent-Hokah hockey slips by Fairmont Area, 6-4
https://www.fairmontsentinel.com/sports/local-sports/2021/02/13/la-crescent-hokah-hockey-slips-by-fairmont-area-6-4/
2021-02-13T00:00:00.0000000Z



ArticleException: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.fairmontsentinel.com/sports/local-sports/2021/02/13/la-crescent-hokah-hockey-slips-by-fairmont-area-6-4/ on URL https://www.fairmontsentinel.com/sports/local-sports/2021/02/13/la-crescent-hokah-hockey-slips-by-fairmont-area-6-4/

In [28]:
# Download Article
url = "https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/"
url = "https://www.modernreaders.com/news/2021/02/09/critical-contrast-deutsche-borse-dboey-and-its-rivals.html"
article = Article(url)
article.download()
article.parse()
article.nlp()

title = article.title
text = article.text
summary = article.summary
date = article.publish_date

### Azure Bing News API 

In [224]:
from azure.cognitiveservices.search.newssearch import NewsSearchClient
from msrest.authentication import CognitiveServicesCredentials

In [261]:
# Add your Bing Search V7 subscription key to your environment variables.
SUBSCRIPTION_KEY = "5521444b482f4dd6a800e85fdfb1c636"
# Note: your endpoint should not include the /bing/v7.0 suffix
ENDPOINT = "https://api.bing.microsoft.com/v7.0/search"
#ENDPOINT = "https://api.bing.microsoft.com/"
#ENDPOINT = "https://api.bing.microsoft.com/bing/v7.0"
ENDPOINT = "https://api.bing.microsoft.com/v7.0/search?q=microsoft+devices"

In [278]:
client = NewsSearchClient(
            endpoint=ENDPOINT,
            credentials=CognitiveServicesCredentials(subscription_key)
        )

2021-02-11 09:28:23 [msrest.universal_http.requests] DEBUG: Configuring retry: max_retries=3, backoff_factor=0.8, max_backoff=90


In [282]:
news_result = client.news.search(query="EUREX",
            #market="en-uk",
            freshness="Week",
            #setLang:"en",
            sort_by="Date"
        )

if news_result.value:
    first_news_result = news_result.value[0]
    print("Total estimated matches value: {}".format(
                                        news_result.total_estimated_matches))
    print("News result count: {}".format(len(news_result.value)))
    print("First news name: {}".format(first_news_result.name))
    print("First news url: {}".format(first_news_result.url))
    print("First news description: {}".format(
                                        first_news_result.description))
    print("First published time: {}".format(
                                        first_news_result.date_published))
    print("First news provider: {}".format(
                                        first_news_result.provider[0].name))
else:
    print("Didn't see any news result data..")

2021-02-11 09:41:32 [msrest.universal_http] DEBUG: Configuring redirects: allow=True, max=30
2021-02-11 09:41:32 [msrest.universal_http] DEBUG: Configuring request: timeout=100, verify=True, cert=None
2021-02-11 09:41:32 [msrest.universal_http] DEBUG: Configuring proxies: ''
2021-02-11 09:41:32 [msrest.universal_http] DEBUG: Evaluate proxies against ENV settings: True


Total estimated matches value: 8
News result count: 8
First news name: Clearstream Reports €15.4 Trillion in Assets Under Custody
First news url: https://www.financemagnates.com/institutional-forex/exchanges/clearstream-reports-e15-4-trillion-in-assets-under-custody/
First news description: During the market turbulence, Deutsche Boerse’s Xetra and Eurex platforms, as well as other platforms that cater to retail investors, have experienced difficulties, resulting in clients getting kept out of the market during sensitive times.
First published time: 2021-02-10T17:25:00.0000000Z
First news provider: Finance Magnates


In [230]:
# Functions
def news_search(searchword="Azure", subscription_key=SUBSCRIPTION_KEY):
    """NewsSearch.
    This will search news for (Quantum  Computing) with market and count parameters then verify number of results and print out totalEstimatedMatches, name, url, description, published time and name of provider of the first news result
    """
    client = NewsSearchClient(
        endpoint=ENDPOINT,
        credentials=CognitiveServicesCredentials(subscription_key)
    )

    try:
        news_result = client.news.search(
            query=searchword, market="en-us", count=10)
        print("Search news for query "+searchword+" with market and count")

        if news_result.value:
            first_news_result = news_result.value[0]
            print("Total estimated matches value: {}".format(
                news_result.total_estimated_matches))
            print("News result count: {}".format(len(news_result.value)))
            print("First news name: {}".format(first_news_result.name))
            print("First news url: {}".format(first_news_result.url))
            print("First news description: {}".format(
                first_news_result.description))
            print("First published time: {}".format(
                first_news_result.date_published))
            print("First news provider: {}".format(
                first_news_result.provider[0].name))
        else:
            print("Didn't see any news result data..")

    except Exception as err:
        print("Encountered exception. {}".format(err))

def news_search_with_filtering(searchword="Azure", subscription_key=SUBSCRIPTION_KEY):
    """NewsSearchWithFilters.
    This will search most recent news for searchword
    with freshness and sortBy parameters 
    then verify number of results 
    and print out totalEstimatedMatches, name, url, description, 
    published time and name of provider of the first news result.
    """
    client = NewsSearchClient(
        endpoint="https://api.cognitive.microsoft.com",
        credentials=CognitiveServicesCredentials(subscription_key)
    )

    try:
        news_result = client.news.search(
            query=searchword,
            market="en-us",
            freshness="Week",
            sort_by="Date"
        )
        print("Search most recent news for query \"Artificial Intelligence\" with freshness and sortBy")

        if news_result.value:
            first_news_result = news_result.value[0]
            print("News result count: {}".format(len(news_result.value)))
            print("First news name: {}".format(first_news_result.name))
            print("First news url: {}".format(first_news_result.url))
            print("First news description: {}".format(
                first_news_result.description))
            print("First published time: {}".format(
                first_news_result.date_published))
            print("First news provider: {}".format(
                first_news_result.provider[0].name))
        else:
            print("Didn't see any news result data..")

    except Exception as err:
        print("Encountered exception. {}".format(err))

def news_category(category="Entertainment_MovieAndTV", subscription_key=SUBSCRIPTION_KEY):
    """NewsCategory.
    This will search category news for movie and TV entertainment with safe search then verify number of results and print out category, name, url, description, published time and name of provider of the first news result.
    """
    client = NewsSearchClient(
        endpoint="https://api.cognitive.microsoft.com",
        credentials=CognitiveServicesCredentials(subscription_key)
    )

    try:
        news_result = client.news.category(
            category=category,
            market="en-us",
            safe_search="strict"
        )
        print("Search category news for movie and TV entertainment with safe search")

        if news_result.value:
            first_news_result = news_result.value[0]
            print("News result count: {}".format(len(news_result.value)))
            print("First news category: {}".format(first_news_result.category))
            print("First news name: {}".format(first_news_result.name))
            print("First news url: {}".format(first_news_result.url))
            print("First news description: {}".format(
                first_news_result.description))
            print("First published time: {}".format(
                first_news_result.date_published))
            print("First news provider: {}".format(
                first_news_result.provider[0].name))
        else:
            print("Didn't see any news result data..")

    except Exception as err:
        print("Encountered exception. {}".format(err))

def news_trending(subscription_key):
    """NewsTrending.
    This will search news trending topics in Bing then verify number of results and print out name, text of query, webSearchUrl, newsSearchUrl and image Url of the first news result.
    """
    client = NewsSearchClient(
        endpoint="https://api.cognitive.microsoft.com",
        credentials=CognitiveServicesCredentials(subscription_key)
    )

    try:
        trending_topics = client.news.trending(market="en-us")
        print("Search news trending topics in Bing")

        if trending_topics.value:
            first_topic = trending_topics.value[0]
            print("News result count: {}".format(len(trending_topics.value)))
            print("First topic name: {}".format(first_topic.name))
            print("First topic query: {}".format(first_topic.query.text))
            print("First topic image url: {}".format(first_topic.image.url))
            print("First topic webSearchUrl: {}".format(
                first_topic.web_search_url))
            print("First topic newsSearchUrl: {}".format(
                first_topic.news_search_url))
        else:
            print("Didn't see any topics result data..")

    except Exception as err:
        print("Encountered exception. {}".format(err))

In [291]:
news_result.as_dict().get("value")[0]

{'_type': 'NewsArticle',
 'name': 'Clearstream Reports €15.4 Trillion in Assets Under Custody',
 'url': 'https://www.financemagnates.com/institutional-forex/exchanges/clearstream-reports-e15-4-trillion-in-assets-under-custody/',
 'image': {'_type': 'ImageObject',
  'thumbnail': {'_type': 'ImageObject',
   'content_url': 'https://www.bing.com/th?id=OVFT.QV2qeWaojG1Mxo1zhu6qTS&pid=News',
   'width': 700,
   'height': 318}},
 'description': 'During the market turbulence, Deutsche Boerse’s Xetra and Eurex platforms, as well as other platforms that cater to retail investors, have experienced difficulties, resulting in clients getting kept out of the market during sensitive times.',
 'provider': [{'_type': 'Organization', 'name': 'Finance Magnates'}],
 'date_published': '2021-02-10T17:25:00.0000000Z'}

In [305]:
#result.get("description")
# result.get("provider")[0].get("name")
article = Article(result.get("url"))
article.download()
article.parse()
article.nlp()
article.

AttributeError: 'Article' object has no attribute 'language'

In [296]:
from newspaper import Article
for result in news_result.as_dict().get("value"):
    print(result.get("name"))
    print(result.get("url"))
    print(result.get("date_published"))
    try:
        # Download Article
        article = Article(result.get("url"))
        article.download()
        article.parse()
        article.nlp()

        title = article.title
        text = article.text
        summary = article.summary
        date = article.publish_date
    except:
        print("Not possible for this Url")
    print()

Clearstream Reports €15.4 Trillion in Assets Under Custody
https://www.financemagnates.com/institutional-forex/exchanges/clearstream-reports-e15-4-trillion-in-assets-under-custody/
2021-02-10T17:25:00.0000000Z

Critical Contrast: Deutsche Börse (DBOEY) and Its Rivals
https://www.modernreaders.com/news/2021/02/09/critical-contrast-deutsche-borse-dboey-and-its-rivals.html
2021-02-09T17:39:00.0000000Z

Vienna Stock Exchange: New Calculation Technology Increases Quality
https://www.vindobona.org/article/vienna-stock-exchange-new-calculation-technology-increases-quality
2021-02-09T16:01:00.0000000Z


2021-02-11 10:29:59 [newspaper.article] DEBUG: Download failed on URL https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/ because of 403 Client Error: Forbidden for url: https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/



Eurex traded contracts fall 13%MoM in January to 129.5 million
https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/
2021-02-08T10:02:00.0000000Z
Not possible for this Url

Contrasting Deutsche Börse (DBOEY) and Its Peers
https://www.modernreaders.com/news/2021/02/08/contrasting-deutsche-borse-dboey-and-its-peers.html
2021-02-08T08:03:00.0000000Z

January 2021 monthly figures at Eurex
https://www.mds.deutsche-boerse.com/dbg-en/media/press-releases/January-2021-monthly-figures-at-Eurex--2433156
2021-02-05T12:58:00.0000000Z

GFF Summit 2021: two days of pure expertise
https://www.mds.deutsche-boerse.com/dbg-en/products-services/insights-post-trading-en/GFF-Summit-2021-two-days-of-pure-expertise-2434356
2021-02-05T00:00:00.0000000Z

'Handel stroomt weg van Eurex door nieuwe regel'
https://fd.nl/beurs/1372845/handel-stroomt-weg-van-eurex-door-nieuwe-regel
2021-02-04T17:57:00.0000000Z



### Bing News Function

In [294]:
# Input
searchword = 'EUREX'
dff = pd.DataFrame()

In [319]:
def add_bing_news(dff, searchword):
    """"""
    # Import Libraries
    from azure.cognitiveservices.search.newssearch import NewsSearchClient
    from msrest.authentication import CognitiveServicesCredentials
    
    # Credentials
    SUBSCRIPTION_KEY = "5521444b482f4dd6a800e85fdfb1c636"
    ENDPOINT = "https://api.bing.microsoft.com/v7.0/search?q=microsoft+devices"
    
    # Client
    bing_client = NewsSearchClient(
            endpoint=ENDPOINT,
            credentials=CognitiveServicesCredentials(subscription_key)
        )
    
    # Fetch News
    news_result = bing_client.news.search(query=searchword,
            #market="en-uk", NOT WORKING 
            freshness="Week",
            #setLang:"en", NOT WORKING
            sort_by="Date"
        )
    

    # Authenticate connection to azure text analytics service
    ta_client = authenticate_azure_textanalytics_client()
    
    #
    for result in news_result.as_dict().get("value"):
        name = result.get("name")
        url = result.get("url")
        date = result.get("date_published")
        description = result.get("description")
        provider = result.get("provider")[0].get("name")
        
        row = {}
        
        try:
            # Download Article
            article = Article(result.get("url"))
            article.download()
            article.parse()
            article.nlp()

            title = article.title
            text = article.text
            summary = article.summary
            date = article.publish_date
            
            # Add Articles to Df 
            row = {'API':provider,
                        'Link':url, 
                        'Date':date, 
                        'Title':title, 
                        'Article':text, 
                        'Summary':summary, 
                        'Topics':'',
                        'Found_Keywords':'', 
                        'Overall_Sentiment':'', 
                        'Verified':'', 
                        'Title_Sentiment':getAzureSentiment(title, ta_client)[0],
                        'Title_Positv':getAzureSentiment(title, ta_client)[1], 
                        'Title_Neutral':getAzureSentiment(title, ta_client)[2],
                        'Title_Negativ':getAzureSentiment(title, ta_client)[3], 
                        'Article_Sentiment':getAzureSentiment(text, ta_client)[0],
                        'Article_Positv':getAzureSentiment(text, ta_client)[1], 
                        'Article_Neutral':getAzureSentiment(text, ta_client)[2], 
                        'Article_Negativ':getAzureSentiment(text, ta_client)[3]}
        except:
            print('Article could not be downloaded. ' + url)
            row = {'API':provider,
                        'Link':url, 
                        'Date':date, 
                        'Title':name, 
                        'Article':description, 
                        'Summary':'', 
                        'Topics':'',
                        'Found_Keywords':'', 
                        'Overall_Sentiment':'', 
                        'Verified':'', 
                        'Title_Sentiment':getAzureSentiment(name, ta_client)[0],
                        'Title_Positv':getAzureSentiment(name, ta_client)[1], 
                        'Title_Neutral':getAzureSentiment(name, ta_client)[2],
                        'Title_Negativ':getAzureSentiment(name, ta_client)[3], 
                        'Article_Sentiment':getAzureSentiment(description, ta_client)[0],
                        'Article_Positv':getAzureSentiment(description, ta_client)[1], 
                        'Article_Neutral':getAzureSentiment(description, ta_client)[2], 
                        'Article_Negativ':getAzureSentiment(description, ta_client)[3]}
            #continue
        
        # Add row to dff
        dff = dff.append(row, ignore_index=True)
        
    return dff 
    
    

In [321]:
x = add_bing_news(df, "EUREX")
x

2021-02-11 14:52:46 [msrest.universal_http.requests] DEBUG: Configuring retry: max_retries=3, backoff_factor=0.8, max_backoff=90
2021-02-11 14:52:46 [msrest.universal_http] DEBUG: Configuring redirects: allow=True, max=30
2021-02-11 14:52:46 [msrest.universal_http] DEBUG: Configuring request: timeout=100, verify=True, cert=None
2021-02-11 14:52:46 [msrest.universal_http] DEBUG: Configuring proxies: ''
2021-02-11 14:52:46 [msrest.universal_http] DEBUG: Evaluate proxies against ENV settings: True


Article could not be downloaded. https://www.vindobona.org/article/vienna-stock-exchange-new-calculation-technology-increases-quality


2021-02-11 14:52:56 [newspaper.article] DEBUG: Download failed on URL https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/ because of 403 Client Error: Forbidden for url: https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/


Article could not be downloaded. https://www.leaprate.com/financial-services/exchanges/eurex-traded-contracts-fall-13mom-in-january-to-129-5-million/


,API,Link,Date,Title,Article,Summary,Topics,Found_Keywords,Overall_Sentiment,Verified,Title_Sentiment,Title_Positv,Title_Neutral,Title_Negativ,Article_Sentiment,Article_Positv,Article_Neutral,Article_Negativ
0,PR Newswire,https://www.prnewswire.com/news-releases/fenic...,None,Fenics GO named OTC Trading Platform of the Year,"LONDON, Feb. 11, 2021 /PRNewswire/ -- Fenics G...","LONDON, Feb. 11, 2021 /PRNewswire/ -- Fenics G...",,,,,neutral,0.01,0.98,0.01,mixed,0.44,0.11,0.45
1,Finance Magnates,https://www.financemagnates.com/institutional-...,2021-02-10 17:10:27+00:00,Clearstream Reports €15.4 Trillion in Assets U...,Settlement volumes at Deutsche Boerse AG’s Cle...,Settlement volumes at Deutsche Boerse AG’s Cle...,,,,,neutral,0.01,0.89,0.10,mixed,0.30,0.10,0.60
2,modernreaders.com,https://www.modernreaders.com/news/2021/02/09/...,2021-02-09 00:00:00,Critical Contrast: Deutsche Börse (DBOEY) and ...,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,ProfitabilityGet Deutsche Börse alerts:This ta...,,,,,neutral,0.07,0.58,0.35,mixed,0.33,0.25,0.42
3,vindobona,https://www.vindobona.org/article/vienna-stock...,None,Vienna Stock Exchange: New Calculation Technol...,The Vienna Stock Exchange (Wiener Börse AG) ha...,,,,,,neutral,0.01,0.99,0.00,positive,0.50,0.48,0.02
4,Leaprate,https://www.leaprate.com/financial-services/ex...,2021-02-08T10:02:00.0000000Z,Eurex traded contracts fall 13%MoM in January ...,Eurex Repo posted drops 25.2% MoM rise in GC P...,,,,,,neutral,0.00,0.96,0.04,neutral,0.01,0.87,0.12
5,modernreaders.com,https://www.modernreaders.com/news/2021/02/08/...,2021-02-08 00:00:00,Contrasting Deutsche Börse (DBOEY) and Its Peers,Deutsche Börse (OTCMKTS: DBOEY) is one of 23 p...,DividendsGet Deutsche Börse alerts:Deutsche Bö...,,,,,neutral,0.03,0.95,0.02,mixed,0.30,0.21,0.49
6,Deutsche Börse,https://www.mds.deutsche-boerse.com/dbg-en/med...,None,January 2021 monthly figures at Eurex,The number of traded financial derivatives con...,The number of traded financial derivatives con...,,,,,neutral,0.00,0.99,0.01,neutral,0.01,0.92,0.07
7,Deutsche Börse,https://www.mds.deutsche-boerse.com/dbg-en/pro...,None,GFF Summit 2021: two days of pure expertise,Following the theme “An open eco- system for a...,Following the theme “An open eco- system for a...,,,,,positive,0.75,0.25,0.00,neutral,0.05,0.91,0.04
8,Het Financieele Dagblad,https://fd.nl/beurs/1372845/handel-stroomt-weg...,None,'Handel stroomt weg van Eurex door nieuwe regel',Heeft u een FD.nl account of bent u abonnee va...,Heeft u een FD.nl account of bent u abonnee va...,,,,,neutral,0.03,0.95,0.02,neutral,0.01,0.97,0.02


__Test__

In [12]:
# Import Libraries
from azure.cognitiveservices.search.newssearch import NewsSearchClient
from msrest.authentication import CognitiveServicesCredentials

# Credentials
subscription_key = "5521444b482f4dd6a800e85fdfb1c636"
ENDPOINT = "https://api.bing.microsoft.com/v7.0/search?q=microsoft+devices"

# Client
bing_client = NewsSearchClient(
        endpoint=ENDPOINT,
        credentials=CognitiveServicesCredentials(subscription_key)
    )

# Fetch News
news_result = bing_client.news.search(query=searchword,
        #market="en-uk", NOT WORKING 
        freshness="Week",
        #setLang:"en", NOT WORKING
        sort_by="Date"
    )


# Authenticate connection to azure text analytics service
ta_client = authenticate_azure_textanalytics_client()

2021-02-11 15:05:37 [msrest.universal_http.requests] DEBUG: Configuring retry: max_retries=3, backoff_factor=0.8, max_backoff=90
2021-02-11 15:05:37 [msrest.universal_http] DEBUG: Configuring redirects: allow=True, max=30
2021-02-11 15:05:37 [msrest.universal_http] DEBUG: Configuring request: timeout=100, verify=True, cert=None
2021-02-11 15:05:37 [msrest.universal_http] DEBUG: Configuring proxies: ''
2021-02-11 15:05:37 [msrest.universal_http] DEBUG: Evaluate proxies against ENV settings: True


In [13]:
result = news_result.as_dict().get("value")[0]

name = result.get("name")
url = result.get("url")
date = result.get("date_published")[:10]
description = result.get("description")
provider = result.get("provider")[0].get("name")

row = {}

In [14]:
article = Article(result.get("url"))
article.download()
article.parse()
article.nlp()

In [15]:
title = article.title
text = article.text
summary = article.summary
date = article.publish_date[:10]

TypeError: 'NoneType' object is not subscriptable

In [17]:
article.publish_date

In [ ]:

#
for result in news_result.as_dict().get("value"):
    name = result.get("name")
    url = result.get("url")
    date = result.get("date_published")[:10]
    description = result.get("description")
    provider = result.get("provider")[0].get("name")

    row = {}

    try:
        # Download Article
        article = Article(result.get("url"))
        article.download()
        article.parse()
        article.nlp()

        title = article.title
        text = article.text
        summary = article.summary
        date = article.publish_date[:10]

        # Add Articles to Df 
        row = {'API':'Bing',
                    'Link':url, 
                    'Date':date, 
                    'Title':title, 
                    'Article':text, 
                    'Summary':summary, 
                    'Topics':'',
                    'Found_Keywords':'', 
                    'Overall_Sentiment':'', 
                    'Verified':'', 
                    'Title_Sentiment':getAzureSentiment(title, ta_client)[0],
                    'Title_Positv':getAzureSentiment(title, ta_client)[1], 
                    'Title_Neutral':getAzureSentiment(title, ta_client)[2],
                    'Title_Negativ':getAzureSentiment(title, ta_client)[3], 
                    'Article_Sentiment':getAzureSentiment(text, ta_client)[0],
                    'Article_Positv':getAzureSentiment(text, ta_client)[1], 
                    'Article_Neutral':getAzureSentiment(text, ta_client)[2], 
                    'Article_Negativ':getAzureSentiment(text, ta_client)[3]}
    except:
        #print('Article could not be downloaded. ' + url)
        row = {'API':'Bing',
                    'Link':url, 
                    'Date':date, 
                    'Title':name, 
                    'Article':description, 
                    'Summary':'xxx', 
                    'Topics':'',
                    'Found_Keywords':'', 
                    'Overall_Sentiment':'', 
                    'Verified':'', 
                    'Title_Sentiment':getAzureSentiment(name, ta_client)[0],
                    'Title_Positv':getAzureSentiment(name, ta_client)[1], 
                    'Title_Neutral':getAzureSentiment(name, ta_client)[2],
                    'Title_Negativ':getAzureSentiment(name, ta_client)[3], 
                    'Article_Sentiment':getAzureSentiment(description, ta_client)[0],
                    'Article_Positv':getAzureSentiment(description, ta_client)[1], 
                    'Article_Neutral':getAzureSentiment(description, ta_client)[2], 
                    'Article_Negativ':getAzureSentiment(description, ta_client)[3]}
        #continue

    # Add row to dff
    dff = dff.append(row, ignore_index=True)

## Twitter

In [74]:
#Import
import tweepy

# Consumer keys and access tokens, used for OAuth
consumer_key = api_key
consumer_secret = api_secret_key

# OAuth process, using the keys and tokens
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Creation of the actual interface, using authentication
api = tweepy.API(auth)

In [89]:
def clearnUpTweet(txt):
    """"""
    txt = re.sub(r'@[A-Za-z0-9_]+', '', txt) # Remove mentions # [A-Za-z0-9_]+
    txt = re.sub(r'#', ' ', txt) # Remove hashtags
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', txt) # Remove urls
    txt = re.sub(r'&gt;', ' is greater than ', txt) # 
    txt = re.sub(r'&gt;', ' is smaller than ', txt) # 
    txt = re.sub(r'\n', ' ', txt) #
    txt = re.sub(r'w/', 'with', txt) # 
    return txt

In [111]:
searchword 

'EUREX'

In [112]:

# Fetch Tweets 
searchterm = searchword + ' -filter:retweets' # Exclude Retweets
tweets = api.search(q=searchterm, lang="en" ) #  , result_type='recent'



# Tweets Data
texts = [clearnUpTweet(tweet.text) for tweet in tweets]
dates = [tweet.created_at.date() for tweet in tweets]
users = [tweet.user.name for tweet in tweets] # Bug: This is not the actual user that posted it
tweets_data = [texts, dates, users]

tweets_data

[['● 17-February-2021 ● Day Trading (Malaysia) ● Timeframe: M30  DAX (EUREX) (H1) ',
  '    Eurex, CME or HKEX?',
  '● 16-February-2021 ● Night Trading (Malaysia) ● Timeframe: M30  DAX [EUREX] ',
  'nice, finally we have Eurex :)  sierrachart ',
  'The 2022 decommissioning of the Swiss  LIBOR rate approaches, and  SARON has been proposed as alternative… ',
  'New EUREX datafeed in v2230  with full  marketdepth and MBO support … ',
  'From markets &amp; regulation to  CollateralManagement to  ResponsibleInvesting and  technology &amp;  innovation – our… ',
  '● 16-February-2021 ● Day Trading (Malaysia) ● Timeframe: M30   Dax (Eurex) (H1) Intraday: intraday support around 1… ',
  'Clearstream and Eurex target enhanced repo netting capabilities   trading',
  '  Have you guys stopped allowing Eurex trading now?',
  '.  hits ESG derivatives daily trading record\u202f  ',
  'The  FixedIncome markets had a relatively strong start to the year. Find a market briefing by our Lee Bartholomew,… ',

In [104]:
len([tweet for tweet in tweets])

9

# Archive

In [154]:
# FUNCTIONS
# # FUNCTIONS

# # Message and DF 
# def get_init_message(searchword):
#     """"""
#     # Date 
#     date = datetime.today().strftime('%Y-%m-%d')
#     time = datetime.today().strftime('%H:%M')
#     message = date+' '+time +'\n'
    
#     message += 'WebScraper: ' + searchword+ '\n\n'
    
#     return message 
# def get_init_df():
#     """"""
#     return pd.DataFrame(columns=columns)
# def save_csv(dff, searchword):
#     """"""
#     date = datetime.today().strftime('%Y%m%d')
#     path = root_path +  searchword+'_'+ date + file_format
#     dff.to_csv(path, index=False)

# # Azure     
# def authenticate_azure_textanalytics_client():
#     ta_credential = AzureKeyCredential(cog_key)
#     text_analytics_client = TextAnalyticsClient(
#             endpoint=cog_endpoint, 
#             credential=ta_credential)
#     return text_analytics_client
# def getAzureSentiment(text, client):
#     """"""
#     documents = [text[:5120]] # limit to 5120 characters. This is the azure data limit for text analytics
#     response = client.analyze_sentiment(documents = documents)[0]
#     document_sentiment = response.sentiment
#     score_positive = response.confidence_scores.positive
#     score_neutral = response.confidence_scores.neutral
#     score_negative = response.confidence_scores.negative
#     return document_sentiment, score_positive, score_neutral, score_negative
# def get_newssearch_client():
#     """"""
#     # Import Libraries
#     from azure.cognitiveservices.search.newssearch import NewsSearchClient
#     from msrest.authentication import CognitiveServicesCredentials
    
#     # Credentials
#     subscription_key = "5521444b482f4dd6a800e85fdfb1c636"
#     ENDPOINT = "https://api.bing.microsoft.com/v7.0/search?q=microsoft+devices"
    
#     # Client
#     bing_client = NewsSearchClient(
#             endpoint=ENDPOINT,
#             credentials=CognitiveServicesCredentials(subscription_key)
#         )
    
#     return bing_client

# # Add news to df
# def articles_to_df(urls, dff, api):
#     # Import News API
#     from newspaper import Article

#     # Authenticate connection to azure text analytics service
#     client = authenticate_azure_textanalytics_client()

#     for url in urls:
#         # Initialize 
#         title, text, summary='','',''
#         date = ''
#         try:
#             # Download Article
#             article = Article(url)
#             article.download()
#             article.parse()
#             article.nlp()

#             title = article.title
#             text = article.text
#             summary = article.summary
#             date = article.publish_date
#         except:
#             print('Article could not be downloaded. ' + url)
#             #continue

#         try:
#             # Call Sentiment Analizer
#             title_sentiment, title_pos, title_neu, title_neg = getAzureSentiment(title, client)
#             text_sentiment, text_pos, text_neu, text_neg = getAzureSentiment(text, client)
#         except:
#             print('Azure Sentiment could not be called. ' + url)

#         dff = dff.append({'API':api,
#                         'Link':url, 
#                         'Date':date, 
#                         'Title':title, 
#                         'Article':text, 
#                         'Summary':summary, 
#                         'Title_Sentiment':title_sentiment,
#                         'Title_Positv':title_pos, 
#                         'Title_Neutral':title_neu, 
#                         'Title_Negativ':title_neg, 
#                         'Article_Sentiment':text_sentiment,
#                         'Article_Positv':text_pos, 
#                         'Article_Neutral':text_neu, 
#                         'Article_Negativ':text_neg}, 
#                        ignore_index=True) 
    
#     return dff
# def tweets_to_df(dff, tweets_data, searchword): # Specific to LCH SA 
#     """"""
    

#     # Authenticate connection to azure text analytics service
#     client = authenticate_azure_textanalytics_client()

#     for i in range(len(tweets_data[0])):
#         # Initialize 
#         # tweets_data : [texts, dates, users]
#         title = tweets_data[0][i]
#         date = tweets_data[1][i]
#         link = 'User: '+ tweets_data[2][i]
        
#         # Verify that tweet is actually about LCH SA (TO BE IMPROVED)
#         #if ((False==(('LCH' in title)or('lch sa' in title))
#         #     or('Light combat Helic' in title))):
#         if ((False==(searchword.lower() in title.lower()))):
#             continue

#         try:
#             # Call Sentiment Analizer
#             title_sentiment, title_pos, title_neu, title_neg = getAzureSentiment(title, client)
#         except:
#             print('Twitter: Azure Sentiment could not be called.')

#         dff = dff.append({'API':'Twitter',
#                         'Link':link, 
#                         'Date':date, 
#                         'Title':title, 
#                         'Article':'', 
#                         'Summary':'', 
#                         'Title_Sentiment':title_sentiment,
#                         'Title_Positv':title_pos, 
#                         'Title_Neutral':title_neu, 
#                         'Title_Negativ':title_neg, 
#                         'Article_Sentiment':'',
#                         'Article_Positv':'', 
#                         'Article_Neutral':'', 
#                         'Article_Negativ':''}, 
#                        ignore_index=True) 
    
#     return dff
# def ft_news_to_df(ft_dict, dff):
#     """"""
#     ft_url_base = 'https://www.ft.com'
    
#     for i in range(len(ft_dict.get('Link'))):
#         # Get Article 
#         title = ft_dict.get('Title')[i]
#         date = ft_dict.get('Date')[i][:10]
#         link = ft_url_base + ft_dict.get('Link')[i]
        
#         # Authenticate connection to azure text analytics service
#         client = authenticate_azure_textanalytics_client()
#         try:
#           # Call Sentiment Analizer
#           title_sentiment, title_pos, title_neu, title_neg = getAzureSentiment(title, client)
#         except:
#             print('Twitter: Azure Sentiment could not be called.')

#         dff = dff.append({'API':'FinancialTimes',
#                             'Link':link, 
#                             'Date':date, 
#                             'Title':title, 
#                             'Article':'', 
#                             'Summary':'', 
#                             'Title_Sentiment':title_sentiment,
#                             'Title_Positv':title_pos, 
#                             'Title_Neutral':title_neu, 
#                             'Title_Negativ':title_neg, 
#                             'Article_Sentiment':'',
#                             'Article_Positv':'', 
#                             'Article_Neutral':'', 
#                             'Article_Negativ':''}, 
#                            ignore_index=True) 
    
#     return dff
# def bing_news_to_df(results, dff):
#     """"""
    
#     # Authenticate connection to azure text analytics service
#     ta_client = authenticate_azure_textanalytics_client()
    
#     for result in results:
#         name = result.get("name")
#         url = result.get("url")
#         date = result.get("date_published")[:10]
#         description = result.get("description")
#         provider = result.get("provider")[0].get("name")
        
#         row = {}
        
#         try:
#             # Download Article
#             article = Article(result.get("url"))
#             article.download()
#             article.parse()
#             article.nlp()

#             title = article.title
#             text = article.text
#             summary = article.summary
            
#             # Add Articles to Df 
#             row = {'API':'Bing',
#                     'Link':url, 
#                     'Date':date, 
#                     'Title':title, 
#                     'Article':text, 
#                     'Summary':summary, 
#                     'Title_Sentiment':getAzureSentiment(title, ta_client)[0],
#                     'Title_Positv':getAzureSentiment(title, ta_client)[1], 
#                     'Title_Neutral':getAzureSentiment(title, ta_client)[2],
#                     'Title_Negativ':getAzureSentiment(title, ta_client)[3], 
#                     'Article_Sentiment':getAzureSentiment(text, ta_client)[0],
#                     'Article_Positv':getAzureSentiment(text, ta_client)[1], 
#                     'Article_Neutral':getAzureSentiment(text, ta_client)[2], 
#                     'Article_Negativ':getAzureSentiment(text, ta_client)[3]}
#         except:
#             #print('Article could not be downloaded. ' + url)
#             row = {'API':'Bing',
#                     'Link':url, 
#                     'Date':date, 
#                     'Title':name, 
#                     'Article':description, 
#                     'Summary':'', 
#                     'Title_Sentiment':getAzureSentiment(name, ta_client)[0],
#                     'Title_Positv':getAzureSentiment(name, ta_client)[1], 
#                     'Title_Neutral':getAzureSentiment(name, ta_client)[2],
#                     'Title_Negativ':getAzureSentiment(name, ta_client)[3], 
#                     'Article_Sentiment':getAzureSentiment(description, ta_client)[0],
#                     'Article_Positv':getAzureSentiment(description, ta_client)[1], 
#                     'Article_Neutral':getAzureSentiment(description, ta_client)[2], 
#                     'Article_Negativ':getAzureSentiment(description, ta_client)[3]}
#             #continue
        
#         # Add row to dff
#         dff = dff.append(row, ignore_index=True)
        
#     return dff  
    
# # Fetch News via APIs
# def add_google_news(dff, searchword):
#     """"""
#     #Import API
#     from GoogleNews import GoogleNews

#     # Max and Min Date of Search
#     start_dt = (datetime.today() - timedelta(days=search_days)).strftime('%m/%d/%Y')
#     end_dt = datetime.today().strftime('%m/%d/%Y')

#     # Fetch News 
#     googlenews=GoogleNews(start=start_dt,end=end_dt, lang='en') # language english
#     googlenews.search(searchword)
#     result=googlenews.result()

#     # List of news urls 
#     urls = [news.get('link') for news in result]
    
#     dff = articles_to_df(urls, dff, api='Google')
    
#     return dff
# def add_scrapy_news(dff, searchword):
#     """"""
#     # Import
#     import scrapy
#     from scrapy.crawler import CrawlerProcess
#     import logging
#     logging.getLogger('scrapy').propagate = False

#     # Create the Spider class
#     class news_Spider(scrapy.Spider):
#         name = "News_Spider"
#         global scrapy_dict
#         global ft_dict

#         # start_requests method
#         def start_requests(self):
#             urls = ['https://www.risk.net/search/articles?query=lch&sort_by=date&date-filter=7+day&topics%5B%5D=249061&organisations%5B%5D=255101',
#                        'https://www.ft.com/stream/3da687f2-11a7-4e28-a18d-17b77aa18460']
#             for url in urls:
#                 yield scrapy.Request(url = url, callback = self.parse_front)

#         # First parsing method
#         def parse_front(self, response):
#             risk_url = 'https://www.risk.net'
#             ft_url = 'https://www.ft.com'
#             if(response.url[:len(risk_url)]==risk_url):
#                 # Extract Link
#                 link = response.xpath('//h5[@class="article-title"]/a/@href')
#                 link_extract = link.extract()
#                 scrapy_dict['risk']=link_extract
#             elif(response.url[:len(ft_url)]==ft_url):
#                 link = response.xpath('//*[@id="stream"]/div[1]/ul/li/div[2]/div/div/div/div[2]/a/@href') 
#                 link_extract = link.extract()
#                 scrapy_dict['ft']= link_extract 
#                 ft_dict['Link'] = link_extract
#                 # Extract Title
#                 title = response.xpath('//*[@id="stream"]/div[1]/ul/li/div[2]/div/div/div/div[2]/a/text()') 
#                 ft_dict['Title'] = title.extract()
#                 # Extract Date
#                 date = response.xpath('//div[@class="stream-card__date"]/time/@datetime') 
#                 ft_dict['Date'] = date.extract()

#     scrapy_dict = dict()
#     ft_dict = dict()

#     # Run the Spider
#     process = CrawlerProcess()
#     process.crawl(news_Spider)
#     process.start()

#     # Build Link List
#     risk_url_base = 'http://www.risk.net'

#     # Add Risk.Net News 
#     urls = [url if (url[:4]=='http') else risk_url_base+url for url in scrapy_dict.get('risk')]
#     dff = articles_to_df(urls, dff, api='Risk.Net')
    
#     # Add FT News 
#     dff = ft_news_to_df(ft_dict, dff)

#     return dff    
# def add_twitter_news(dff, searchword):
#     """"""
#     #Import
#     import tweepy
    
#     # Consumer keys and access tokens, used for OAuth
#     consumer_key = api_key
#     consumer_secret = api_secret_key

#     # OAuth process, using the keys and tokens
#     auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#     auth.set_access_token(access_token, access_token_secret)

#     # Creation of the actual interface, using authentication
#     api = tweepy.API(auth)
    
#     # Fetch Tweets 
#     searchterm = searchword + ' -filter:retweets' # Exclude Retweets
#     tweets = api.search(q=searchterm, lang="en" ) #  , result_type='recent'
    
#     def clearnUpTweet(txt):
#         """"""
#         txt = re.sub(r'@[A-Za-z0-9_]+', '', txt) # Remove mentions # [A-Za-z0-9_]+
#         txt = re.sub(r'#', ' ', txt) # Remove hashtags
#         txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', txt) # Remove urls
#         txt = re.sub(r'&gt;', ' is greater than ', txt) # 
#         txt = re.sub(r'&gt;', ' is smaller than ', txt) # 
#         txt = re.sub(r'\n', ' ', txt) #
#         txt = re.sub(r'w/', 'with', txt) # 
#         return txt
    
#     # Tweets Data
#     texts = [clearnUpTweet(tweet.text) for tweet in tweets]
#     dates = [tweet.created_at.date() for tweet in tweets]
#     users = [tweet.user.name for tweet in tweets] # Bug: This is not the actual user that posted it
#     tweets_data = [texts, dates, users]
    
#     # Add Tweets to DF
#     dff = tweets_to_df(dff, tweets_data, searchword)
    
#     return dff
# def add_bing_news(dff, searchword):
#     """"""
#     # Authenticate connection to azure bing news search service
#     bing_client = get_newssearch_client()
    
#     # Fetch News
#     news_result = bing_client.news.search(query=searchword,
#             #market="en-uk", NOT WORKING 
#             freshness="Week",
#             #setLang:"en", NOT WORKING
#             sort_by="Date"
#         )
    
#     # Add Bing news to df  
#     dff =  bing_news_to_df(news_result.as_dict().get("value"), dff)
    
#     return dff


# #  Additional Features
# def verify_articles(dff, searchword):
#     """"""
    
#     # Delete Articles that are older than a week
    
    
#     # Authenticate Azure text analysis client 
#     ta_client = authenticate_azure_textanalytics_client()
    
#     def language_detection_example(text, client):
#         try:
#             documents = [text]
#             response = client.detect_language(documents = documents)[0]
#             language = response.primary_language.name
#             return language
#         except Exception as err:
#             return 'Unknown'
        
#     verified_col = []
#     for row in dff.itertuples():
#         verified = '' 

#         #Check Language of Article
#         text = row.Article if len(row.Article)>25 else row.Title
#         language = language_detection_example(row.Article, ta_client)
#         if (language != 'English'):
#             verified += 'Detected language is ' + language + '. '

#         # Check if Article is actually about searchword
#         if (searchword.lower() not in row.Title.lower()):
#             verified += searchword + ' does not appear in the title.'

#         # Is verified
#         verified_col.append(verified)

#     dff['Verified'] = verified_col
#     dff['Verified_Flag'] = 'True' if (len(verified_col)<1) else 'False'
    
#     return dff
# def add_keyword_search(dff):
#     """"""
#     # Keyword Search
#     def find_keywords(article):
#         """"""
#         from fuzzysearch import find_near_matches
#         keywords = ['fraud', 'claim', 'incident', 'bankruptcy', 'fine', 'bankrupt',
#                     'Fraud', 'Claim', 'Incident', 'Bankruptcy', 'Fine', 'Bankrupt']
#         words = ''
#         for keyword in keywords:
#             count = len(find_near_matches(keyword, article, max_l_dist=0))
#             if(count>0):
#                 words += str(count)+'x '+keyword+', '
#         if(len(words)>1):
#             words = words[:-2]
#         else:
#             words = '-'
#         return words
    
#     dff['Found_Keywords'] = dff.Article.apply(find_keywords)
#     return dff
# def add_ner_search(dff):
#     """"""
    
#     # Authenticate connection to azure text analytics service
#     text_analytics_client = authenticate_azure_textanalytics_client()
    
#     # NER Search
#     def get_ner_string(article):
#         """"""
#         try:
#             article = article[:5120] # Max length for article # should be split in multisearch
#             document = [article]

#             response = text_analytics_client.recognize_entities(document , language="en")
#             result = [doc for doc in response if not doc.is_error]

#             ner_entities = []
#             ner_categories, ner_scores = [], []

#             for entity in result[0].entities:
#                 if (entity.category in ['Organization']):#,'Skill']):
#                     ner_entities.append(entity.text) 
#                     ner_categories.append(entity.category)
#                     #ner_scores += entity.confidence_score

#             return str(list(set(ner_entities)))[1:-1]
#         except:
#             return '-'
    
#     # Add Ner column 
#     dff['NER'] = dff.Title.apply(get_ner_string)+dff.Article.apply(get_ner_string)
    
#     return dff 
# def add_topic_search(dff):
#     """"""
#     # Authenticate Azure text analysis client 
#     text_analytics_client = authenticate_azure_textanalytics_client()
    
#     def get_article_topics(article):
#         """"""
#         article = article[:5120]
#         document = [article]

#         try:
#             response = text_analytics_client.extract_key_phrases(document)
#             result = [doc for doc in response if not doc.is_error]
#             topics = result[0].key_phrases
#         except: 
#             topics = []
#         topics = str(topics)[1:-1]
#         return topics
    
#     # Add Ner column 
#     dff['Topics'] = dff.Title.apply(get_article_topics)+dff.Article.apply(get_article_topics)
    
#     return dff 

# # Text Message 
# def get_sentiment_message(dff, message):
#     """"""
#     # Overall Sentiment  
#     sentiment_list = [*dff.Title_Sentiment] + [*dff.Article_Sentiment]
#     sentiment = max(set(sentiment_list), key = sentiment_list.count) 
#     message += 'The overall sentiment of the articles was '+sentiment + '.\n\n'
#     # Mean sentiment
#     def str_mean(x):
#         x = np.array([val for val in x if isinstance(val, numbers.Number)])
#         return str(round(x.mean(),2))

#     message += 'Mean Sentiment: Title ('\
#                                 + str_mean(dff.Title_Positv)+', '\
#                                 + str_mean(dff.Title_Neutral)+', '\
#                                 + str_mean(dff.Title_Negativ)+'); '\
#                             +'Article ('+str_mean(dff.Article_Positv)+', '\
#                                         +str_mean(dff.Article_Positv)+', '\
#                                         +str_mean(dff.Article_Negativ)+').+\n\n'
#     # Negativ Articles
#     # Articles with a negativ score >0.33
#     neg_th =  0.33
#     neg_df = dff[(dff.Title_Negativ>neg_th)|(dff.Title_Negativ>neg_th)]
#     neg_count=len(neg_df) 
#     if(neg_count==1):
#         message += 'There is '+str(neg_count) + ' article with a negativ sentiment:\n'
#     elif(neg_count>1):
#         message += 'There are '+str(neg_count) + ' articles with a negativ sentiment:\n'
#     for row in neg_df.itertuples():
#          message += '\t'+row.Title + ' ['+row.Title_Sentiment+' title ('\
#                                             +str(row.Title_Positv)+', '\
#                                             +str(row.Title_Neutral)+', '\
#                                             +str(row.Title_Negativ)+'); '\
#                                         +row.Article_Sentiment+' article ('\
#                                             +str(row.Article_Positv)+', '\
#                                             +str(row.Article_Neutral)+', '\
#                                             +str(row.Article_Negativ)+')]+\n\n'
    
#     return message

In [155]:
# # Main
# def main(searchword):
    
#     # Get Message and DF
#     message = get_init_message(searchword)
#     df = get_init_df()
    
#     # Add News 
#     df = add_google_news(df, searchword) # API can be blocked via Azure
#     #df = add_scrapy_news(df, searchword) # For now only for LCH SA
#     #df = add_bing_news(df, searchword)
#     #df = add_twitter_news(df, searchword)
    
#     # Add specific Articles
#     urls = ['https://www.vanityfair.com/news/2020/06/bankrupt-hertz-is-a-pandemic-zombie',
#             #'https://www.forbes.com/sites/jackkelly/2020/05/23/hertz-files-for-bankruptcy-after-16000-employees-were-let-go-and-ceo-made-over-9-million/'
#             'https://www.nbcnews.com/politics/meet-the-press/hertz-jcpenny-j-crew-join-list-businesses-filing-bankruptcy-n1213961',
#             'https://edition.cnn.com/2020/05/22/business/hertz-bankruptcy/index.html'
#            ]  
#     #df = articles_to_df(urls, df, 'Urls')
    
    
#     # Verification Step: Are news actually about searchword
#     df = verify_articles(df, searchword)
    
#     # Find Keywords like Fraud, etc.
#     df = add_keyword_search(df)
    
#     # Ner and Topic Search 
#     df = add_ner_search(df)
#     df = add_topic_search(df)
    
#     #Get Overall Sentiment
#     message = get_sentiment_message(df, message)
    
#     #Save results 
#     save_csv(df, searchword)
    
#     return message, df

# import logging
# logging.getLogger('scrapy').propagate = False
 
# #searchword='LCH SA'
# searchword='EUREX'
# #searchword='HERTZ'
# message, df = main(searchword)
# # import pprint as pp
# # pp.pprint(message)
# df